In [ ]:
import pandas as pd
import xml.etree.ElementTree as ET
import glob, os
import numpy as np
from comet_ml import Experiment, Optimizer
import pickle
import logging
import sys
from sklearn.utils import class_weight
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix


from matplotlib import pyplot as plt
import json
import re

In [ ]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1' # When cudnn implementation not found, run this
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # Note: when starting kernel, for gpu_available to be true, this needs to be run
# only reserve 1 GPU
os.environ['TFHUB_CACHE_DIR'] = '/home/anasab/tf_cache'


In [ ]:
# confirm TensorFlow sees the GPU
from tensorflow.python.client import device_lib
assert 'GPU' in str(device_lib.list_local_devices())

# # confirm Keras sees the GPU (for TensorFlow 1.X + Keras)
# from keras import backend
# assert len(backend.tensorflow_backend._get_available_gpus()) > 0

# # confirm PyTorch sees the GPU
# from torch import cuda
# assert cuda.is_available()
# assert cuda.device_count() > 0
# print(cuda.get_device_name(cuda.current_device()))

In [ ]:
device_lib.list_local_devices()

In [ ]:
os.environ['TF_KERAS'] = '1'
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, Lambda, BatchNormalization, TimeDistributed, \
    Bidirectional, Input, InputLayer, concatenate, Flatten, RepeatVector, Activation, Multiply, Permute, \
    Conv1D, GlobalMaxPooling1D
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import callbacks, optimizers
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model, Sequence

from keras.utils import np_utils

from nltk.tokenize import RegexpTokenizer, TweetTokenizer
from nltk.corpus import stopwords

import tensorflow as tf


In [ ]:
dataset_type = "depression"
transfer_type = None

In [ ]:
# transfer_type

In [ ]:
logger = logging.getLogger('training')
ch = logging.StreamHandler(sys.stdout)
# create formatter
formatter = logging.Formatter("%(asctime)s;%(levelname)s;%(message)s")
# add formatter to ch
ch.setFormatter(formatter)
# add ch to logger
logger.addHandler(ch)
logger.setLevel(logging.DEBUG)

In [ ]:
import sys
sys.setrecursionlimit(10000)

In [ ]:
# tf.enable_eager_execution()
my_seed = 1234
# tf.set_random_seed(my_seed)

In [ ]:
tf.test.is_gpu_available()

In [ ]:
from keras.backend import manual_variable_initialization 
manual_variable_initialization(True)

# Read data

In [ ]:
def read_subject_writings(subject_file):
    writings = []
    with open(subject_file) as sf:
        contents = sf.read()
        root = ET.fromstring(contents)
        try:
            subject = root.findall('ID')[0].text.strip()
        except Exception:
            print('Cannot extract ID', contents[:500], '\n-------\n')        
        for w in root.iter('WRITING'):
            subject_writings = {'subject': subject}
            for title in w.findall('TITLE'):
                subject_writings['title'] = title.text
            for text in w.findall('TEXT'):
                subject_writings['text'] = text.text
            for date in w.findall('DATE'):
                subject_writings['date'] = date.text
            writings.append(subject_writings)
    return writings

In [ ]:
root_dir = '/home/anasab/' 
# root_dir = '/home/ana/'

### eRisk 2020 T1

In [ ]:
# datadir_T1 = root_dir + '/eRisk/data/eRisk2020_T1_train/eRISK2020_T1_training_data/eRISK2020_training_data/data/'
# labels_file_T1 = root_dir + '/eRisk/data//eRisk2020_T1_train/eRISK2020_T1_training_data/eRISK2020_training_data/golden_truth.txt'

datadirs_T1_2020 = {
    'train': ['./data/'],
    'test': ['./DATA/']
}
datadir_root_T1_2020 = {
    'train': root_dir + '/eRisk/data/eRisk2020_T1_train/eRISK2020_T1_training_data/eRISK2020_training_data/',
    'test': root_dir + '/eRisk/data/2020/T1/'
}
    
labels_files_T1_2020 = {
    'train': ['golden_truth.txt'],
    'test': ['T1_erisk_golden_truth.txt']
}

In [ ]:
def read_texts_2020(datadir_root_T1_2020,
                   datadirs_T1_2020,
                   labels_files_T1_2020,
                   test_suffix='0000',
                    chunked_subsets=None):


    writings = {'train': [], 'test': []}
    writings_df = pd.DataFrame()
    labels_df = pd.DataFrame()
#     for subset in ('train', 'test'):
    for subset in ('test',):
        for subdir in [os.path.join(datadir_root_T1_2020[subset], subp) for subp in datadirs_T1_2020[subset]]:

            for subject_file in os.listdir(subdir):
                writings[subset].extend(read_subject_writings(os.path.join(subdir, subject_file)))
        writings_df_part = pd.DataFrame(writings[subset])
        # add a suffix for users in the test -- the numbers are duplicated with the ones in train
        if subset=='test':
            writings_df_part['subject'] = writings_df_part['subject'].apply(lambda s: s+test_suffix)
            print(subset, writings_df_part.subject)
        writings_df_part['subset'] = subset
        writings_df = pd.concat([writings_df, writings_df_part])
        writings_df.reindex()

        for label_file in labels_files_T1_2020[subset]:
            labels = pd.read_csv(os.path.join(datadir_root_T1_2020[subset], label_file), 
                                 delimiter='\s+', names=['subject', 'label'])
            # add a suffix for users in the test -- the numbers are duplicated with the ones in train
            if subset=='test':
                labels['subject'] = labels['subject'].apply(lambda s: s+test_suffix)
            labels_df = pd.concat([labels_df, labels])
    labels_df = labels_df.drop_duplicates()
    labels_df = labels_df.set_index('subject')

    writings_df = writings_df.drop_duplicates()
    
    writings_df = writings_df.join(labels_df, on='subject')
    
    return writings_df


### eRisk 2019 T1 (Anorexia)

In [ ]:
datadirs_T1_2019 = {
    'train': ['2018 test/', '2018 train/positive_examples/', '2018 train/negative_examples/'],
    'test': ['data/']
}
datadir_root_T1_2019 = {
    'train': root_dir + '/eRisk/data/past/eRisk2019_T1/training data - t1/',
    'test': root_dir + '/eRisk/data/past/eRisk2019_T1/test data - T1/'
}
    
labels_files_T1_2019 = {
    'train': ['2018 train/risk_golden_truth.txt', '2018 test/risk-golden-truth-test.txt'],
    'test': ['T1_erisk_golden_truth.txt']
}

In [ ]:
def read_texts_2019(datadir_root_T1_2019,
                   datadirs_T1_2019,
                   labels_files_T1_2019,
                   test_suffix='0000',
                    chunked_subsets='train'):
    writings = {'train': [], 'test': []}
    writings_df = pd.DataFrame()
    labels_df = pd.DataFrame()

    for subset in ('train', 'test'):
        for subdir in [os.path.join(datadir_root_T1_2019[subset], subp) for subp in datadirs_T1_2019[subset]]:
            if subset in chunked_subsets:
                chunkdirs = [os.path.join(datadir_root_T1_2019[subset], subdir, chunkdir) 
                             for chunkdir in os.listdir(subdir)]
            else:
                chunkdirs = [os.path.join(datadir_root_T1_2019[subset], subdir)]
                
            for chunkdir in chunkdirs:
                print(chunkdir)
                if not os.path.isdir(chunkdir):
                    continue
                for subject_file in os.listdir(chunkdir):
                    writings[subset].extend(read_subject_writings(os.path.join(chunkdir, subject_file)))
        writings_df_part = pd.DataFrame(writings[subset])
        # add a suffix for users in the test -- the numbers are duplicated with the ones in train
        if subset=='test':
            writings_df_part['subject'] = writings_df_part['subject'].apply(lambda s: s+test_suffix)
            print(subset, writings_df_part.subject)
        writings_df_part['subset'] = subset
        writings_df = pd.concat([writings_df, writings_df_part])
        writings_df.reindex()

        for label_file in labels_files_T1_2019[subset]:
            labels = pd.read_csv(os.path.join(datadir_root_T1_2019[subset], label_file), 
                                 delimiter='\s+', names=['subject', 'label'])
            # add a suffix for users in the test -- the numbers are duplicated with the ones in train
            if subset=='test':
                labels['subject'] = labels['subject'].apply(lambda s: s+test_suffix)
            labels_df = pd.concat([labels_df, labels])
    labels_df = labels_df.drop_duplicates()
    labels_df = labels_df.set_index('subject')

    writings_df = writings_df.drop_duplicates()
    
    writings_df = writings_df.join(labels_df, on='subject')
    
    return writings_df

### eRisk 2018 (Depression)

In [ ]:
datadirs_T1_2018 = {
    'train': ['train/positive_examples_anonymous_chunks/', 'train/positive_examples_anonymous_chunks/', 'test/'],
    'test': ['task 1 - depression (test split, train split is 2017 data)/']
}
datadir_root_T1_2018 = {
    'train': root_dir + '/eRisk/data/2017/',
    'test': root_dir + '/eRisk/data/2018/'
}
    
labels_files_T1_2018 = {
    'train': ['train/risk_golden_truth.txt', 'test/test_golden_truth.txt'],
    'test': ['task 1 - depression (test split, train split is 2017 data)/risk-golden-truth-test.txt']
}

## CLPsych


In [ ]:
datadirs_clpsych = {
    'train': [''],
    'test': ['']
}
datadir_root_clpsych = {
    'train': root_dir + '/eRisk/data/clpsych/final_training_data/',
    'test': root_dir + '/eRisk/data/clpsych/final_testing_data/'
}
    
labels_files_clpsych = [root_dir + '/eRisk/data/clpsych/anonymized_user_info_by_chunk.csv']

In [ ]:
def read_subject_data_clpsych(subject_file):
    writings = []
    with open(subject_file, "rt", encoding="utf-8") as sf:
        user = subject_file.split("/")[-1].split(".")[0]
        print(subject_file)

        for line in sf:
            data = json.loads(line)#.encode('utf-16','surrogatepass').decode('utf-16'))
            data['subject'] = user
            writings.append(data)
    return writings

In [ ]:
def read_texts_clpsych(datadir_root_clpsych,
                   datadirs_clpsych,
                   labels_files_T1_2019,
                      label_by=['depression']):
    writings = {'train': [], 'test': []}
    writings_df = pd.DataFrame()
    labels_df = pd.DataFrame()

    for subset in ('test',):#, 'test'):
        for subdir in [os.path.join(datadir_root_clpsych[subset], subp) for subp in datadirs_clpsych[subset]]:
            for subject_file in glob.glob(subdir + "/*.tweets"):
#                 if subject_file.split("/")[-1] != 'sZVVktDN8qqjA.tweets':
#                     continue
                writings[subset].extend(read_subject_data_clpsych(os.path.join(subdir, subject_file)))
        writings_df_part = pd.DataFrame(writings[subset])
        writings_df_part['subset'] = subset
        writings_df = pd.concat([writings_df, writings_df_part])
        writings_df.reindex()

    for label_file in labels_files_clpsych:
        labels = pd.read_csv(label_file, 
                    names=['subject','age','num_tweets','gender','condition','chunk_index'])
        labels['label'] = labels['condition'].apply(lambda c: 1 if c in label_by else 0)
        
        labels_df = pd.concat([labels_df, labels])
        labels_df = labels_df.drop_duplicates()
        labels_df = labels_df.set_index('subject')

        # TODO: this deduplication throws some unicode, surrogates not allowed, exception
#     writings_df = writings_df.drop_duplicates(subset=['id', 'subject', 'subset', 'created_at', 'text'])
    
    writings_df = writings_df.join(labels_df, on='subject')
    writings_df['date'] = writings_df['created_at']
    
    return writings_df

### Symanto

In [ ]:
def read_texts_symanto(tsv_path="/eRisk/data/symanto/clean_dataset_with_timeline_balancedUsers.tsv"):
    label_key = {'REAL_LABEL_IS_DEPRESSED': 1,
             'REAL_LABEL_IS_NON_DEPRESSED': 0}
    prediction_key = {'predicted_as_depressed': 1,
                     'predicted_as_non_depressed': 0,
                     'DEPRESS_STRING_WAS_MENTIONED': -1}
    
    writings_df = pd.read_csv(root_dir + tsv_path, 
                              sep='\t', names=['subject', 'date', 'text', 'prediction_text', 'real_label_text'])

    writings_df = writings_df.dropna()
    writings_df['label'] = writings_df['real_label_text'].apply(lambda l: label_key[l])
    writings_df['prediction'] = writings_df['prediction_text'].apply(lambda l: prediction_key[l])
#     By ommitting -1s we are excluding tweets where "DEPRESSED" was mentioned, and all after
    writings_df = writings_df.drop(writings_df[writings_df['prediction']==-1].index)

    
    return writings_df

## Preprocess text

In [ ]:
# writings_df

In [ ]:
# writings_df = read_texts_2020(datadir_T1, labels_file_T1)
# writings_df = read_texts_2019(datadir_root_T1_2019,
#                    datadirs_T1_2019,
#                    labels_files_T1_2019)
# writings_df_depression = read_texts_2019(datadir_root_T1_2018,
#                    datadirs_T1_2018,
#                    labels_files_T1_2018,
#                              chunked_subsets=['train', 'test'])

if dataset_type == "combined":
    writings_df_selfharm = pickle.load(open('data/writings_df_selfharm_all', 'rb'))
    writings_df_anorexia = pickle.load(open('data/writings_df_anorexia_liwc', 'rb'))
    writings_df_depression = pickle.load(open('data/writings_df_depression_liwc', 'rb'))
    writings_df_selfharm['source'] = 'selfharm'
    writings_df_anorexia['source'] = 'anorexia'
    writings_df_depression['source'] = 'depression'
    writings_df = pd.DataFrame()
    writings_df = pd.concat([writings_df, writings_df_depression])
    writings_df = pd.concat([writings_df, writings_df_selfharm])
    writings_df = pd.concat([writings_df, writings_df_anorexia])
elif dataset_type == "combined_depr":
    writings_df = pd.DataFrame.from_dict(json.load(open('data/writings_df_depression_all.json')))
elif dataset_type == "clpsych":
    writings_df = pd.DataFrame.from_dict(json.load(open('data/writings_df_%s_liwc_affect.json' % dataset_type)))#read_texts_clpsych(datadir_root_clpsych, datadirs_clpsych, labels_files_clpsych)
#     writings_df_test = pd.DataFrame.from_dict(json.load(open('writings_df_%s_test.json' % dataset_type)))#read_texts_clpsych(datadir_root_clpsych, datadirs_clpsych, labels_files_clpsych)
#     writings_df_test = read_texts_clpsych(datadir_root_clpsych, datadirs_clpsych, labels_files_clpsych)
    label_by = ['depression', 'ptsd']
    writings_df.drop(writings_df[writings_df['condition']=='depression'].index, inplace=True)
    writings_df['label'] = writings_df['condition'].apply(lambda c: 1 if c in label_by else 0)
#     writings_df['date'] = writings_df['created_at']
elif dataset_type == "symanto":
    writings_df = read_texts_symanto()
elif dataset_type == 'selfharm':
    writings_df = pickle.load(open('data/writings_df_%s_all' % dataset_type, 'rb'))
elif dataset_type in ["depression", "anorexia", "selfharm"]:
    writings_df = pickle.load(open('data/writings_df_%s_liwc' % dataset_type, 'rb'))
else:
    logger.error("Unknown dataset %s" % dataset_type)

In [ ]:
# writings_df[writings_df['condition']!='control'].head(150)[['label', 'condition', 'date']]

In [ ]:
# writings_df.label.hist()

In [ ]:
len(set(writings_df[writings_df['label']==1].subject))
len(set(writings_df[writings_df['label']==0].subject))
len(set(writings_df[writings_df['subset']=='test'].subject))

In [ ]:
# set(writings_df['source'].values)

In [ ]:
regtokenizer = RegexpTokenizer(r'\w+')
tweet_tokenizer = TweetTokenizer()
sw = stopwords.words("english")

def tokenize(t, tokenizer=regtokenizer):
    return regtokenizer.tokenize(t.lower())

def tokenize_tweets(t, stop=True):
    tokens = tweet_tokenizer.tokenize(t.lower())
    tokens_clean = [re.sub("^#", "", token) for token in tokens]
    tokens_clean = [token for token in tokens_clean 
                            if re.match("^[a-z]*$", token)]
    if not stop:
        tokens_clean = [token for token in tokens
                       if (token not in sw)]
    return tokens_clean
    

In [ ]:
# re.sub("^#", "", "#h#ash")

In [ ]:
def tokenize_fields(writings_df, tokenize_fct=tokenize, columns=['title', 'text']):
    for c in columns:
        writings_df['tokenized_%s' % c] = writings_df['%s' % c].apply(lambda t: tokenize_fct(t) 
                                                                if type(t)==str and t else None)
        writings_df['%s_len' % c] = writings_df['tokenized_%s' % c].apply(lambda t: len(t) 
                                                                    if type(t)==list and t else None)
    return writings_df

In [ ]:
# writings_df = tokenize_fields(writings_df, tokenize_fct=tokenize_tweets, columns=['title'])
# writings_df = tokenize_fields(writings_df, tokenize_fct=tokenize_tweets, columns=['text'])
# writings_df = tokenize_fields(writings_df, tokenize_fct=tokenize, columns=['text'])

In [ ]:
# writings_df.text_len.describe()

In [ ]:
# writings_df.title_len.describe()

In [ ]:
# writings_df.groupby('subject').mean().describe()

In [ ]:
# writings_df.groupby('subject').max().groupby('label').count()

In [ ]:
# print("Average number of posts per user", writings_df.groupby('subject').count().title.mean())
# print("Average number of comments per user", writings_df.groupby('subject').count().text.mean())


In [ ]:
# writings_df.groupby('subject').count().title.describe()

In [ ]:
# writings_df.groupby('subject').count().text.describe()

In [ ]:
# writings_df.tokenized_text.head()

In [ ]:
# writings_dict = writings_df.to_dict()
# json.dump(writings_dict, open("writings_df_clpsych_all.json", "w+"))

In [ ]:
# print("Total examples", writings_df[['subject', 'subset', 'label'
#                                     ]].groupby('subject').min().groupby('subset').count())
# print("Positive examples", writings_df[['subject', 'subset', 'label'
#                                     ]].groupby('subject').min().groupby('subset').sum())

In [ ]:
# writings_df[['label', 'source']].groupby('source').sum()

In [ ]:
# writings_df['text'] = writings_df['text'].fillna("")
# writings_df['title'] = writings_df['title'].fillna("")

# Recurrent NN

## Extract features and encode data

In [ ]:
def save_model_and_params(model, model_path, hyperparams, hyperparams_features):
    model.save_weights(model_path + "_weights.h5", save_format='h5')
#     model.save(model_path + "_model.model")
#     model.save(model_path + "_model.h5")
    with open(model_path + '.hp.json', 'w+') as hpf:
        hpf.write(json.dumps({k:v for (k,v) in hyperparams.items() if k!='optimizer'}))
    with open(model_path + '.hpf.json', 'w+') as hpff:
        hpff.write(json.dumps(hyperparams_features))

In [ ]:
def load_params(model_path):
    with open(model_path + '.hp.json', 'r') as hpf:
        hyperparams = json.loads(hpf.read())
    with open(model_path + '.hpf.json', 'r') as hpff:
        hyperparams_features = json.loads(hpff.read())
    return hyperparams, hyperparams_features

In [ ]:
hyperparams_features = {
    "max_features": 20002,
    # cut texts after this number of words
    # (among top max_features most common words)
    "embedding_dim": 300,
    'vocabulary_path': 'data/all_vocab_clpsych_erisk_stop_20000.pkl',
#     'embeddings_path': root_dir + '/eRisk/embeddings/finetuned_glove_clpsych_erisk_stop_normalized_20000.pkl',
    'embeddings_path': root_dir + '/resources/glove.840B/glove.840B.300d.txt',
    'liwc_words_cached': 'data/liwc_categories_for_vocabulary_erisk_clpsych_stop_20K.pkl',
    "user_level": True, # deprecated
    "transfer": transfer_type,
#     "pretrained_model_path": 'models/lstm_clpsych_hierarchical64_ptsdvsdepr'
#     "pretrained_model_path": 'models/lstm_clpsych_hierarchical64_ptsd'
    "pretrained_model_path": 'models/lstm_symanto_hierarchical64'
#     "pretrained_model_path": 'models/lstm_clpsych_hierarchical65'
#     "pretrained_model_path": 'models/lstm_depression_hierarchical56'
#     "pretrained_model_path": 'models/lstm_anorexia_hierarchical64'
#     "pretrained_model_path": 'models/lstm_selfharm_hierarchical59'
}


In [ ]:
if transfer_type:
    pretrained_model_path = hyperparams_features['pretrained_model_path']
    hyperparams, hyperparams_features = load_params(hyperparams_features['pretrained_model_path'])
    hyperparams_features['pretrained_model_path'] = pretrained_model_path

### Emotions

In [ ]:
def load_NRC(nrc_path):
    word_emotions = {}
    emotion_words = {}
    with open(nrc_path) as in_f:
        for line in in_f:
            line = line.strip()
            if not line:
                continue
            word, emotion, label = line.split()
            if word not in word_emotions:
                word_emotions[word] = set()
            if emotion not in emotion_words:
                emotion_words[emotion] = set()
            label = int(label)
            if label:
                word_emotions[word].add(emotion)
                emotion_words[emotion].add(word)
    return emotion_words

nrc_lexicon_path = root_dir + '/resources/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt'
nrc_lexicon = load_NRC(nrc_lexicon_path)
emotions = list(nrc_lexicon.keys())


In [ ]:
def encode_emotions(tokens, emotion_lexicon, emotions, relative=True):
    text_len = len(tokens)
    encoded_emotions = [0 for e in emotions]
    for i, emotion in enumerate(emotions):
        try:
            emotion_words = [t for t in tokens if t in emotion_lexicon[emotion]]
            if relative and len(tokens):
                encoded_emotions[i] = len(emotion_words) / len(tokens)
            else:
                encoded_emotions[i] = len(emotion_words)
        except ValueError:
            print("Emotion not found.")
    return encoded_emotions

In [ ]:
from liwc_readDict import readDict

liwc_dict = {}
for (w, c) in readDict(root_dir + '/resources/liwc.dic'):
    if c not in liwc_dict:
        liwc_dict[c] = []
    liwc_dict[c].append(w)

categories = set(liwc_dict.keys())
len(categories)

In [ ]:
liwc_categories=[c for c in categories]# if c in writings_df.columns]

In [ ]:
liwc_words_for_categories = pickle.load(open("data/liwc_categories_for_vocabulary_erisk_clpsych_stop_20K.pkl", "rb"))

In [ ]:
with open("liwc_categories.txt","w+") as f:
    for cat in liwc_dict:
        f.write("%s: %s\n" % (cat, ",".join(liwc_dict[cat])))

In [ ]:
with open("nrc_emotions.txt","w+") as f:
    for cat in nrc_lexicon:
        f.write("%s: %s\n" % (cat, ",".join(nrc_lexicon[cat])))

### Style features

#### Char n-grams

In [ ]:
def extract_ngrams(tokens):
    pass

#### Personal pronouns

In [ ]:
first_person_pronouns = {"i", "me", "my", "mine", "myself"}
def encode_pronouns(tokens, pronouns={"i", "me", "my", "mine", "myself"}, relative=True):
    if not tokens:
        return 0
    text_len = len(tokens)
    nr_pronouns = len([t for t in tokens if t in pronouns])
    if relative and text_len:
        return nr_pronouns/text_len
    else:
        return nr_pronouns

#### Stopwords

In [ ]:
stopword_list = stopwords.words("english")
def encode_stopwords(tokens, stopwords=stopword_list):
    encoded_stopwords = [0 for s in stopword_list]
    if not tokens:
        return encoded_stopwords
    for i, stopword in enumerate(stopwords):
        if stopword in tokens:
            encoded_stopwords[i] += 1
    return encoded_stopwords

### Topics

## BERT


In [ ]:
import tensorflow_hub as hub
bert_path = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
from bert.tokenization import FullTokenizer



In [ ]:
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.
    Args:
      guid: Unique id for the example.
      text_a: string. The untokenized text of the first sequence. For single
        sequence tasks, only this sequence must be specified.
      text_b: (Optional) string. The untokenized text of the second sequence.
        Only must be specified for sequence pair tasks.
      label: (Optional) string. The label of the example. This should be
        specified for train and dev examples, but not for test examples.
    """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label

In [ ]:
def encode_text_for_bert(tokenizer, example, max_seq_length=256):
    """Converts a single `InputExample` into a single `InputFeatures`."""

#     if isinstance(example, PaddingInputExample):
#         input_ids = [0] * max_seq_length
#         input_mask = [0] * max_seq_length
#         segment_ids = [0] * max_seq_length
#         label = 0
#         return input_ids, input_mask, segment_ids, label

    tokens_a = tokenizer.tokenize(example.text_a)
    if len(tokens_a) > max_seq_length - 2:
        tokens_a = tokens_a[0 : (max_seq_length - 2)]

    tokens = []
    segment_ids = []
    tokens.append("[CLS]")
    segment_ids.append(0)
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(0)
    tokens.append("[SEP]")
    segment_ids.append(0)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    return input_ids, input_mask, segment_ids, example.label



In [ ]:
def create_tokenizer_from_hub_module(sess):
    """Get the vocab file and casing info from the Hub module."""
    bert_module =  hub.Module(bert_path)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    vocab_file, do_lower_case = sess.run(
        [
            tokenization_info["vocab_file"],
            tokenization_info["do_lower_case"],
        ]
    )

    return FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

In [ ]:
def initialize_sess(sess=None):
    if not sess:
        sess_config = tf.ConfigProto(
            device_count={ 'GPU' : 1, 'CPU': 4 },
            intra_op_parallelism_threads = 0,
            inter_op_parallelism_threads = 4,
            allow_soft_placement=False,
            log_device_placement=True,
        )
        sess_config.gpu_options.allow_growth = True
        sess_config.gpu_options.per_process_gpu_memory_fraction = 1
        sess = tf.Session(config=sess_config)

    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    K.set_session(sess)
    return sess

In [ ]:
def load_saved_model(model_path, hyperparams):
    metrics_class = Metrics(threshold=hyperparams['threshold'])
    dependencies = {
    'f1_m': metrics_class.f1_m,
    'auc': metrics_class.auc,
    'precision_m': metrics_class.precision_m,
    'recall_m': metrics_class.recall_m,
    'binary_crossentropy_custom': binary_crossentropy_custom,
    'BertLayer': BertLayer
    }
    loaded_model = load_model(model_path + "_model.h5", custom_objects=dependencies)
#     loaded_model = load_model(model_path + "_model.model", custom_objects=dependencies)
    return loaded_model

In [ ]:
def load_saved_model_weights(model_path, hyperparams, emotions, stopword_list, liwc_categories, classes, h5=False):
    metrics_class = Metrics(threshold=hyperparams['threshold'])
    dependencies = {
    'f1_m': metrics_class.f1_m,
    'auc': metrics_class.auc,
    'precision_m': metrics_class.precision_m,
    'recall_m': metrics_class.recall_m,
    'binary_crossentropy_custom': binary_crossentropy_custom,
    'BertLayer': BertLayer
    }
    loaded_model = initialize_model(hyperparams=hyperparams, hyperparams_features=hyperparams_features, 
                                    embedding_matrix=embedding_matrix, 
                                 emotions=emotions, stopword_list=stopword_list, liwc_categories=liwc_categories,
                                classes=classes)
    loaded_model.summary()
#     loaded_model.load_weights(model_path + "_weights")
    path = model_path + "_weights"
    by_name = False
    if h5:
        path += ".h5"
        by_name=True
    loaded_model.load_weights(path, by_name=by_name)
    #     loaded_model = load_model(model_path + "_model.model", custom_objects=dependencies)
    return loaded_model

In [ ]:

tf.test.is_gpu_available()

### Encode data

In [ ]:
from collections import Counter
def load_erisk_data(writings_df, voc_size, emotion_lexicon, emotions =  
                    ['anger', 'anticipation', 'disgust', 'fear', 'joy', 
                     'negative', 'positive', 'sadness', 'surprise', 'trust'],
                    liwc_categories = categories, by_subset=True,
                    pronouns = ["i", "me", "my", "mine", "myself"],
                    train_prop=0.7, valid_prop=0.3, test_slice=2,
                    nr_slices=5,
                    min_post_len=3, min_word_len=1, 
                    user_level=True, vocabulary=None, labelcol='label', label_index=None,
                   logger=logger):
    logger.debug("Loading data...\n")
    
    ## Build vocabulary
    vocabulary_all = {}
    word_freqs = Counter()
    
    for words in writings_df.tokenized_text:
        word_freqs.update(words)
    if 'tokenized_title' in writings_df.columns:
        for words in writings_df.tokenized_title:
            word_freqs.update(words)
    i = 1
    for w, f in word_freqs.most_common(voc_size-2): # keeping voc_size-1 for unk
        if len(w) < min_word_len:
            continue
        vocabulary_all[w] = i
        i += 1
    if not vocabulary:
        vocabulary = vocabulary_all
    else:
        logger.info("Words not found in the vocabulary: %d\n" % len(set(vocabulary_all.keys()).difference(
            set(vocabulary.keys()))))

    if labelcol != 'label' and not label_index:
        label_index = {}
        l = 0
        for label in set(writings_df[labelcol]):
            label_index[label] = l
            l += 1
        print("Label index", label_index)
   
    if by_subset and 'subset' in writings_df.columns:
        training_subjects = list(set(writings_df[writings_df['subset']=='train'].subject))
        test_subjects = list(set(writings_df[writings_df['subset']=='test'].subject))
    else:
        all_subjects = sorted(list(set(writings_df.subject)))
        training_subjects_size = int(len(all_subjects) * train_prop)
        test_subjects_size = len(all_subjects) - training_subjects_size
        logger.info("%d training subjects, %d test subjects\n" % (training_subjects_size, test_subjects_size))
        # Cross-validation, with fixed slice as input
        test_prop = 1-train_prop
        test_slice = min(test_slice, nr_slices)
        logger.debug("start index: %f, from %f\n" % (
            len(all_subjects)*(1/nr_slices)*test_slice, test_prop*test_slice))
        start_slice = int(len(all_subjects)*(1/nr_slices)*test_slice)
        test_subjects = all_subjects[start_slice: start_slice+test_subjects_size]
        training_subjects = [s for s in all_subjects if s not in test_subjects]
    training_subjects = sorted(training_subjects) # ensuring reproducibility
    valid_subjects_size = int(len(training_subjects) * valid_prop)
    valid_subjects = training_subjects[:valid_subjects_size]
    training_subjects = training_subjects[valid_subjects_size:]
    categories = [c for c in liwc_categories if c in writings_df.columns]
    logger.debug("%d training users, %d validation users, %d test users." % (
        len(training_subjects), 
          len(valid_subjects),
          len(test_subjects)))
    subjects_split = {'train': training_subjects, 
                      'valid': valid_subjects, 
                      'test': test_subjects}

    user_level_texts = {}
    for row in writings_df.sort_values(by='date').itertuples():
        words = []
        raw_text = ""
        if hasattr(row, 'tokenized_title'):
            if row.tokenized_title:
                words.extend(row.tokenized_title)
                raw_text += row.title
        if hasattr(row, 'tokenized_text'):
            if row.tokenized_text:
                words.extend(row.tokenized_text)
                raw_text += row.text
        if not words or len(words)<min_post_len:
#             logger.debug(row.subject)
            continue
        if labelcol == 'label':
            label = row.label
        else:
            label = label_index[getattr(row, labelcol)]
        liwc_categs = [getattr(row, categ) for categ in categories]
        if row.subject not in user_level_texts.keys():
            user_level_texts[row.subject] = {}
            user_level_texts[row.subject]['texts'] = [words]
            user_level_texts[row.subject]['label'] = label
            user_level_texts[row.subject]['liwc'] = [liwc_categs]
            user_level_texts[row.subject]['raw'] = [raw_text]
        else:
            user_level_texts[row.subject]['texts'].append(words)
            user_level_texts[row.subject]['liwc'].append(liwc_categs)
            user_level_texts[row.subject]['raw'].append(raw_text)
            
    return user_level_texts, subjects_split, vocabulary



In [ ]:
# del writings_df['tokenized_title']


In [ ]:
vocabulary_list = pickle.load(open(hyperparams_features['vocabulary_path'], 'rb'))
vocabulary_dict={}
for i,w in enumerate(vocabulary_list):
    vocabulary_dict[w] = i
user_level_data, subjects_split, vocabulary = load_erisk_data(writings_df, 
                                                            voc_size=hyperparams_features['max_features'],
                                                           emotion_lexicon=nrc_lexicon,
                                                           emotions=emotions,
                                                           user_level=hyperparams_features['user_level'],
                                                                                logger=logger,
#                                                            vocabulary=pickle.load(open('vocabulary_40K_all.pkl', 'rb')),
#                                                            vocabulary=pickle.load(open('vocab_clpsych_10000.pkl', 'rb')),
                                                              vocabulary=vocabulary_dict,
                                                              by_subset=True,
#                                                               labelcol = 'condition',
#                                                               label_index={'depression':1, "ptsd":0}
                                                                               )

In [ ]:
# writings_df['subset'] = writings_df['subject'].apply(lambda s: 'test' if s in subjects_split['test'] else 'train')

In [ ]:
len(vocabulary_dict)

In [ ]:
writings_df['tokenized_text'].head()

In [ ]:
writings_df.text_len.hist()

In [ ]:
# pickle.dump(writings_df, open('writings_df_selfharm_liwc_subsets', 'wb+'))

In [ ]:
sorted(vocabulary.items(), key=lambda t:t[1])

In [ ]:
# session_collection = {}
# models_collection = {}
# hyperparams_collection = {}
# user_level_data['subject57080000']['label']

### Data Generator

In [ ]:
class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, user_level_data, subjects_split, session=None, use_bert=False, set_type='train',
                 batch_size=32, seq_len=512, vocabulary=vocabulary,
                 voc_size=hyperparams_features['max_features'], emotion_lexicon=nrc_lexicon,
                 hierarchical=False, pad_value=0, padding='pre',
                 post_groups_per_user=None, posts_per_group=10, post_offset = 0,
                 sampling_distr_alfa=0.1, sampling_distr='exp', # 'exp', 'uniform'
                 emotions=emotions, pronouns=["i", "me", "my", "mine", "myself"], liwc_categories=liwc_categories,
                 liwc_dict=liwc_dict, compute_liwc=False, liwc_words_for_categories=None,
                 pad_with_duplication=False,
                 max_posts_per_user=None, sample_seqs=True,
                 shuffle=True, return_subjects=False, keep_last_batch=True, class_weights=None,
                classes=1):
        'Initialization'
        self.seq_len = seq_len
        # Instantiate tokenizer
        if session:
            self.bert_tokenizer = create_tokenizer_from_hub_module(session)
            session.run(tf.local_variables_initializer())
            session.run(tf.global_variables_initializer())
            session.run(tf.tables_initializer())
        else:
            if use_bert:
                logger.error("Need a session to use bert in data generation")
            self.bert_tokenizer = None
        self.use_bert = use_bert
        self.subjects_split = subjects_split
        self.set = set_type
        self.emotion_lexicon = emotion_lexicon
        self.batch_size = batch_size
        self.hierarchical = hierarchical
        self.data = user_level_data
        self.pad_value = pad_value
        self.return_subjects = return_subjects
        self.sampling_distr_alfa = sampling_distr_alfa
        self.sampling_distr = sampling_distr
        self.emotions = emotions
        self.pronouns = pronouns
        self.liwc_categories = liwc_categories
        self.liwc_dict = liwc_dict
        self.liwc_words_for_categories = liwc_words_for_categories
        self.compute_liwc = compute_liwc
        self.sample_seqs = sample_seqs
        self.pad_with_duplication = pad_with_duplication
        self.padding = padding
        self.keep_last_batch = keep_last_batch
        self.shuffle = shuffle
        self.voc_size = voc_size
        self.vocabulary = vocabulary
        self.max_posts_per_user = max_posts_per_user
        self.post_groups_per_user = post_groups_per_user
        self.post_offset = post_offset
        self.posts_per_group = posts_per_group
        self.classes = classes
        self.class_weights = class_weights
        self.generated_labels = []
        self.__post_indexes_per_user()
        self.on_epoch_end()
        
    @staticmethod
    def _random_sample(population_size, sample_size, sampling_distr, alfa=0.1, replacement=False):
        if sampling_distr == 'exp':
            # Exponential sampling
            sample = sorted(np.random.choice(population_size, 
                            min(sample_size, population_size),
                            p = DataGenerator.__generate_reverse_exponential_indices(population_size, alfa),
                            replace=replacement))
                                                                # if pad_with_duplication, 
                                                                # pad by adding the same post multiple times
                                                                # if there are not enough posts
        elif sampling_distr == 'uniform':
            # Uniform sampling
            sample = sorted(np.random.choice(population_size,
                            min(sample_size, population_size),
                            replace=replacement))
        return sample
    
    @staticmethod
    def __generate_reverse_exponential_indices(max_index, alfa=1):
        probabilities = []
        for x in range(max_index):
            probabilities.append(alfa * (np.exp(alfa*x)))
        reverse_probabilities = [p for p in probabilities]
        sump = sum(reverse_probabilities)
        normalized_probabilities = [p/sump for p in reverse_probabilities]
        return normalized_probabilities
    
    def __post_indexes_per_user(self):
        self.indexes_per_user = {u: [] for u in range(len(self.subjects_split[self.set]))}
        self.indexes_with_user = []
        self.item_weights = []
        for u in range(len(self.subjects_split[self.set])):
            if self.subjects_split[self.set][u] not in self.data:
                logger.warning("User %s has no posts in %s set. Ignoring.\n" % (
                    self.subjects_split[self.set][u], self.set))
                continue
            user_posts = self.data[self.subjects_split[self.set][u]]['texts']
            if self.max_posts_per_user:
                user_posts = user_posts[:self.max_posts_per_user]
            nr_post_groups = int(np.ceil(len(user_posts) / self.posts_per_group))
            
            if self.post_groups_per_user:
                nr_post_groups = min(self.post_groups_per_user, nr_post_groups)
            for i in range(nr_post_groups):
                # Generate random ordered samples of the posts
                if self.sample_seqs:
                    indexes_sample = DataGenerator._random_sample(population_size=len(user_posts),
                                                         sample_size=self.posts_per_group,
                                                         sampling_distr=self.sampling_distr,
                                                         alfa=self.sampling_distr_alfa,
                                                         replacement=self.pad_with_duplication)
                    self.indexes_per_user[u].append(indexes_sample)
                    self.indexes_with_user.append((u, indexes_sample))
                    # break # just generate one?
                # Generate all subsets of the posts in order
                # TODO: Change here if you want a sliding window
                else:
                    self.indexes_per_user[u].append(range(i*self.posts_per_group + self.post_offset,
                                                        min((i+1)*self.posts_per_group + self.post_offset, len(user_posts))))
                    self.indexes_with_user.append((u, range(i*self.posts_per_group ,
                                                        min((i+1)*self.posts_per_group + self.post_offset, len(user_posts)))))

        if self.class_weights:
            for item in self.indexes_with_user:
                u, _ = item
                s = self.subjects_split[self.set][u]
                # Note: weight 1 is default.
                try:
                    self.item_weights.append(1./self.class_weights[self.data[s]['label']])
                except Exception as e:
                    self.item_weights.append(1)
                    logger.error("Could not compute item weight for user %s. " % s + str(e) + "\n")
        else:
            self.item_weights = []

    def __encode_text(self, tokens, raw_text):
        # Using voc_size-1 value for OOV token
        encoded_tokens = [self.vocabulary.get(w, self.voc_size-1) for w in tokens]
        encoded_emotions = encode_emotions(tokens, self.emotion_lexicon, self.emotions)
        encoded_pronouns = encode_pronouns(tokens, self.pronouns)
        encoded_stopwords = encode_stopwords(tokens)
        if not self.compute_liwc:
            encoded_liwc = None
        else:
            encoded_liwc = self.__encode_liwc_categories(tokens)
        if self.bert_tokenizer:
            bert_ids, bert_masks, bert_segments, label = encode_text_for_bert(self.bert_tokenizer, InputExample(None, 
                                               raw_text), self.seq_len)
        else:
            bert_ids, bert_masks, bert_segments = [[0]*self.seq_len, [0]*self.seq_len, [0]*self.seq_len]
        return (encoded_tokens, encoded_emotions, encoded_pronouns, encoded_stopwords, encoded_liwc,
               bert_ids, bert_masks, bert_segments)
    
    def __encode_liwc_categories_full(self, tokens, relative=True):
        categories_cnt = [0 for c in self.liwc_categories]
        if not tokens:
            return categories_cnt
        text_len = len(tokens)
        for i, category in enumerate(self.liwc_categories):
            category_words = self.liwc_dict[category]
            for t in tokens:
                for word in category_words:
                    if t==word or (word[-1]=='*' and t.startswith(word[:-1])) \
                    or (t==word.split("'")[0]):
                        categories_cnt[i] += 1
                        break # one token cannot belong to more than one word in the category
            if relative and text_len:
                categories_cnt[i] = categories_cnt[i]/text_len
        return categories_cnt
        
        
    def __encode_liwc_categories(self, tokens, relative=True):
        categories_cnt = [0 for c in self.liwc_categories]
        if not tokens:
            return categories_cnt
        text_len = len(tokens)
        for i, category in enumerate(self.liwc_categories):
            for t in tokens:
                if t in self.liwc_words_for_categories[category]:
                    categories_cnt[i] += 1
            if relative and text_len:
                categories_cnt[i] = categories_cnt[i]/text_len
        return categories_cnt
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        if self.keep_last_batch:
            return int(np.ceil(len(self.indexes) / self.batch_size)) # + 1 to not discard last batch
        return int((len(self.indexes))/self.batch_size)
        
    def __getitem__(self, index):
        'Generate one batch of data'
        # Reset generated labels
        if index == 0:
             self.generated_labels = []
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # Find users
        user_indexes = [t[0] for t in indexes]
        users = set([self.subjects_split[self.set][i] for i in user_indexes
                    if self.subjects_split[self.set][i] in self.data.keys()]) # TODO: maybe needs a warning that user is missing
        post_indexes_per_user = {u: [] for u in users}
        # Sample post ids
        for u, post_indexes in indexes:
            user = self.subjects_split[self.set][u]
            # Note: was bug here - changed it into a list
            post_indexes_per_user[user].append(post_indexes)

        # Generate data
        if self.hierarchical:
            X, s, y = self.__data_generation_hierarchical(users, post_indexes_per_user)
        else:
            X, s, y = self.__data_generation(users, post_indexes_per_user)

        if self.return_subjects:
            return X, s, y
        else:
            return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = self.indexes_with_user
#         np.arange(len(self.subjects_split[self.set]))
        if self.shuffle:
            np.random.shuffle(self.indexes)
        if self.class_weights:
            # Sample users according to class weight (Or do this for each batch instead?)
            normalized_weights = [w/sum(self.item_weights) for w in self.item_weights]
            random_user_indexes = np.random.choice(len(self.indexes_with_user), 
                            len(self.indexes_with_user),
                            p = normalized_weights, replace=True)
            self.indexes = [self.indexes_with_user[i] for i in random_user_indexes]
    def __data_generation(self, users, post_indexes):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        tokens_data = []
        categ_data = []
        sparse_data = []
        subjects = []
        bert_ids_data = []
        bert_masks_data = []
        bert_segments_data = []
        labels = []

        for subject in users:

            if 'label' in self.data[subject]:
                label = self.data[subject]['label']
            else:
                label = None

            
            all_words = []
            all_raw_texts = []
            liwc_aggreg = []

            for post_index_range in post_indexes[subject]:
                # Sample
                texts = [self.data[subject]['texts'][i] for i in post_index_range]
                if 'liwc' in self.data[subject] and not self.compute_liwc:
                    liwc_selection = [self.data[subject]['liwc'][i] for i in post_index_range]
                raw_texts = [self.data[subject]['raw'][i] for i in post_index_range]

                all_words.append(sum(texts, [])) # merge all texts in group in one list
                if 'liwc' in self.data[subject] and not self.compute_liwc:
                    liwc_aggreg.append(np.array(liwc_selection).mean(axis=0).tolist())
                all_raw_texts.append(" ".join(raw_texts))
            for i, words in enumerate(all_words):
                encoded_tokens, encoded_emotions, encoded_pronouns, encoded_stopwords, encoded_liwc, \
                    bert_ids, bert_masks, bert_segments = self.__encode_text(words, all_raw_texts[i])
                try:
                    subject_id = int(re.findall('[0-9]+', subject)[0])
                except IndexError:
                    subject_id = subject
                tokens_data.append(encoded_tokens)
                # TODO: what will be the difference between these?
                # I think instead of averaging for the post group, it just does it correctly
                # for the whole post group (when computing, non-lazily)
                if 'liwc' in self.data[subject] and not self.compute_liwc:  
                    categ_data.append(encoded_emotions + [encoded_pronouns] + liwc_aggreg[i])
                   
                else:
                    categ_data.append(encoded_emotions + [encoded_pronouns] + encoded_liwc)
                    
                sparse_data.append(encoded_stopwords)
                bert_ids_data.append(bert_ids)
                bert_masks_data.append(bert_masks)
                bert_segments_data.append(bert_segments)
                
                labels.append(label)
                subjects.append(subject_id)

        
        self.generated_labels.extend(labels)
        # using zeros for padding
        tokens_data_padded = sequence.pad_sequences(tokens_data, maxlen=self.seq_len, 
                                                    padding=self.padding,
                                                   truncating=self.padding)

        if self.use_bert:
            return ([np.array(tokens_data_padded), np.array(categ_data), np.array(sparse_data),
                 np.array(bert_ids_data), np.array(bert_masks_data), np.array(bert_segments_data),
                ],
                np.array(subjects),
                np.array(labels))
        else:
            return ([np.array(tokens_data_padded), np.array(categ_data), np.array(sparse_data),
                ],
                np.array(subjects),
                np.array(labels))
    
    def __data_generation_hierarchical(self, users, post_indexes):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        user_tokens = []
        user_categ_data = []
        user_sparse_data = []
        user_bert_ids_data = []
        user_bert_masks_data = []
        user_bert_segments_data = []
        
        labels = []
        subjects = []
        for subject in users:
             
            all_words = []
            all_raw_texts = []
            liwc_scores = []
            
            if 'label' in self.data[subject]:
                if self.classes==1:
                    label = self.data[subject]['label']
                else:
                    label = list(np_utils.to_categorical(self.data[subject]['label'], num_classes=self.classes))
            else:
                label = None

            for post_index_range in post_indexes[subject]:
                # Sample
                texts = [self.data[subject]['texts'][i] for i in post_index_range]
                if 'liwc' in self.data[subject] and not self.compute_liwc:
                    liwc_selection = [self.data[subject]['liwc'][i] for i in post_index_range]
                raw_texts = [self.data[subject]['raw'][i] for i in post_index_range]

                all_words.append(texts)
                if 'liwc' in self.data[subject] and not self.compute_liwc:
                    liwc_scores.append(liwc_selection)
                all_raw_texts.append(raw_texts)
            
#             if len(texts) < self.max_posts_per_user:
#                 # TODO: pad with zeros
#                 pass

            for i, words in enumerate(all_words):
                tokens_data = []
                categ_data = []
                sparse_data = []
                bert_ids_data = []
                bert_masks_data = []
                bert_segments_data = []
                
                raw_text = all_raw_texts[i]
                words = all_words[i]
                
                for p, posting in enumerate(words): 
                    encoded_tokens, encoded_emotions, encoded_pronouns, encoded_stopwords, encoded_liwc, \
                        bert_ids, bert_masks, bert_segments = self.__encode_text(words[p], raw_text[p])
                    if 'liwc' in self.data[subject] and not self.compute_liwc:
                        liwc = liwc_scores[i][p]
                    else:
                        liwc = encoded_liwc
                    try:
                        subject_id = int(re.findall('[0-9]+', subject)[0])
                    except IndexError:
                        subject_id = subject
                    tokens_data.append(encoded_tokens)
                    # using zeros for padding
                    # TODO: there is something wrong with this
                    categ_data.append(encoded_emotions + [encoded_pronouns] + liwc)
                    sparse_data.append(encoded_stopwords)
                    bert_ids_data.append(bert_ids)
                    bert_masks_data.append(bert_masks)
                    bert_segments_data.append(bert_segments)
                
                # For each range
                tokens_data_padded = np.array(sequence.pad_sequences(tokens_data, maxlen=self.seq_len,
                                              padding=self.padding,
                                            truncating=self.padding))
                user_tokens.append(tokens_data_padded)

                user_categ_data.append(categ_data)
                user_sparse_data.append(sparse_data)

                user_bert_ids_data.append(bert_ids_data)
                user_bert_masks_data.append(bert_masks_data)
                user_bert_segments_data.append(bert_segments_data)

                labels.append(label)
                subjects.append(subject_id)

        user_tokens = sequence.pad_sequences(user_tokens, 
                                             maxlen=self.posts_per_group, 
                                             value=self.pad_value)
        user_tokens = np.rollaxis(np.dstack(user_tokens), -1)
        user_categ_data = sequence.pad_sequences(user_categ_data,  
                                                 maxlen=self.posts_per_group, 
                                                 value=self.pad_value, dtype='float32')
        user_categ_data = np.rollaxis(np.dstack(user_categ_data), -1)
        
        user_sparse_data = sequence.pad_sequences(user_sparse_data, 
                                                  maxlen=self.posts_per_group, 
                                                  value=self.pad_value)
        user_sparse_data = np.rollaxis(np.dstack(user_sparse_data), -1)
        
        user_bert_ids_data = sequence.pad_sequences(user_bert_ids_data, 
                                                    maxlen=self.posts_per_group, 
                                                    value=self.pad_value)
        user_bert_ids_data = np.rollaxis(np.dstack(user_bert_ids_data), -1)
        
        user_bert_masks_data = sequence.pad_sequences(user_bert_masks_data, 
                                                      maxlen=self.posts_per_group, 
                                                      value=self.pad_value)
        user_bert_masks_data = np.rollaxis(np.dstack(user_bert_masks_data), -1)
        
        user_bert_segments_data = sequence.pad_sequences(user_bert_segments_data, 
                                                         maxlen=self.posts_per_group, 
                                                         value=self.pad_value)
        user_bert_segments_data = np.rollaxis(np.dstack(user_bert_segments_data), -1)

        self.generated_labels.extend(labels)

        labels = np.array(labels)
        
        if self.use_bert:
            return ((user_tokens, user_categ_data, user_sparse_data, 
                 user_bert_ids_data, user_bert_masks_data, uifser_bert_segments_data),
                np.array(subjects),
                labels)
        else:
            return ((user_tokens, user_categ_data, user_sparse_data), 
                np.array(subjects),
                labels)

In [ ]:
class DataGeneratorFromText(Sequence):
    'Generates data for Keras'
    def __init__(self, texts, batch_size=32, seq_len=512, vocabulary=vocabulary,
                 voc_size=hyperparams_features['max_features'], emotion_lexicon=nrc_lexicon,
                 hierarchical=False, pad_value=0, padding='pre',
                 emotions=emotions, pronouns=["i", "me", "my", "mine", "myself"], liwc_categories=liwc_categories,
                 liwc_dict=liwc_dict, compute_liwc=False, liwc_words_for_categories=None,
                 pad_with_duplication=False, posts_per_group=50,
                 shuffle=True, keep_last_batch=True):
        'Initialization'
        self.seq_len = seq_len
        # Instantiate tokenizer
        
        self.texts = texts
        self.emotion_lexicon = emotion_lexicon
        self.batch_size = batch_size
        self.hierarchical = hierarchical
 
        self.pad_value = pad_value

        self.keep_last_batch = keep_last_batch
        self.emotions = emotions
        self.pronouns = pronouns
        self.liwc_categories = liwc_categories
        self.liwc_dict = liwc_dict
        self.liwc_words_for_categories = liwc_words_for_categories
        self.compute_liwc = compute_liwc
        
        self.use_bert = False
      
        self.pad_with_duplication = pad_with_duplication
        self.padding = padding
        self.shuffle = shuffle
        self.voc_size = voc_size
        self.vocabulary = vocabulary
        
        self.posts_per_group = posts_per_group
       
        self.on_epoch_end()
        
   

    def __encode_text(self, tokens, raw_text):
        # Using voc_size-1 value for OOV token
        encoded_tokens = [self.vocabulary.get(w, self.voc_size-1) for w in tokens]
        encoded_emotions = encode_emotions(tokens, self.emotion_lexicon, self.emotions)
        encoded_pronouns = encode_pronouns(tokens, self.pronouns)
        encoded_stopwords = encode_stopwords(tokens)

        encoded_liwc = self.__encode_liwc_categories(tokens)
      
        bert_ids, bert_masks, bert_segments = [[0]*self.seq_len, [0]*self.seq_len, [0]*self.seq_len]
        return (encoded_tokens, encoded_emotions, encoded_pronouns, encoded_stopwords, encoded_liwc,
               bert_ids, bert_masks, bert_segments)
    
    def __encode_liwc_categories_full(self, tokens, relative=True):
        categories_cnt = [0 for c in self.liwc_categories]
        if not tokens:
            return categories_cnt
        text_len = len(tokens)
        for i, category in enumerate(self.liwc_categories):
            category_words = self.liwc_dict[category]
            for t in tokens:
                for word in category_words:
                    if t==word or (word[-1]=='*' and t.startswith(word[:-1])) \
                    or (t==word.split("'")[0]):
                        categories_cnt[i] += 1
                        break # one token cannot belong to more than one word in the category
            if relative and text_len:
                categories_cnt[i] = categories_cnt[i]/text_len
        return categories_cnt
        
        
    def __encode_liwc_categories(self, tokens, relative=True):
        categories_cnt = [0 for c in self.liwc_categories]
        if not tokens:
            return categories_cnt
        text_len = len(tokens)
        for i, category in enumerate(self.liwc_categories):
            for t in tokens:
                if t in self.liwc_words_for_categories[category]:
                    categories_cnt[i] += 1
            if relative and text_len:
                categories_cnt[i] = categories_cnt[i]/text_len
        return categories_cnt
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        if self.keep_last_batch:
            return int(np.ceil(len(self.indexes) / self.batch_size)) # + 1 to not discard last batch
        return int((len(self.indexes))/self.batch_size)
        
    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
       
        # Generate data
        if self.hierarchical:
            X, s, y = self.__data_generation_hierarchical(indexes)
        else:
            X, s, y = self.__data_generation(indexes)
      
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.texts))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        tokens_data = []
        categ_data = []
        sparse_data = []
        subjects = []
        bert_ids_data = []
        bert_masks_data = []
        bert_segments_data = []
        labels = []

        label = None


        all_words = []
        all_raw_texts = []
        liwc_aggreg = []

        # Sample
        all_words = [tokenize(self.texts[i]) for i in indexes]

        all_raw_texts = [self.texts[i] for i in indexes]


        for i, words in enumerate(all_words):
            encoded_tokens, encoded_emotions, encoded_pronouns, encoded_stopwords, encoded_liwc, \
                bert_ids, bert_masks, bert_segments = self.__encode_text(words, all_raw_texts[i])

            tokens_data.append(encoded_tokens)
            # TODO: what will be the difference between these?
            # I think instead of averaging for the post group, it just does it correctly
            # for the whole post group (when computing, non-lazily)

            categ_data.append(encoded_emotions + [encoded_pronouns] + encoded_liwc)

            sparse_data.append(encoded_stopwords)
            bert_ids_data.append(bert_ids)
            bert_masks_data.append(bert_masks)
            bert_segments_data.append(bert_segments)

            labels.append(None)
            subjects.append(None)

        
        # using zeros for padding
        tokens_data_padded = sequence.pad_sequences(tokens_data, maxlen=self.seq_len, 
                                                    padding=self.padding,
                                                   truncating=self.padding)

       
        return ([np.array(tokens_data_padded), np.array(categ_data), np.array(sparse_data),
                ],
                np.array(subjects),
                np.array(labels))
    
    def __data_generation_hierarchical(self, indexes):
#         pass
#         'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        user_tokens = []
        user_categ_data = []
        user_sparse_data = []
        user_bert_ids_data = []
        user_bert_masks_data = []
        user_bert_segments_data = []

        labels = []
        subjects = []
#         for subject in users:
             
        all_words = []
        all_raw_texts = []
        liwc_scores = []

        label = -1

#         for index_range in indexes:
            # Sample
        texts = [tokenize(self.texts[i]) for i in indexes] # is this good or am I repeating the same texts?

        raw_texts = [self.texts[i] for i in indexes]
        all_words.append(texts)

        all_raw_texts.append(raw_texts)

#             if len(texts) < self.max_posts_per_user:
#                 # TODO: pad with zeros
#                 pass

        for i, words in enumerate(all_words):
            tokens_data = []
            categ_data = []
            sparse_data = []
            bert_ids_data = []
            bert_masks_data = []
            bert_segments_data = []

            raw_text = all_raw_texts[i]
            words = all_words[i]

            for p, posting in enumerate(words): 
                encoded_tokens, encoded_emotions, encoded_pronouns, encoded_stopwords, encoded_liwc, \
                    bert_ids, bert_masks, bert_segments = self.__encode_text(words[p], raw_text[p])
                

                liwc = encoded_liwc
      
                tokens_data.append(encoded_tokens)
                # using zeros for padding
                # TODO: there is something wrong with this
                categ_data.append(encoded_emotions + [encoded_pronouns] + liwc)
                sparse_data.append(encoded_stopwords)
                bert_ids_data.append(bert_ids)
                bert_masks_data.append(bert_masks)
                bert_segments_data.append(bert_segments)
                
                labels.append(None)
                subjects.append(None)

                # For each range
                tokens_data_padded = np.array(sequence.pad_sequences(tokens_data, maxlen=self.seq_len,
                                              padding=self.padding,
                                            truncating=self.padding))
                user_tokens.append(tokens_data_padded)

                user_categ_data.append(categ_data)
                user_sparse_data.append(sparse_data)

                user_bert_ids_data.append(bert_ids_data)
                user_bert_masks_data.append(bert_masks_data)
                user_bert_segments_data.append(bert_segments_data)



        user_tokens = sequence.pad_sequences(user_tokens, 
                                             maxlen=self.posts_per_group, 
                                             value=self.pad_value)
        user_tokens = np.rollaxis(np.dstack(user_tokens), -1)
        user_categ_data = sequence.pad_sequences(user_categ_data,  
                                                 maxlen=self.posts_per_group, 
                                                 value=self.pad_value, dtype='float32')
        user_categ_data = np.rollaxis(np.dstack(user_categ_data), -1)
        
        user_sparse_data = sequence.pad_sequences(user_sparse_data, 
                                                  maxlen=self.posts_per_group, 
                                                  value=self.pad_value)
        user_sparse_data = np.rollaxis(np.dstack(user_sparse_data), -1)
        
        user_bert_ids_data = sequence.pad_sequences(user_bert_ids_data, 
                                                    maxlen=self.posts_per_group, 
                                                    value=self.pad_value)
        user_bert_ids_data = np.rollaxis(np.dstack(user_bert_ids_data), -1)
        
        user_bert_masks_data = sequence.pad_sequences(user_bert_masks_data, 
                                                      maxlen=self.posts_per_group, 
                                                      value=self.pad_value)
        user_bert_masks_data = np.rollaxis(np.dstack(user_bert_masks_data), -1)
        
        user_bert_segments_data = sequence.pad_sequences(user_bert_segments_data, 
                                                         maxlen=self.posts_per_group, 
                                                         value=self.pad_value)
        user_bert_segments_data = np.rollaxis(np.dstack(user_bert_segments_data), -1)
        

        labels = np.array(labels)

        return ([user_tokens, user_categ_data, user_sparse_data], 
            np.array(subjects),
            np.array(labels))

In [ ]:
for x, y in DataGeneratorFromText(["This is the first text", "and this is the second text"], 
                                  liwc_words_for_categories=liwc_words_for_categories, 
                                  compute_liwc=True):
    print(x[0].shape, x[1].shape, x[2].shape)

In [ ]:
# g = DataGenerator(user_level_data, subjects_split, 
#                                          set_type='test', 
#                                     seq_len=hyperparams_collection[key]['maxlen'],   
# #                                         batch_size=len(subjects_split['test']), # on all data at once
#                                         batch_size=hyperparams_collection[key]['batch_size'], # on all data at once
#                                        hierarchical=hyperparams_collection[key]['hierarchical'],
#                                          max_posts_per_user=None,
#                                        pad_with_duplication=False,
#                                         posts_per_group=hyperparams_collection[key]['posts_per_group'],
#                                         post_groups_per_user=1,  compute_liwc=True,
#                                         liwc_words_for_categories=liwc_words_for_categories,
#                                         emotions=emotions, liwc_categories=liwc_categories,
#                                          sample_seqs=False, shuffle=False, classes=classes,
#                  class_weights={0:2,1:1,2:1})
# # g = DataGenerator(user_level_data, subjects_split, sample_seqs=False, max_posts_per_user=None,
# #                                           set_type='test', hierarchical=True, post_groups_per_user=None,
# #                               posts_per_group=50, shuffle=False,
# #                              sampling_distr='exp', liwc_words_for_categories=liwc_words_for_categories,
# #                           "Co"   compute_liwc=True, classes=3)
# for cnt, x in enumerate(g):
#     continue
# print(cnt)
# print(g.generated_labels)

In [ ]:
# len(g.generated_labels)

In [ ]:
%%time
logger.setLevel(logging.DEBUG)

# TODO: it is skipping the last batch
x_data = {'train': [], 'valid': [], 'test': []}
y_data = {'train': [], 'valid': [], 'test': []}
for set_type in ['train', 'valid', 'test']:
    total_positive = 0
    for x, y in DataGenerator(user_level_data, subjects_split, sample_seqs=False, max_posts_per_user=None,
                                          set_type=set_type, hierarchical=True, post_groups_per_user=1,
                              posts_per_group=50, shuffle=False,
                             sampling_distr='exp', liwc_words_for_categories=liwc_words_for_categories,
                             compute_liwc=True, classes=3):
#         total_positive += pd.Series(y).sum()
        x_data[set_type].append(x)
        y_data[set_type].append(y)
        logger.info("%s %s positive examples\n" % (total_positive, set_type))


In [ ]:
%%time
# TODO: it is skipping the last batch
x_data2 = {'train': [], 'valid': [], 'test': []}
y_data2 = {'train': [], 'valid': [], 'test': []}
for set_type in ['train', 'valid', 'test']:
    total_positive = 0
    for x, y in DataGenerator(user_level_data, subjects_split, sample_seqs=False, max_posts_per_user=None,
                                          set_type=set_type, hierarchical=False, post_groups_per_user=1,
                              posts_per_group=50, shuffle=False,
                             sampling_distr='exp', liwc_words_for_categories=liwc_words_for_categories,
                             compute_liwc=False):
        total_positive += pd.Series(y).sum()
        x_data2[set_type].append(x)
        y_data2[set_type].append(y)
        logger.info("%d %s positive examples\n" % (total_positive, set_type))

In [ ]:
x_data['train'][0][1][0]
y_data['train']

In [ ]:
x_data2['train'][0][1][0]

In [ ]:
# #x_data['train'][]
# featureindex = 1

# from scipy.stats import pearsonr, spearmanr
# for i in range(len(x_data['train'][0][featureindex])):
#     print(spearmanr(x_data['train'][0][featureindex][i], x_data2['train'][0][featureindex][i]))    
#     plt.scatter(x_data['train'][0][featureindex][i], x_data2['train'][0][featureindex][i])
# #     break


In [ ]:
sum([len(liwc_words_for_categories[c]) for c in categories])
len(categories)
len(set(categories))

In [ ]:
# (x_data['valid'][0][0].shape, x_data['valid'][0][1].shape)

In [ ]:
# encoded_for_bert = encode_text_for_bert(bert_tokenizer, InputExample(None, 
#                                                "Ana are mere"), 200)

In [ ]:
# ids, masks, segments, label = encoded_for_bert

In [ ]:
# class_weights = class_weight.compute_class_weight('balanced',
#                                                  np.unique(y_data['train']),
#                                                  y_data['train'])
# class_weights

In [ ]:
def load_embeddings(path, embedding_dim, voc):
    # random matrix with mean value = 0
    embedding_matrix = np.random.random((len(voc)+2, embedding_dim)) - 0.5 # voc + unk + pad value(0)
    cnt_inv = 0
    f = open(path, encoding='utf8')
    for i, line in enumerate(f):
#         print(i)
        values = line.split()
        word = ''.join(values[:-hyperparams_features['embedding_dim']])
        coefs = np.asarray(values[-hyperparams_features['embedding_dim']:], dtype='float32')
        word_i = voc.get(word)
        if word_i is not None:
            embedding_matrix[word_i] = coefs
            cnt_inv += 1
    f.close()

    print('Total %s word vectors.' % len(embedding_matrix))
    print('Words not found in embedding space %d' % (len(embedding_matrix)-cnt_inv))
 
    return embedding_matrix

def load_embeddings2(path, embedding_dim, voc):
    # random matrix with mean value = 0
    embedding_matrix = np.random.random((len(voc)+2, embedding_dim)) #- 0.5 # voc + unk + pad value(0)
    cnt_inv = 0
    with open(path, "rb") as f:
        embedding_dict = pickle.load(f)
    for word, coefs in embedding_dict.items():
        word_i = voc.get(word)
        if word_i is not None:
            embedding_matrix[word_i] = coefs
            cnt_inv += 1
    print('Total %s word vectors.' % len(embedding_matrix))
    print('Words not found in embedding space %d' % (len(embedding_matrix)-cnt_inv))
 
    return embedding_matrix
# 
# pretrained_embeddings_path = root_dir + '/resources/glove.twitter.27B/glove.twitter.27B.%dd.txt' % hyperparams_features['embedding_dim']
# pretrained_embeddings_path = root_dir + '/resources/glove.840B/glove.840B.%dd.txt' % hyperparams_features['embedding_dim']
pretrained_embeddings_path = hyperparams_features['embeddings_path']#root_dir + '/eRisk/finetuned_glove_clpsych_erisk_normalized_2_20000.pkl'
embedding_matrix = load_embeddings(pretrained_embeddings_path, hyperparams_features['embedding_dim'], 
                                    voc=vocabulary_dict)


In [ ]:
embedding_matrix.mean()
# hyperparams_features

## Define model

In [ ]:
class Metrics():
    def __init__(self, threshold=0.5):
        self.threshold=threshold
        
    def recall_m(self, y_true, y_pred):
            y_labels = y_true
            y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), self.threshold), K.floatx())        
            possible_positives = K.sum(K.round(K.clip(y_labels, 0, 1)))
            true_positives = K.sum(K.round(K.clip(y_labels * y_pred, 0, 1)))
            recall = true_positives / (possible_positives + K.epsilon())
            return recall

    def precision_m(self, y_true, y_pred):
            y_labels = y_true
            y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), self.threshold), K.floatx())        
            true_positives = K.sum(K.round(K.clip(y_labels * y_pred, 0, 1)))
            predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
            precision = true_positives / (predicted_positives + K.epsilon())
            return precision

    def f1_m(self, y_true, y_pred):
        precision = self.precision_m(y_true, y_pred)
        recall = self.recall_m(y_true, y_pred)
        return 2*((precision*recall)/(precision+recall+K.epsilon()))
    
    def auc2(self, y_true, y_pred):
        auc = tf.metrics.auc(y_true, y_pred)[1]
        K.get_session().run(tf.local_variables_initializer())
        return auc

    def auc(self, y_true, y_pred):
#         has_true_examples = K.greater(K.cast(K.sum(y_true), K.floatx()),0)
#         has_false_examples = K.less(K.cast(K.mean(y_true), K.floatx()),1)
#         score = tf.cond(tf.logical_and(has_true_examples, has_false_examples), 
#                         lambda:tf.py_function(roc_auc_score, (
#                             K.cast(y_true, K.floatx()), 
#                             K.cast(y_pred, K.floatx())), tf.float32), 
#                         lambda:0.0)
        return 0
        
def binary_crossentropy_custom(y_true, y_pred):
    y_labels = y_true
    return K.binary_crossentropy(y_labels, 
                                 y_pred)



In [ ]:
class BertLayer(tf.keras.layers.Layer):
    def __init__(
        self,
        n_fine_tune_layers=10,
        pooling="first",
        trainable=True,
        bert_path="https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1", 
        **kwargs
    ):
        self.n_fine_tune_layers = n_fine_tune_layers
        self.trainable = trainable
        self.output_size = 768
        self.pooling = pooling
        self.bert_path = bert_path
        if self.pooling not in ["first", "mean"]:
            raise NameError(
               "Undefined pooling type (must be either first or mean, but is %s)" % self.pooling
            )

        super(BertLayer, self).__init__(**kwargs)
        
    def get_config(self):

        config = super(BertLayer, self).get_config().copy()
        config.update({
            'n_fine_tune_layers': self.n_fine_tune_layers,
            'trainable': self.trainable,
            'output_size': self.output_size,
            'pooling': self.pooling,
            'bert_path': self.bert_path,
        })

        return config

    def build(self, input_shape):
        self.bert = hub.Module(
            self.bert_path, trainable=self.trainable, name="%s_module" % self.name
        )

        # Remove unused layers
        trainable_vars = self.bert.variables
        if self.pooling == "first":
            trainable_vars = [var for var in trainable_vars if not "/cls/" in var.name]
            trainable_layers = ["pooler/dense"]

        elif self.pooling == "mean":
            trainable_vars = [
                var
                for var in trainable_vars
                if not "/cls/" in var.name and not "/pooler/" in var.name
            ]
            trainable_layers = []
        else:
            raise NameError(
                "Undefined pooling type (must be either first or mean, but is %s)" % self.pooling
            )

        # Select how many layers to fine tune
        for i in range(self.n_fine_tune_layers):
            trainable_layers.append("encoder/layer_%s" % str(11 - i))

        # Update trainable vars to contain only the specified layers
        trainable_vars = [
            var
            for var in trainable_vars
            if any([l in var.name for l in trainable_layers])
        ]

        # Add to trainable weights
        for var in trainable_vars:
            self._trainable_weights.append(var)

        for var in self.bert.variables:
            if var not in self._trainable_weights:
                self._non_trainable_weights.append(var)

        super(BertLayer, self).build(input_shape)

    def call(self, inputs):
        inputs = [K.cast(x, dtype="int32") for x in inputs]
        input_ids, input_mask, segment_ids = inputs
        bert_inputs = dict(
            input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids
        )
        if self.pooling == "first":
            pooled = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)[
                "pooled_output"
            ]
        elif self.pooling == "mean":
            result = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)[
                "sequence_output"
            ]

            mul_mask = lambda x, m: x * tf.expand_dims(m, axis=-1)
            masked_reduce_mean = lambda x, m: tf.reduce_sum(mul_mask(x, m), axis=1) / (
                    tf.reduce_sum(m, axis=1, keepdims=True) + 1e-10)
            input_mask = tf.cast(input_mask, tf.float32)
            pooled = masked_reduce_mean(result, input_mask)
        else:
            raise NameError("Undefined pooling type (must be either first or mean, but is %s)" % self.pooling)

        return pooled

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_size)

In [ ]:
def build_model(hyperparams, hyperparams_features, embedding_matrix, emotions, stopwords_list,
                liwc_categories,
               ignore_layer=[], classes=1):
    def attention(xin):
        return K.sum(xin, axis=1) 
    tokens_features = Input(shape=(hyperparams['maxlen'],), name='word_seq')
    embedding_layer = Embedding(hyperparams_features['max_features'], 
                                hyperparams_features['embedding_dim'], 
                                input_length=hyperparams['maxlen'],
                                embeddings_regularizer=regularizers.l2(hyperparams['l2_embeddings']),
                                weights=[embedding_matrix], 
                                trainable=hyperparams['trainable_embeddings'],
                               name='embeddings_layer')(
        tokens_features)
#     if 'batchnorm' not in ignore_layer:
#         embedding_layer_norm = BatchNormalization(axis=1, momentum=hyperparams['norm_momentum'],
#                                                      name='embeddings_layer_norm')(embedding_layer)
    embedding_layer = Dropout(hyperparams['dropout'], name='embedding_dropout')(embedding_layer)

    if 'lstm' not in hyperparams['ignore_layer']:
        if False: #tf.test.is_gpu_available():
            lstm_layers = Bidirectional(CuDNNLSTM(
                            hyperparams['lstm_units'], 
                                    return_sequences='attention' not in hyperparams['ignore_layer'], # only True if using attention
                          name='LSTM_layer'), name='bilstm')(embedding_layer)
        else:
            lstm_layers = Bidirectional(LSTM(hyperparams['lstm_units'], 
                               return_sequences='attention' not in ignore_layer,
                          name='LSTM_layer'), name='bilstm')(embedding_layer)
#             lstm_layers = LSTM(hyperparams['lstm_units'], 
#                                return_sequences='attention' not in hyperparams['ignore_layer'],
#                           name='LSTM_layer')(embedding_layer)
            
    elif 'cnn' not in hyperparams['ignore_layer']:
        cnn_layers = Conv1D(hyperparams['filters'],
                             hyperparams['kernel_size'],
                             padding='valid',
                             activation='relu',
                             strides=1, name='convolution')(embedding_layer)
        # we use max pooling:
        cnn_layers = GlobalMaxPooling1D()(cnn_layers)
    
    # Attention
    if 'attention' not in hyperparams['ignore_layer']:
        attention = Dense(1, activation='tanh', name='attention')(lstm_layers)
        attention = Flatten()(attention)
        attention = Activation('softmax')(attention)
        attention = RepeatVector(hyperparams['lstm_units']*2)(attention)
        attention = Permute([2, 1])(attention)

        sent_representation = Multiply()([lstm_layers, attention])
        sent_representation = Lambda(lambda xin: K.sum(xin, axis=1), 
                                     output_shape=(hyperparams['lstm_units'],)
                                    )(sent_representation)
#         sent_representation = Lambda(attention, 
#                                          output_shape=(hyperparams['lstm_units'],
#                                         ))(sent_representation)

        
    elif 'lstm' not in hyperparams['ignore_layer']:
        sent_representation = lstm_layers
    elif 'cnn' not in hyperparams['ignore_layer']:
        sent_representation = cnn_layers
    else:
        sent_representation = None
        
    if sent_representation is not None:
        sent_representation = Dropout(hyperparams['dropout'], name='lstm_att_dropout')(sent_representation)
        if hyperparams['dense_sentence_units']:
            sent_representation = Dense(units=hyperparams['dense_sentence_units'], activation='relu',
                                       name='dense_sent_representation')(sent_representation)
    
    # Other features
    numerical_features = Input(shape=(len(emotions) + 1 + len(liwc_categories),), name='numeric_input') # emotions and pronouns
    sparse_features = Input(shape=(len(stopwords_list),), name='sparse_input') # stopwords

    dense_layer_sparse = Dense(units=hyperparams['dense_bow_units'],
                              name='sparse_feat_dense_layer', activation='relu',
                                kernel_regularizer=regularizers.l2(hyperparams['l2_dense']),
                              )(sparse_features)
    
    # BERT encoder
    if 'bert_layer' not in hyperparams['ignore_layer']:
        in_id_bert = Input(shape=(hyperparams['maxlen'],), name="input_ids_bert")
        in_mask_bert = Input(shape=(hyperparams['maxlen'],), name="input_masks_bert")
        in_segment_bert = Input(shape=(hyperparams['maxlen'],), name="segment_ids_bert")
        bert_inputs = [in_id_bert, in_mask_bert, in_segment_bert]

        bert_output = BertLayer(n_fine_tune_layers=hyperparams['bert_finetune_layers'], 
                                pooling=hyperparams['bert_pooling'],
                               trainable=hyperparams['bert_trainable'],
                               name='bert_layer')(bert_inputs)
        dense_bert = Dense(hyperparams['bert_dense_units'], activation='relu',
                           kernel_regularizer=regularizers.l2(hyperparams['l2_bert']),
                          name='bert_dense_layer')(bert_output)
    else:
        dense_bert = None


    
    # Batch normalization
    if 'batchnorm' not in hyperparams['ignore_layer']:
        numerical_features_norm = BatchNormalization(axis=1, momentum=hyperparams['norm_momentum'],
                                                     name='numerical_features_norm')(numerical_features)
        sent_representation_norm = BatchNormalization(axis=1, momentum=hyperparams['norm_momentum'],
                                                      name='sent_repr_norm')(sent_representation)
        dense_layer_sparse_norm = BatchNormalization(axis=1, momentum=hyperparams['norm_momentum'],
                                                     name='sparse_features_norm')(dense_layer_sparse)
        if 'bert_layer' not in hyperparams['ignore_layer']:
            if hyperparams['bert_dense_units']:
                dense_bert_norm = BatchNormalization(axis=1, momentum=hyperparams['norm_momentum'],
                                                         name='bert_layer_norm')(dense_bert)
            else:
                dense_bert_norm = BatchNormalization(axis=1, momentum=hyperparams['norm_momentum'],
                                                         name='bert_layer_norm')(bert_output)
        else:
            dense_bert_norm = None
        
#     subjects = Input(shape=(1,), name='subjects')
    

    all_layers = {
        'user_encoded': sent_representation,
        'numerical_dense_layer': numerical_features,
        'sparse_feat_dense_layer': dense_layer_sparse,
        'bert_layer': dense_bert if hyperparams['bert_dense_units'] else bert_output,
    }
    if 'batchnorm' not in hyperparams['ignore_layer']:
        all_layers = {
            'user_encoded': sent_representation_norm,
            'numerical_dense_layer': numerical_features_norm,
            'sparse_feat_dense_layer': dense_layer_sparse_norm,
            'bert_layer': dense_bert_norm
        }
    layers_to_merge = []
    for n, l in all_layers.items():
        if n in hyperparams['ignore_layer'] or l is None:
            continue
        layers_to_merge.append(l)
        
    if len(layers_to_merge) == 1:
        merged_layers = layers_to_merge[0]
    else:
        merged_layers = concatenate(layers_to_merge)
    output_layer = Dense(classes, activation='sigmoid' if classes==1 else 'softmax',
                         name='output_layer',
                        kernel_regularizer=regularizers.l2(hyperparams['l2_dense']),
                        )(merged_layers)

    # Compile model
    if 'bert_layer' not in hyperparams['ignore_layer']:
        inputs=[tokens_features, numerical_features, sparse_features, 
                              in_id_bert, in_mask_bert, in_segment_bert,
    #                           subjects
                             ]
    else:
        inputs=[tokens_features, numerical_features, sparse_features]
    model = Model(inputs=inputs, outputs=output_layer)
        

    metrics_class = Metrics(threshold=hyperparams['threshold'])
    model.compile(hyperparams['optimizer'], binary_crossentropy_custom,
                  metrics=[metrics_class.f1_m, metrics_class.precision_m, metrics_class.recall_m,
                          metrics_class.auc, 
                           tf.keras.metrics.AUC(name="AUC_keras"), 
                           tf.keras.metrics.FalsePositives(), 
                           tf.keras.metrics.FalseNegatives()])

    return model



In [ ]:
def build_tl_model(hyperparams, hyperparams_features, embedding_matrix, emotions, stopwords_list,
                liwc_categories,
               ignore_layer=[]):
    def attention(xin):
        return K.sum(xin, axis=1) 
    tokens_features = Input(shape=(hyperparams['maxlen'],), name='word_seq')
    embedding_layer = Embedding(hyperparams_features['max_features'], 
                                hyperparams_features['embedding_dim'], 
                                input_length=hyperparams['maxlen'],
                                embeddings_regularizer=regularizers.l2(hyperparams['l2_embeddings']),
                                weights=[embedding_matrix], 
                                trainable=False,
                               name='embeddings_layer')(
        tokens_features)
#     if 'batchnorm' not in ignore_layer:
#         embedding_layer_norm = BatchNormalization(axis=1, momentum=hyperparams['norm_momentum'],
#                                                      name='embeddings_layer_norm')(embedding_layer)
    embedding_layer = Dropout(hyperparams['dropout'], name='embedding_dropout')(embedding_layer)

    if 'lstm' not in ignore_layer:
        if False: #tf.test.is_gpu_available():
            lstm_layers = Bidirectional(CuDNNLSTM(hyperparams['lstm_units'], trainable=False,
                                    return_sequences='attention' not in ignore_layer, # only True if using attention
                          name='LSTM_layer'), name='bilstm')(embedding_layer)
        else:
            lstm_layers = Bidirectional(LSTM(hyperparams['lstm_units'], trainable=False,
                               return_sequences='attention' not in ignore_layer,
                          name='LSTM_layer'), name='bilstm')(embedding_layer)
            
    elif 'cnn' not in ignore_layer:
        cnn_layers = Conv1D(hyperparams['filters'],
                             hyperparams['kernel_size'],
                             padding='valid',
                             activation='relu',
                             strides=1,
                           trainable=False,
                           name='convolution')(embedding_layer)
        # we use max pooling:
        cnn_layers = GlobalMaxPooling1D(trainable=False)(cnn_layers)
    
    # Attention
    if 'attention' not in ignore_layer:
        attention = Dense(1, activation='tanh', name='attention', trainable=False)(lstm_layers)
        attention = Flatten()(attention)
        attention = Activation('softmax')(attention)
        attention = RepeatVector(hyperparams['lstm_units']*2)(attention)
        attention = Permute([2, 1])(attention)

        sent_representation = Multiply()([lstm_layers, attention])
        sent_representation = Lambda(lambda xin: K.sum(xin, axis=1), 
                                     output_shape=(hyperparams['lstm_units'],)
                                    )(sent_representation)
#         sent_representation = Lambda(attention, 
#                                          output_shape=(hyperparams['lstm_units'],
#                                         ))(sent_representation)

        
    elif 'lstm' not in ignore_layer:
        sent_representation = lstm_layers
    elif 'cnn' not in ignore_layer:
        sent_representation = cnn_layers
        
    
    sent_representation = Dropout(hyperparams['dropout'], name='lstm_att_dropout')(sent_representation)
    if hyperparams['dense_sentence_units']:
        sent_representation = Dense(units=hyperparams['dense_sentence_units'], activation='relu',
                                   name='dense_sent_representation',
                                   trainable=False)(sent_representation)
    
    # Other features
    numerical_features = Input(shape=(len(emotions) + 1 + len(liwc_categories),), name='numeric_input') # emotions and pronouns
    sparse_features = Input(shape=(len(stopwords_list),), name='sparse_input') # stopwords

    dense_layer_sparse = Dense(units=hyperparams['dense_bow_units'],
                              name='sparse_feat_dense_layer', activation='relu',
                                kernel_regularizer=regularizers.l2(hyperparams['l2_dense']),
                               trainable=False,
                              )(sparse_features)
    
    # BERT encoder
    if 'bert_layer' not in hyperparams['ignore_layer']:
        in_id_bert = Input(shape=(hyperparams['maxlen'],), name="input_ids_bert")
        in_mask_bert = Input(shape=(hyperparams['maxlen'],), name="input_masks_bert")
        in_segment_bert = Input(shape=(hyperparams['maxlen'],), name="segment_ids_bert")
        bert_inputs = [in_id_bert, in_mask_bert, in_segment_bert]

        bert_output = BertLayer(n_fine_tune_layers=hyperparams['bert_finetune_layers'], 
                                pooling=hyperparams['bert_pooling'],
                               trainable=False,
                               name='bert_layer')(bert_inputs)
        dense_bert = Dense(hyperparams['bert_dense_units'], activation='relu',
                           kernel_regularizer=regularizers.l2(hyperparams['l2_bert']),
                          name='bert_dense_layer',
                          trainable=False)(bert_output)
    else:
        dense_bert = None


    
    # Batch normalization
    if 'batchnorm' not in ignore_layer:
        numerical_features_norm = BatchNormalization(axis=1, momentum=hyperparams['norm_momentum'],
                                                     name='numerical_features_norm')(numerical_features)
        sent_representation_norm = BatchNormalization(axis=1, momentum=hyperparams['norm_momentum'],
                                                      name='sent_repr_norm')(sent_representation)
        dense_layer_sparse_norm = BatchNormalization(axis=1, momentum=hyperparams['norm_momentum'],
                                                     name='sparse_features_norm')(dense_layer_sparse)
        if 'bert_layer' not in hyperparams['ignore_layer']:
            if hyperparams['bert_dense_units']:
                dense_bert_norm = BatchNormalization(axis=1, momentum=hyperparams['norm_momentum'],
                                                         name='bert_layer_norm')(dense_bert)
            else:
                dense_bert_norm = BatchNormalization(axis=1, momentum=hyperparams['norm_momentum'],
                                                         name='bert_layer_norm')(bert_output)
        else:
            dense_bert_norm = None
        
#     subjects = Input(shape=(1,), name='subjects')
    

    all_layers = {
        'user_encoded': sent_representation,
        'numerical_dense_layer': numerical_features,
        'sparse_feat_dense_layer': dense_layer_sparse,
        'bert_layer': dense_bert if hyperparams['bert_dense_units'] else bert_output,
    }
    if 'batchnorm' not in ignore_layer:
        all_layers = {
            'user_encoded': sent_representation_norm,
            'numerical_dense_layer': numerical_features_norm,
            'sparse_feat_dense_layer': dense_layer_sparse_norm,
            'bert_layer': dense_bert_norm
        }
    layers_to_merge = []
    for n, l in all_layers.items():
        if n in ignore_layer:
            continue
        layers_to_merge.append(l)
        
    if len(layers_to_merge) == 1:
        merged_layers = layers_to_merge[0]
    else:
        merged_layers = concatenate(layers_to_merge)
        
    TL_layer = Dense(hyperparams['transfer_units'], activation='sigmoid',
                         name='tl_layer',
                        kernel_regularizer=regularizers.l2(hyperparams['l2_dense']),
                         trainable=True
                        )(merged_layers)
    output_layer = Dense(1, activation='sigmoid',
                         name='output_layer_tl',
                        kernel_regularizer=regularizers.l2(hyperparams['l2_dense']),
                         trainable=True,
                        )(TL_layer)

    # Compile model
    if 'bert_layer' not in hyperparams['ignore_layer']:
        inputs=[tokens_features, numerical_features, sparse_features, 
                              in_id_bert, in_mask_bert, in_segment_bert,
    #                           subjects
                             ]
    else:
        inputs=[tokens_features, numerical_features, sparse_features]
    model = Model(inputs=inputs, outputs=output_layer)
    model.summary()


    if classes==1:
        metrics_class = Metrics(threshold=hyperparams['threshold'])
        model.compile(hyperparams['optimizer'], binary_crossentropy_custom,
                      metrics=[metrics_class.f1_m, metrics_class.precision_m, metrics_class.recall_m,
                              metrics_class.auc, 
                               tf.keras.metrics.AUC(name="AUC_keras"), 
                               tf.keras.metrics.FalsePositives(), 
                               tf.keras.metrics.FalseNegatives()])
    else:
        model.compile(hyperparams['optimizer'], K.categorical_crossentropy,
                     metrics=[tf.keras.metrics.CategoricalAccuracy(name='cat_acc'),
#                              tf.keras.metrics.Precision(name='prec'), tf.keras.metrics.Recall(name='rec'), 
                              ])

    return model

In [ ]:
def build_hierarchical_model(hyperparams, hyperparams_features, embedding_matrix, emotions, stopwords_list,
                liwc_categories,
               ignore_layer=[], activations=None, classes=1):
    def attention(xin):
        return K.sum(xin, axis=1) 


    # Post/sentence representation - word sequence
    tokens_features = Input(shape=(hyperparams['maxlen'],), name='word_seq')
    embedding_layer = Embedding(hyperparams_features['max_features'], 
                                hyperparams_features['embedding_dim'], 
                                input_length=hyperparams['maxlen'],
                                embeddings_regularizer=regularizers.l2(hyperparams['l2_embeddings']),
                                weights=[embedding_matrix], 
                                trainable=hyperparams['trainable_embeddings'],
                               name='embeddings_layer')(
        tokens_features)
    embedding_layer = Dropout(hyperparams['dropout'], name='embedding_dropout')(embedding_layer)

    
    if 'lstm' not in ignore_layer:
        if False: #tf.test.is_gpu_available():
            lstm_layers = CuDNNLSTM(hyperparams['lstm_units'], 
                                    return_sequences='attention' not in ignore_layer, # only True if using attention
                          name='LSTM_layer')(embedding_layer)
        else:
            lstm_layers = LSTM(hyperparams['lstm_units'], 
                               return_sequences='attention' not in ignore_layer,
                          name='LSTM_layer')(embedding_layer)

        # Attention
        if 'attention' not in ignore_layer:
            attention_layer = Dense(1, activation='tanh', name='attention')
            attention = attention_layer(lstm_layers)
            attention = Flatten()(attention)
            attention_output = Activation('softmax')(attention)
            attention = RepeatVector(hyperparams['lstm_units'])(attention_output)
            attention = Permute([2, 1])(attention)

            sent_representation = Multiply()([lstm_layers, attention])
            sent_representation = Lambda(lambda xin: K.sum(xin, axis=1), 
                                     output_shape=(hyperparams['lstm_units'],)
                                    )(sent_representation)

#             sent_representation = Lambda(attention, 
#                                          output_shape=(hyperparams['lstm_units'],
#                                         ))(sent_representation)
        else:
            sent_representation = lstm_layers

    elif 'cnn' not in ignore_layer:
        cnn_layers = Conv1D(hyperparams['filters'],
                             hyperparams['kernel_size'],
                             padding='valid',
                             activation='relu',
                             strides=1)(embedding_layer)
        # we use max pooling:
        sent_representation = GlobalMaxPooling1D()(cnn_layers)
    
    
    if 'batchnorm' not in ignore_layer:
        sent_representation = BatchNormalization(axis=1, momentum=hyperparams['norm_momentum'],
                                                          name='sent_repr_norm')(sent_representation)
    sent_representation = Dropout(hyperparams['dropout'], name='sent_repr_dropout')(sent_representation)

            # Other features 
    numerical_features_history = Input(shape=(
            hyperparams['posts_per_group'],
            len(emotions) + 1 + len(liwc_categories)
        ), name='numeric_input_hist') # emotions and pronouns
    sparse_features_history = Input(shape=(
            hyperparams['posts_per_group'],
            len(stopwords_list)
        ), name='sparse_input_hist') # stopwords



    if activations == 'attention':
#         sent_representation = Flatten()(attention_layer.output)
        sent_representation = attention_output


    posts_history_input = Input(shape=(hyperparams['posts_per_group'], 
                                     hyperparams['maxlen']
                                          ), name='hierarchical_word_seq_input')

    # Hierarchy
    sentEncoder = Model(inputs=tokens_features, 
                        outputs=sent_representation)
    sentEncoder.summary()

    user_encoder = TimeDistributed(sentEncoder, name='user_encoder')(posts_history_input)



    
    if activations != 'attention':
        
        
        # BERT encoder
        if 'bert_layer' not in hyperparams['ignore_layer']:
            in_id_bert = Input(shape=(hyperparams['maxlen'],), name="input_ids_bert")
            in_mask_bert = Input(shape=(hyperparams['maxlen'],), name="input_masks_bert")
            in_segment_bert = Input(shape=(hyperparams['maxlen'],), name="segment_ids_bert")
            bert_inputs = [in_id_bert, in_mask_bert, in_segment_bert]

            bert_output = BertLayer(n_fine_tune_layers=hyperparams['bert_finetune_layers'], 
                                    pooling=hyperparams['bert_pooling'],
                                   trainable=hyperparams['bert_trainable'],
                                   name='bert_layer')(bert_inputs)
            dense_bert = Dense(hyperparams['bert_dense_units'], 
                               activation='relu',
                              kernel_regularizer=regularizers.l2(hyperparams['l2_dense']),
                              name='bert_dense_layer')(bert_output)

            bertSentEncoder = Model(bert_inputs, dense_bert)


            in_id_bert_history = Input(shape=(hyperparams['posts_per_group'],
                                                              hyperparams['maxlen'],), name="input_ids_bert_hist")
            in_mask_bert_history = Input(shape=(hyperparams['posts_per_group'],
                                                                hyperparams['maxlen'],), name="input_masks_bert_hist")
            in_segment_bert_history = Input(shape=(hyperparams['posts_per_group'],
                                                                   hyperparams['maxlen'],), name="segment_ids_bert_hist")
            bert_inputs_history = [in_id_bert_history, in_mask_bert_history, in_segment_bert_history]
            bert_inputs_concatenated = concatenate(bert_inputs_history)
            inputs_indices = [hyperparams['maxlen']*i for i in range(3)]
            # slice the input in equal slices on the last dimension
            bert_encoder_layer = TimeDistributed(Lambda(lambda x: bertSentEncoder([x[:,inputs_indices[0]:inputs_indices[1]], 
                                                                          x[:,inputs_indices[1]:inputs_indices[2]],
                                                                                  x[:,inputs_indices[2]:]])),
                                                name='bert_distributed_layer')(
                                bert_inputs_concatenated)
            bertUserEncoder = Model(bert_inputs_history, bert_encoder_layer)
            bertUserEncoder.summary()

            bert_user_encoder = bertUserEncoder(bert_inputs_history)
        else:
            bert_user_encoder = None


        dense_layer_sparse = Dense(units=hyperparams['dense_bow_units'],
                                  name='sparse_feat_dense_layer', activation='relu',
                                    kernel_regularizer=regularizers.l2(hyperparams['l2_dense']),
                                  )
        dense_layer_sparse_user = TimeDistributed(dense_layer_sparse,
                                                 name='sparse_dense_layer_user')(sparse_features_history)
        
                
        dense_layer_numerical = Dense(units=hyperparams['dense_numerical_units'],
                                  name='numerical_feat_dense_layer', activation='relu',
                                    kernel_regularizer=regularizers.l2(hyperparams['l2_dense']),
                                  )
        dense_layer_numerical_user = TimeDistributed(dense_layer_numerical,
                                                 name='numerical_dense_layer_user')(numerical_features_history)


        # Concatenate features
        if 'batchnorm' not in ignore_layer:
#             numerical_features_history_norm = BatchNormalization(axis=1, momentum=hyperparams['norm_momentum'],
#                                                          name='numerical_features_norm')(numerical_features_history)
            dense_layer_numerical_user = BatchNormalization(axis=1, momentum=hyperparams['norm_momentum'],
                                                         name='numerical_features_norm')(dense_layer_numerical_user)
            dense_layer_sparse_user = BatchNormalization(axis=1, momentum=hyperparams['norm_momentum'],
                                                         name='sparse_features_norm')(dense_layer_sparse_user)
        all_layers = {
            'user_encoded': user_encoder,
            'bert_layer': bert_user_encoder,
#             'numerical_dense_layer': numerical_features_history if 'batchnorm' in ignore_layer \
#                         else numerical_features_history_norm
            'numerical_dense_layer': dense_layer_numerical_user,

            'sparse_feat_dense_layer': dense_layer_sparse_user,
        }

        layers_to_merge = [l for n,l in all_layers.items() if n not in ignore_layer]
        if len(layers_to_merge) == 1:
            merged_layers = layers_to_merge[0]
        else:
            merged_layers = concatenate(layers_to_merge)

        if 'lstm_user' not in ignore_layer:

            if False:#tf.test.is_gpu_available():
                lstm_user_layers = CuDNNLSTM(hyperparams['lstm_units_user'], 
                                        return_sequences='attention_user' not in ignore_layer, # only True if using attention
                              name='LSTM_layer_user')(merged_layers)
            else:
                lstm_user_layers = LSTM(hyperparams['lstm_units_user'], 
                                   return_sequences='attention_user' not in ignore_layer,
                              name='LSTM_layer_user')(merged_layers)

            # Attention
            if 'attention_user' not in ignore_layer:
                attention_user_layer = Dense(1, activation='tanh', name='attention_user')
                attention_user = attention_user_layer(lstm_user_layers)
                attention_user = Flatten()(attention_user)
                attention_user_output = Activation('softmax')(attention_user)
                attention_user = RepeatVector(hyperparams['lstm_units_user'])(attention_user_output)
                attention_user = Permute([2, 1])(attention_user)

                user_representation = Multiply()([lstm_user_layers, attention_user])
                user_representation = Lambda(lambda xin: K.sum(xin, axis=1), 
                                             output_shape=(hyperparams['lstm_units_user'],))(user_representation)
    #             user_representation = Lambda(attention, 
    #                                          output_shape=(hyperparams['lstm_units_user'],
    #                                         ))(user_representation)
            else:
                user_representation = lstm_user_layers


        elif 'cnn_user' not in ignore_layer:
            cnn_layers_user = Conv1D(hyperparams['filters_user'],
                                 hyperparams['kernel_size_user'],
                                 padding='valid',
                                 activation='relu',
                                 strides=1)(merged_layers)
            # we use max pooling:
            user_representation = GlobalMaxPooling1D()(cnn_layers_user)
    #         user_representation = Flatten()(user_representation)


        user_representation = Dropout(hyperparams['dropout'], name='user_repr_dropout')(user_representation)


        if hyperparams['dense_user_units']:
            user_representation = Dense(units=hyperparams['dense_user_units'], activation='relu',
                                       name='dense_user_representation')(user_representation)

        output_layer = Dense(classes, activation='sigmoid' if classes==1 else 'softmax',
                             name='output_layer',
                            kernel_regularizer=regularizers.l2(hyperparams['l2_dense']),
                             input_shape=(hyperparams['lstm_units_user'],),
                            )(user_representation)

    # Compile model

#     elif activations == 'attention':
#         outputs = attention_layer.output
    if activations == 'attention':
        outputs = user_encoder

        
    elif activations == 'attention_user':
#         outputs = attention_user.output
        outputs = attention_user_output
    elif activations == 'output_layer':
        outputs = user_representation

    
    else:
        outputs = output_layer
    if 'bert_layer' not in hyperparams['ignore_layer']:

        hierarchical_model = Model(inputs=[posts_history_input, 
                                       numerical_features_history, sparse_features_history,
                                      in_id_bert_history, in_mask_bert_history, in_segment_bert_history], 
                  outputs=outputs)
    else:
        hierarchical_model = Model(inputs=[posts_history_input, 
                                       numerical_features_history, sparse_features_history,
                                      ], 
                  outputs=outputs)
    hierarchical_model.summary()
    
    if classes==1:
        metrics_class = Metrics(threshold=hyperparams['threshold'])
        hierarchical_model.compile(hyperparams['optimizer'], binary_crossentropy_custom,
                      metrics=[metrics_class.f1_m, metrics_class.precision_m, metrics_class.recall_m,
                              metrics_class.auc])
    else:
        
        hierarchical_model.compile(hyperparams['optimizer'], K.categorical_crossentropy,
                     metrics=[ tf.keras.metrics.CategoricalAccuracy(name='cat_acc'),
#                              tf.keras.metrics.Precision(name='prec'), tf.keras.metrics.Recall(name='rec'), 
                              ])
    return hierarchical_model



In [ ]:
# def build_tl_model(pretrained_model, hyperparams, units=100,  emotions=emotions, stopwords_list=stopword_list,
#                 liwc_categories=liwc_categories):
#     tokens_features = Input(shape=(hyperparams['maxlen'],), name='word_seq_tl')
#     numerical_features = Input(shape=(len(emotions) + 1 + len(liwc_categories),), name='numeric_input_tl') # emotions and pronouns
#     sparse_features = Input(shape=(len(stopwords_list),), name='sparse_input_tl') # stopwords
#     pretrained_model.summary()
#     print("inputs", tokens_features, numerical_features, sparse_features)
#     pretrained_output = pretrained_model(inputs=[tokens_features, numerical_features, sparse_features])
#     # TODO: set trainable to false
#     tl_layer = Dense(units=units, activation='relu',
#                                    name='tl_layer', trainable=False,
#                         kernel_regularizer=regularizers.l2(hyperparams['l2_dense'])
#                     )(pretrained_output)
#     output_layer = Dense(1, activation='sigmoid',
#                          name='tl_output_layer',
#                         kernel_regularizer=regularizers.l2(hyperparams['l2_dense'])
#                         )(tl_layer)
# #     pretrained_model.outputs = output_layer ## I DUNNO
#     tl_model = Model(inputs=[tokens_features, numerical_features, sparse_features], 
#           outputs=output_layer)
#     metrics_class = Metrics(threshold=hyperparams['threshold'])
#     tl_model.compile(hyperparams['optimizer'], binary_crossentropy_custom,
#                   metrics=[metrics_class.f1_m, metrics_class.precision_m, metrics_class.recall_m,
#                           metrics_class.auc, 
#                            tf.keras.metrics.AUC(name="AUC_keras"), 
#                            tf.keras.metrics.FalsePositives(), 
#                            tf.keras.metrics.FalseNegatives()])

#     return tl_model

In [ ]:
# model = build_model(hyperparams, hyperparams_features, embedding_matrix, emotions, stopword_list,
#                     liwc_categories=[c for c in categories if c in writings_df.columns]
# ,
#                    ignore_layer=hyperparams['ignore_layer'])
# model.summary()

In [ ]:
# hierarchical_model = build_hierarchical_model(hyperparams, hyperparams_features, embedding_matrix, emotions, stopword_list,
#                     liwc_categories=[c for c in categories if c in writings_df.columns]
# ,
#                    ignore_layer=hyperparams['ignore_layer'])

In [ ]:
# plot_model(model, 'models/sequential_bert_model.png')

In [ ]:
# initialize_vars(sess)

## Train

In [ ]:
class WeightsHistory(callbacks.Callback):
    def __init__(self, logs={}):
        super(WeightsHistory, self).__init__()
    def on_train_begin(self, logs={}):
        self.log_weights(0)
    def on_epoch_end(self, epoch, logs={}):
        if epoch % 10 == 0:
            self.log_weights(epoch)
    def log_weights(self, step):
        for layer in self.model.layers:
            try:
                experiment.log_histogram_3d(layer.get_weights()[0], 
                                            name=layer.name + "_weight", step=step)
            except Exception as e:
#                 logger.debug("Logging weights error: " + layer.name + "; " + str(e) + "\n")
                # Layer probably does not exist
                pass

class OutputsHistory(callbacks.Callback):
    def __init__(self, logs={}, generator=None, generator_type=""):
        super(OutputsHistory, self).__init__()
        self.generator_type = generator_type
        if generator:
            self.generator = generator
        elif generator_type:
            self.generator = DataGenerator(user_level_data, subjects_split, 
                                     set_type=generator_type, 
                                   hierarchical=hyperparams['hierarchical'],
                                seq_len=hyperparams['maxlen'], batch_size=hyperparams['batch_size'],
                                     max_posts_per_user=None,
                                   pad_with_duplication=False,
                                    posts_per_group=hyperparams['posts_per_group'],
                                    post_groups_per_user=None, 
                                           liwc_words_for_categories=liwc_words_for_categories,
                                           compute_liwc=True,
                                         emotions=emotions, liwc_categories=liwc_categories,
                                     sample_seqs=False, shuffle=False)
    def on_train_begin(self, logs={}):
        self.log_outputs(0)
    def on_epoch_end(self, epoch, logs={}):
        if epoch % 2 == 0:
            self.log_outputs(epoch)
    def log_outputs(self, step):
        try:
            experiment.log_histogram_3d(self.model.predict(self.generator,  verbose=1, steps=2),
                                        name='output_%s' % self.generator_type, step=step)
        except Exception as e:
            logger.debug("Logging outputs error: " + str(e) + "\n")
#                 Layer probably does not exist
            pass

class ActivationsAttention(callbacks.Callback):
    def __init__(self, logs={}, generator=None, generator_type=""):
        super(ActivationsAttention, self).__init__()
        self.generator_type = generator_type
        if generator:
            self.generator = generator
        elif generator_type:
            self.generator = DataGenerator(user_level_data, subjects_split, 
                                     set_type=generator_type, 
                                   hierarchical=hyperparams['hierarchical'],
                                seq_len=hyperparams['maxlen'], batch_size=hyperparams['batch_size'],
                                     max_posts_per_user=None,
                                   pad_with_duplication=False,
                                    posts_per_group=hyperparams['posts_per_group'],
                                    post_groups_per_user=None, 
                                     sample_seqs=False, shuffle=False)
    def on_train_begin(self, logs={}):
        self.log_outputs(0)
    def on_epoch_end(self, epoch, logs={}):
        if epoch % 10 == 0:
            self.log_outputs(epoch)
    def log_outputs(self, step):
        try:
            experiment.log_histogram_3d(self.model.get_layer('attention_user').output.eval())
        except Exception as e:
            logger.debug("Logging activations error: " + str(e) + "\n")
            pass

class LRHistory(callbacks.Callback):
    def __init__(self, logs={}):
        super(LRHistory, self).__init__()
    
    def on_epoch_begin(self, epoch, logs={}):
        self.log_lr()
        
    def log_lr(self):
        lr = K.eval(self.model.optimizer.lr)
        logger.debug("Learning rate is %f...\n" % lr)
        experiment.log_parameter('lr', lr)

class FreezeLayer(callbacks.Callback):
    def __init__(self, logs={}, patience=5, layer={'user_encoder':'embeddings_layer'}, verbose=1, set_to=False):
        super(FreezeLayer, self).__init__()
        self.freeze_epoch = patience
        self.freeze_layer = layer
        self.verbose = verbose
        self.set_to = set_to

    def on_epoch_begin(self, epoch, logs={}):
        if type(self.freeze_layer)==dict:
            submodel = self.model.get_layer(list(self.freeze_layer.keys())[0])
        else:
            submodel = self.model
        logging.debug("Trainable embeddings", submodel.get_layer(self.freeze_layer).trainable)
        if epoch == self.freeze_epoch:
            try:
                layer = submodel.get_layer(self.freeze_layer)
                old_value = layer.trainable
                layer.trainable = self.set_to
                # TODO: does this reset the optimizer? should I also compile the top-level model?
                self.model.compile(hyperparams['optimizer'], binary_crossentropy_custom,
                  metrics=[metrics_class.f1_m, metrics_class.precision_m, metrics_class.recall_m])
                if self.verbose:
                    logging.debug("Setting %s layer from %s to trainable=%s...\n" % (layer.name, old_value,
                                                                   submodel.get_layer(self.freeze_layer).trainable))
            except Exception as e:
                # layer probably does not exist
                pass

In [ ]:
def train_model(model, hyperparams,
                data_generator_train, data_generator_valid,
                epochs, class_weight, start_epoch=0, workers=4,
                callback_list = [],
                
                model_path='/tmp/model',
                validation_set='valid',
               verbose=1):
    
    logger.info("Initializing callbacks...\n")
    # Initialize callbacks
    freeze_layer = FreezeLayer(patience=hyperparams['freeze_patience'], set_to=not hyperparams['trainable_embeddings'])
    weights_history = WeightsHistory()
    outputs_history_valid = OutputsHistory(generator_type=validation_set)
    outputs_history_train = OutputsHistory(generator_type='train')
    activations_history_train = ActivationsAttention(generator_type='train')
    lr_history = LRHistory()

    reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=hyperparams['reduce_lr_factor'],
                              patience=hyperparams['reduce_lr_patience'], min_lr=0.000001, verbose=1)
    lr_schedule = callbacks.LearningRateScheduler(lambda epoch, lr: 
                                                  lr if (epoch+1)%hyperparams['scheduled_reduce_lr_freq']!=0 else
                                                  lr*hyperparams['scheduled_reduce_lr_factor'], verbose=1)
    callbacks_dict = {'freeze_layer': freeze_layer, 'weights_history': weights_history,
           'outputs_history_valid': outputs_history_valid, 'outputs_history_train': outputs_history_train,
           'lr_history': lr_history,
            'activations': activations_history_train,
           'reduce_lr_plateau': reduce_lr,
            'lr_schedule': lr_schedule}

    
    logging.info('Train...')


    history = model.fit_generator(data_generator_train,
                steps_per_epoch=100,
              epochs=epochs, initial_epoch=start_epoch, 
              class_weight=class_weight,
              validation_data=data_generator_valid,
                        verbose=verbose,
#               validation_split=0.3,
                       workers=workers,
            callbacks = [
                callbacks.ModelCheckpoint(filepath='%s_best.h5' % model_path, verbose=1, 
                                          save_best_only=True, save_weights_only=True),
#                 callbacks.EarlyStopping(patience=hyperparams['early_stopping_patience'],
#                                        restore_best_weights=True)
            ] + [
                callbacks_dict[c] for c in [
#                     'weights_history', 
                    'outputs_history_valid', 
#                     'outputs_history_train', 
#                     'reduce_lr_plateau', 
#                     'lr_schedule', 
#                     'activations'
                ]])
    return model, history

In [ ]:
def get_network_type(hyperparams):
    if 'lstm' in hyperparams['ignore_layer']:
        network_type = 'cnn'
    else:
        network_type = 'lstm'
    if 'user_encoded' in hyperparams['ignore_layer']:
        if 'bert_layer' not in hyperparams['ignore_layer']:
            network_type = 'bert'
        else:
            network_type = 'extfeatures'
    if hyperparams['hierarchical']:
        hierarch_type = 'hierarchical'
    else:
        hierarch_type = 'seq'
    return network_type, hierarch_type

def initialize_experiment(hyperparams, nrc_lexicon_path, emotions, pretrained_embeddings_path, 
                          dataset_type, transfer_type):

    experiment = Experiment(api_key="eoBdVyznAhfg3bK9pZ58ZSXfv",
                            project_name="mental", workspace="ananana", disabled=False)

    experiment.log_parameters(hyperparams_features)

    experiment.log_parameter('emotion_lexicon', nrc_lexicon_path)
    experiment.log_parameter('emotions', emotions)
    experiment.log_parameter('embeddings_path', pretrained_embeddings_path)
    experiment.log_parameter('dataset_type', dataset_type)
    experiment.log_parameter('transfer_type', transfer_type)
    experiment.add_tag(dataset_type)
    experiment.log_parameters(hyperparams)
    network_type, hierarch_type = get_network_type(hyperparams)
    experiment.add_tag(network_type)
    experiment.add_tag(hierarch_type)
    
    return experiment
    

In [ ]:
def initialize_datasets(user_level_data, subjects_split, hyperparams, hyperparams_features, 
                        validation_set, emotions, liwc_categories, session=None, classes=1):
    liwc_words_for_categories = pickle.load(open(hyperparams_features['liwc_words_cached'], 'rb'))
    if classes!=1 and 'class_weights' in hyperparams:
        class_weights = hyperparams['class_weights']
    else:
        class_weights = None
    data_generator_train = DataGenerator(user_level_data, subjects_split, set_type='train',
                                        seq_len=hyperparams['maxlen'], batch_size=hyperparams['batch_size'],
                                        sample_seqs=hyperparams['sample_seqs'], sampling_distr=hyperparams['sampling_distr'],
                                        posts_per_group=hyperparams['posts_per_group'], post_groups_per_user=hyperparams['post_groups_per_user'],
                                        max_posts_per_user=hyperparams['posts_per_user'], 
                                         hierarchical=hyperparams['hierarchical'], 
                                         use_bert='bert_layer' not in hyperparams['ignore_layer'],
                                         compute_liwc=True, liwc_words_for_categories=liwc_words_for_categories,
                                         emotions=emotions, liwc_categories=liwc_categories,
                                        session=session, classes=classes, class_weights=class_weights)
    data_generator_valid = DataGenerator(user_level_data, subjects_split, set_type=validation_set,
                                        seq_len=hyperparams['maxlen'], batch_size=hyperparams['batch_size'],
                                        posts_per_group=hyperparams['posts_per_group'], 
                                         post_groups_per_user=1,#hyperparams['post_groups_per_user'],
                                        max_posts_per_user=None, 
                                        sample_seqs=False, shuffle=False, hierarchical=hyperparams['hierarchical'],
                                         use_bert='bert_layer' not in hyperparams['ignore_layer'],
                                         compute_liwc=True, liwc_words_for_categories=liwc_words_for_categories,
                                        emotions=emotions, liwc_categories=liwc_categories,
                                        session=session, classes=classes)

    return data_generator_train, data_generator_valid

def initialize_model(hyperparams, hyperparams_features, embedding_matrix, emotions, stopword_list,
                    liwc_categories, session=None, transfer=False, classes=1):

    logger.info("Initializing model...\n")
    # Initialize model
    if hyperparams['hierarchical']:
        model = build_hierarchical_model(hyperparams, hyperparams_features, embedding_matrix, 
                                         emotions, stopword_list, liwc_categories,
                       ignore_layer=hyperparams['ignore_layer'], classes=classes)
    else:
        model = build_model(hyperparams, hyperparams_features, embedding_matrix, 
                                        emotions, stopword_list, liwc_categories,
                       ignore_layer=hyperparams['ignore_layer'], classes=classes)
    if transfer:
        model = build_tl_model(hyperparams, hyperparams_features, embedding_matrix, 
                                        emotions, stopword_list, liwc_categories,
                       ignore_layer=hyperparams['ignore_layer'])
        model.load_weights(hyperparams_features['pretrained_model_path'] + '_weights.h5', by_name=True)
    # Needed just for bert
    if 'bert_layer' not in hyperparams['ignore_layer']:
        initialize_sess(session)                  
    model.summary()
    return model

In [ ]:
def train(user_level_data, subjects_split, 
          hyperparams, hyperparams_features, 
          embedding_matrix, emotions, stopword_list, liwc_categories,
          experiment, validation_set='valid',
          version=0, epochs=50, start_epoch=0,
         session=None, model=None, transfer_layer=False,
         classes=1):
    network_type, hierarch_type = get_network_type(hyperparams)
    for feature in ['LIWC', 'emotions', 'numerical_dense_layer', 'sparse_feat_dense_layer', 'user_encoded']:
        if feature in hyperparams['ignore_layer']:
            network_type += "no%s" % feature
    if not transfer_layer:
        model_path='models/%s_%s_%s%d' % (network_type, dataset_type, hierarch_type, version)
    else:
        model_path='models/%s_%s_%s_transfer_%s%d' % (network_type, dataset_type, hierarch_type, transfer_type, version)
        
    
    # Ablate emotions or LIWC
    if 'emotions' in hyperparams['ignore_layer']:
        emotions = []
    if 'LIWC' in hyperparams['ignore_layer']:
        liwc_categories = []

    logger.info("Initializing datasets...\n")
    data_generator_train, data_generator_valid = initialize_datasets(user_level_data, subjects_split, 
                                                                     hyperparams,hyperparams_features,
                                                                     validation_set=validation_set,
                                                                     emotions=emotions, liwc_categories=liwc_categories, 
                                                                    session=session, classes=classes)
    if not model:
        if transfer_layer:
            logger.info("Initializing pretrained model...\n")
        else:
            logger.info("Initializing model...\n")
        model = initialize_model(hyperparams, hyperparams_features, embedding_matrix, 
                                 emotions, stopword_list, liwc_categories, session=session, transfer=transfer_layer,
                                classes=classes)

       
    print(model_path)
    logger.info("Training model...\n")
    model, history = train_model(model, hyperparams,
                                 data_generator_train, data_generator_valid,
                       epochs=epochs, start_epoch=start_epoch,
                      class_weight={0:1, 1:hyperparams['positive_class_weight']} if classes==1 else None,
                      callback_list = [
                          'weights_history',
                          'lr_history',
                          'outputs_history_valid',
                          'outputs_history_train',
                          'reduce_lr_plateau',
                          'lr_schedule'
                                      ],
                      model_path=model_path, workers=4,
                                validation_set=validation_set)
    logger.info("Saving model...\n")
    try:
        save_model_and_params(model, model_path, hyperparams, hyperparams_features)
        experiment.log_parameter("model_path", model_path)
    except:
        logger.error("Could not save model.\n")

    return model, history
# except Exception as e:# tf.errors.ResourceExhaustedError:
#     print(e)
#     sess.close()
#     sess = tf.Session(config=sess_config)
#     initialize_vars(sess)


## Train below

In [ ]:
key = 'lstm_seq'
# # non-duplicate key
# while key in models_collection.keys():
#     key=key + "1"
# # models_collection[key] = cur_model
# # session.close()
# session = initialize_sess()
# # all_sessions.append(session)
# session_collection[key] = session


In [ ]:
models_collection = {}
hyperparams_collection = {}

In [ ]:
len(liwc_categories)

In [ ]:
# writings_df.groupby('source').subject.nunique()
# Counter(writings_df.label.values)

In [ ]:
hyperparams = {
    # Network parmeters
    
    # Sequential + hierarchical layers
    'trainable_embeddings': True,

    'lstm_units': 128,
#     'lstm_units': 256,
    
    'dense_bow_units': 20,
    'dense_sentence_units': 0,
    'dense_numerical_units': 20,

    
    # CNN
    'filters': 100,
    'kernel_size': 5,
    
    # Just hierarchical layers
    'lstm_units_user': 32,
    'dense_user_units': 0,
    
    'filters_user': 10,
    'kernel_size_user': 3,
        
    # BERT layers
    'bert_dense_units': 256,
    'bert_finetune_layers': 0,
    'bert_trainable': False ,
    'bert_pooling': 'first', # mean, first
    
    'transfer_units': 20,

    # Regularization etc
    'dropout': 0.1,
#     'dropout': 0,
    'l2_dense': 0.00011,
    'l2_embeddings': 0.0000001,
    'l2_bert': 0.0001,
    'norm_momentum': 0.1,
    
    'ignore_layer': [
#         'lstm', 'attention', 
#         'cnn',
#         'user_encoded',
#         'lstm_user', 'attention_user', 
#                      'batchnorm',
#                      'user_encoded', # remove LSTM/CNN
                     'bert_layer',
#                      'numerical_dense_layer', 
#         'sparse_feat_dense_layer', # remove extracted features
#         'LIWC',
#         'emotions'
                    ],

    # Learning parameters
    'optimizer': None, #'adam',
    'decay': 0.001,
    'lr': 0.00005,
#     'lr': 0.00005,#     'lr': 0.01,
#     'lr': 0.00001,#     'lr': 0.01,
#     'lr': 0.0001,
    "reduce_lr_factor": 0.5,
    "reduce_lr_patience": 55,
    'scheduled_reduce_lr_freq': 95,
    'scheduled_reduce_lr_factor': 0.5,
    "freeze_patience": 2000,
    'threshold': 0.5,
    'early_stopping_patience': 5,
    'positive_class_weight': 1,
    # set this to None for uniform class sampling (only applies to multiclass setting)
    'class_weights': {0:1304, 1:1287, 2:763},
#     'positive_class_weight': 2,
    
    # Generator parameters
    
    # Note: average text length in eRisk: 300
    #       average text length in CLPsych: 13
#     "maxlen": 512,
#     "maxlen": 128,
#     "maxlen": 512,
    "maxlen": 256,
    "posts_per_user": None, # if you want to limit total nr of posts considered per user
    "post_groups_per_user": None, # if you want a fixed number of post groups per user
                                  # to even out user weights
    "posts_per_group": 50, # how long are the "batches" of posts. maxlen/avglen~=posts_per_group
    "batch_size": 32,
    "padding": "pre",
    "hierarchical": True,
    'sample_seqs': False,
    'sampling_distr': 'exp',

}
if not hyperparams['optimizer']:
    hyperparams['optimizer'] = optimizers.Adam(lr=hyperparams['lr'], #beta_1=0.9, beta_2=0.999, epsilon=0.0001,
                                   decay=hyperparams['decay'])
    
if transfer_type:
#     hyperparams, _ = load_params(hyperparams_features['pretrained_model_path'])
    if 'optimizer' not in hyperparams:
        hyperparams['optimizer'] = optimizers.Adam(lr=hyperparams['lr'], #beta_1=0.9, beta_2=0.999, epsilon=0.0001,
                                       decay=hyperparams['decay'])

In [ ]:
hyperparams_collection[key] = hyperparams

In [ ]:

# session_collection['cnn_hierarch'].close()
# K.clear_session()
# transfer_type=None
dataset_type
# transfer_type

In [ ]:
hyperparams['optimizer']

In [ ]:
%%time
# transfer_type="depression"
transfer_type=None
classes=1
# transfer_type="depression"
# with session_collection[key].as_default():
#     with session_collection[key].graph.as_default():
hyperparams_collection[key] = hyperparams


if transfer_type:
    # TODO: try to compile? with proper optimizer?
    hyperparams_collection[key]['trainable_embeddings'] = True
    hyperparams_collection[key]['transfer_units'] = 20
    hyperparams_collection[key]['positive_class_weight'] = 1
#     hyperparams_collection[key]['positive_class_weight'] = 5
    hyperparams_collection[key]['lr'] = 0.001
    hyperparams_collection[key]['optimizer'] = optimizers.Adam(lr=hyperparams_collection[key]['lr'], #beta_1=0.9, beta_2=0.999, epsilon=0.0001,
                                   decay=hyperparams_collection[key]['decay'])
    hyperparams_features['pretrained_model_path'] = 'models/lstmnonumerical_dense_layer_clpsych_hierarchical62'

#     hyperparams_collection[key]['lr'] = 0.01

#     models_collection[key] = load_saved_model(hyperparams_features['pretrained_model_path'], 
#                                               hyperparams_collection[key])
    models_collection[key] = load_saved_model_weights(hyperparams_features['pretrained_model_path'], 
                                                      hyperparams_collection[key], 
                                                      emotions, stopword_list, liwc_categories, classes, 
                                                      h5=True)

#     metrics_class = Metrics(threshold=hyperparams_collection[key]['threshold'])
#     models_collection[key].compile(hyperparams_collection[key]['optimizer'], binary_crossentropy_custom,
#                   metrics=[metrics_class.f1_m, metrics_class.precision_m, metrics_class.recall_m,
#                           metrics_class.auc, 
#                            tf.keras.metrics.AUC(name="AUC_keras"), 
#                            tf.keras.metrics.FalsePositives(), 
#                            tf.keras.metrics.FalseNegatives()])


    experiment = initialize_experiment(hyperparams_collection[key], nrc_lexicon_path, emotions, pretrained_embeddings_path, 
                                  dataset_type, transfer_type)
    models_collection[key], history = train(user_level_data, subjects_split, 
              hyperparams_collection[key], hyperparams_features, 
              embedding_matrix, emotions, stopword_list, liwc_categories,
                  experiment,
                  validation_set='valid',
              version=63, epochs=90, start_epoch=80, transfer_layer=False,
#                     session=session_collection[key],
                model=models_collection[key]                        # Remove this to use transfer layer(?)
                                           )
else:
    experiment = initialize_experiment(hyperparams, nrc_lexicon_path, emotions, pretrained_embeddings_path, 
                          dataset_type, transfer_type)
    models_collection[key], history = train(user_level_data, subjects_split, 
              hyperparams_collection[key], hyperparams_features, 
              embedding_matrix, emotions, stopword_list, liwc_categories,
                  experiment,
                  validation_set='valid',
              version=102, epochs=25, start_epoch=0, classes=classes,
#                     session=session_collection[key],
#                     model=models_collection[key]
                                           )

In [ ]:
print(len(emotions))
print(len(liwc_categories))
# models_collection[key].get_layer('LSTM_layer_user').weights
models_collection[key].summary()
# hyperparams_collection[key]['ignore_layer']

In [ ]:
# # model_path = "models/lstm_%s_hierarchical53" % dataset_type
# models_collection[key] = initialize_model(hyperparams_collection[key], 
#                                           hyperparams_features, embedding_matrix, emotions, stopword_list,
#                     liwc_categories)
# # models_collection[key].load_weights(model_path + "_weights")


# # embedding_matrix.shape
# models_collection[key].load_weights(hyperparams_features['pretrained_model_path'] + "_weights")
# metrics_class = Metrics(threshold=hyperparams_collection[key]['threshold'])
# # models_collection[key].compile(hyperparams_collection[key]['optimizer'], binary_crossentropy_custom,
# #               metrics=[metrics_class.f1_m, metrics_class.precision_m, metrics_class.recall_m,
# #                       metrics_class.auc, 
# #                        tf.keras.metrics.AUC(name="AUC_keras"), 
# #                        tf.keras.metrics.FalsePositives(), 
# #                        tf.keras.metrics.FalseNegatives()])

In [ ]:
# models_collection[key] = initialize_model(hyperparams_collection[key], 
#                                           hyperparams_features, embedding_matrix, emotions, stopword_list,
#                     liwc_categories)
# models_collection[key] = load_weights(hyperparams_features['pretrained_model_path'] + "_weights.h5")

In [ ]:
# models_collection[key] = depression_model
# hyperparams_collection[key]
# classes


In [ ]:
%%time
import pickle

# # Evaluate on entire posts history, final F1-score
# print("Evaluating on same nr of groups as train (%d)..." % hyperparams['post_groups_per_user'] if 
#       hyperparams['post_groups_per_user'] else 0)
# cur_model.evaluate_generator(DataGenerator(user_level_data, subjects_split, 
#                                          set_type='test', 
#                                     seq_len=hyperparams['maxlen'], batch_size=hyperparams['batch_size'],
#                                        hierarchical=hyperparams['hierarchical'],
#                                          max_posts_per_user=None,
#                                        pad_with_duplication=False,
#                                         posts_per_group=hyperparams['posts_per_group'],
#                                         post_groups_per_user=hyperparams['post_groups_per_user'], 
#                                          sample_seqs=False, shuffle=False), verbose=1)
# print("Evaluating on entire posts history...")
# cur_model.evaluate_generator(DataGenerator(user_level_data, subjects_split, 
#                                          set_type='test', 
#                                     seq_len=hyperparams['maxlen'], batch_size=hyperparams['batch_size'],
#                                        hierarchical=hyperparams['hierarchical'],
#                                          max_posts_per_user=None,
#                                        pad_with_duplication=False,
#                                         posts_per_group=hyperparams['posts_per_group'],
#                                         post_groups_per_user=hyperparams['post_groups_per_user'], 
#                                          sample_seqs=False, shuffle=False), verbose=1)
# hyperparams_collection[key] = hyperparams


liwc_words_for_categories = pickle.load(open(hyperparams_features['liwc_words_cached'], 'rb'))
if 'emotions' in hyperparams_collection[key]['ignore_layer']:
    emotions = []
if 'LIWC' in hyperparams_collection[key]['ignore_layer']:
    liwc_categories = []
# # 
# with session_collection[key].as_default():
#     with session_collection[key].graph.as_default():
print("Evaluating only on last group (1)...")
if classes==1:
    metrics_class = Metrics(threshold=0.5)
    models_collection[key].compile(hyperparams_collection[key]['optimizer'], binary_crossentropy_custom,
                      metrics=[metrics_class.f1_m, metrics_class.precision_m, metrics_class.recall_m,
                              metrics_class.auc, 
                               tf.keras.metrics.AUC(name="AUC_keras"), 
                               tf.keras.metrics.FalsePositives(), 
                               tf.keras.metrics.FalseNegatives()])
else:
    models_collection[key].compile(hyperparams_collection[key]['optimizer'], K.categorical_crossentropy,
                     metrics=[tf.keras.metrics.CategoricalAccuracy(name='cat_acc'),
                             tf.keras.metrics.Precision(name='prec'), tf.keras.metrics.Recall(name='rec'), 
                              ])

test_generator = DataGenerator(user_level_data, subjects_split, 
                                         set_type='test', 
                                    seq_len=hyperparams_collection[key]['maxlen'],   
#                                         batch_size=len(subjects_split['test']), # on all data at once
                                        batch_size=hyperparams_collection[key]['batch_size'], # on all data at once
                                       hierarchical=hyperparams_collection[key]['hierarchical'],
                                         max_posts_per_user=None,
                                       pad_with_duplication=False,
                                        posts_per_group=hyperparams_collection[key]['posts_per_group'],
                                        post_groups_per_user=1,  compute_liwc=True,
                                        liwc_words_for_categories=liwc_words_for_categories,
                                        emotions=emotions, liwc_categories=liwc_categories, keep_last_batch=False,
                                         sample_seqs=False, shuffle=False, classes=classes,
                              )
print("Evaluating...")
results1 = models_collection[key].evaluate_generator(test_generator, verbose=1)

if classes>1:
    test_labels_sparse = list(test_generator.generated_labels) # need to get these after iterating
    print(len(test_labels_sparse), len(test_generator.generated_labels))
    print("Predicting")
    Y_pred = models_collection[key].predict_generator(test_generator)#, num_of_test_samples // batch_size+1)
    y_pred = np.argmax(Y_pred, axis=1)
    
    test_labels_sparse = list(test_generator.generated_labels) # need to get these after iterating
    print(len(test_labels_sparse), len(test_generator.generated_labels))
    print('Confusion Matrix')
    # TODO: check
    # keeping as many batches as we have for predictions, hopefully they match... I got one more
    # TODO: no, can't assume it, have to fix it

    test_labels = np.argmax(test_generator.generated_labels, axis=1) 
    print(test_labels.shape, y_pred.shape)
    print(confusion_matrix(test_labels, y_pred))
    print('Classification Report')
    target_names = ['selfharm', 'anorexia', 'depression']
    print(classification_report(test_labels, y_pred, target_names=target_names))
    # experiment.log_metric("test_f1", results1[1])
# experiment.log_metric("test_prec", results1[2])
# experiment.log_metric("test_recall", results1[3])
# experiment.log_metric("test_auc", results1[4])


#     Optimal thresh
if classes==1:
    print("Finding optimal threshold...")
    f1s = {}
    eval_generator = DataGenerator(user_level_data, subjects_split, 
                                             set_type='valid', compute_liwc=True,
                                            liwc_words_for_categories=liwc_words_for_categories,
                                            emotions=emotions, liwc_categories=liwc_categories,
                                        seq_len=hyperparams_collection[key]['maxlen'], 
                                            batch_size=hyperparams_collection[key]['batch_size'],
                                           hierarchical=hyperparams_collection[key]['hierarchical'],
                                             max_posts_per_user=None,
                                           pad_with_duplication=False,
                                            posts_per_group=hyperparams_collection[key]['posts_per_group'],
                                            post_groups_per_user=1, 
                                                     sample_seqs=False, shuffle=False)
    for thresh in range(0, 10, 1):
        print(thresh)
        # model1.load_weights(model_path + "_weights", by_name=True)
        metrics_class = Metrics(threshold=thresh/10)
        models_collection[key].compile(hyperparams_collection[key]['optimizer'], binary_crossentropy_custom,
                          metrics=[metrics_class.f1_m, metrics_class.precision_m, metrics_class.recall_m,
                                metrics_class.auc, 
                               tf.keras.metrics.AUC(name="AUC_keras"), 
                               tf.keras.metrics.FalsePositives(), 
                               tf.keras.metrics.FalseNegatives()])
    
        results = models_collection[key].evaluate_generator(eval_generator, verbose=1)
        f1s[thresh] = results[1]
    best_f1 = 0
    best_thresh = 0.5
    for t, f1 in f1s.items():
        if f1 >= best_f1:
            best_thresh = t
            best_f1 = f1
    print(f1s, best_thresh)
    hyperparams_features['best_thresh'] = best_thresh/10
    # experiment.log_metric("best_thresh", best_thresh/10)

    print("Computing with best threshold...")
    metrics_class = Metrics(best_thresh/10)
    models_collection[key].compile(hyperparams_collection[key]['optimizer'], binary_crossentropy_custom,
                          metrics=[metrics_class.f1_m, metrics_class.precision_m, metrics_class.recall_m,
                                metrics_class.auc, 
                               tf.keras.metrics.AUC(name="AUC_keras"), 
                               tf.keras.metrics.FalsePositives(), 
                               tf.keras.metrics.FalseNegatives()])

    results = models_collection[key].evaluate_generator(DataGenerator(user_level_data, subjects_split, 
                                             set_type='test', compute_liwc=True,
                                            liwc_words_for_categories=liwc_words_for_categories,
                                            emotions = emotions, liwc_categories=liwc_categories,

                                        seq_len=hyperparams_collection[key]['maxlen'], 
                                           batch_size=hyperparams_collection[key]['batch_size'],
                                           hierarchical=hyperparams_collection[key]['hierarchical'],
                                             max_posts_per_user=None,
                                           pad_with_duplication=False,
                                            posts_per_group=hyperparams_collection[key]['posts_per_group'],
                                            post_groups_per_user=1, 
                                                     sample_seqs=False, shuffle=False), verbose=1)


In [ ]:
# np.argmax(test_generator.generated_labels, axis=1)
test_labels_sparse = list(test_generator.generated_labels)
len(test_generator.generated_labels)

In [ ]:
user_level_data2, subjects_split2, vocabulary = load_erisk_data(writings_df[writings_df['source']=='depression'], 
                                                            voc_size=hyperparams_features['max_features'],
                                                           emotion_lexicon=nrc_lexicon,
                                                           emotions=emotions,
                                                           user_level=hyperparams_features['user_level'],
                                                                                logger=logger,
#                                                            vocabulary=pickle.load(open('vocabulary_40K_all.pkl', 'rb')),
#                                                            vocabulary=pickle.load(open('vocab_clpsych_10000.pkl', 'rb')),
                                                              vocabulary=vocabulary_dict,
                                                              by_subset=False
                                                                               )
results2 = models_collection[key].evaluate_generator(DataGenerator(user_level_data2, subjects_split2, 
                                         set_type='test', 
                                    seq_len=hyperparams_collection[key]['maxlen'],   
#                                         batch_size=len(subjects_split['test']), # on all data at once
                                        batch_size=hyperparams_collection[key]['batch_size'], # on all data at once
                                       hierarchical=hyperparams_collection[key]['hierarchical'],
                                         max_posts_per_user=None,
                                       pad_with_duplication=False,
                                        posts_per_group=hyperparams_collection[key]['posts_per_group'],
                                        post_groups_per_user=1,  compute_liwc=True,
                                        liwc_words_for_categories=liwc_words_for_categories,
                                         sample_seqs=False, shuffle=False), verbose=1)

In [ ]:
iterations = 1500

# Evaluate on partial post history, simulating stream
print("Evaluating on partial posts history...")
cnt=0
cnt+=1
f1_per_iteration = []
auc_per_iteration = []
precision_per_iteration = []
recall_per_iteration = []
predictions_per_iteration = []
for iteration in range(0, iterations, hyperparams_collection[key]['posts_per_group']):
    print("Iteration", iteration)
    results = models_collection[key].evaluate_generator(DataGenerator(user_level_data, subjects_split, 
                                         set_type='test', 
                                       hierarchical=hyperparams_collection[key]['hierarchical'],
                                        seq_len=hyperparams['maxlen'], 
#                                         batch_size=len(subjects_split['test'])//2, # on all data at once
                                        batch_size=hyperparams_collection[key]['batch_size'], # on all data at once
                                         max_posts_per_user=iteration,
                                       pad_with_duplication=False,
                                        posts_per_group=hyperparams_collection[key]['posts_per_group'],
                                        post_groups_per_user=1, 
                                        post_offset=iteration,
                                        compute_liwc=True,
                                        liwc_words_for_categories=liwc_words_for_categories,
                                        emotions=emotions, liwc_categories=liwc_categories,
                                         sample_seqs=False, shuffle=False), verbose=1)
    f1_per_iteration.append(results[1])
    precision_per_iteration.append(results[2])
    recall_per_iteration.append(results[3])
    auc_per_iteration.append(results[5])
    predictions = models_collection[key].predict(DataGenerator(user_level_data, subjects_split, 
                                         set_type='test', 
                                       hierarchical=hyperparams_collection[key]['hierarchical'],
                                        seq_len=hyperparams['maxlen'], 
#                                         batch_size=len(subjects_split['test'])//2, # on all data at once
                                        batch_size=hyperparams_collection[key]['batch_size'], # on all data at once
                                         max_posts_per_user=iteration,
                                       pad_with_duplication=False,
                                        posts_per_group=hyperparams_collection[key]['posts_per_group'],
                                        post_groups_per_user=1, 
                                        post_offset=iteration,
                                        compute_liwc=True,
                                        liwc_words_for_categories=liwc_words_for_categories,
                                        emotions=emotions, liwc_categories=liwc_categories,
                                         sample_seqs=False, shuffle=False), verbose=1)
    predictions_per_iteration.extend(predictions)

In [ ]:
predictions_per_iteration

In [ ]:
print("""average precision across %d iterations: %f;
average recall across %d iterations: %f;
average f1 across %d iterations: %f;
average auc across %d iterations: %f""" % (
    iterations, np.mean(precision_per_iteration[:iterations]),
    iterations, np.mean(recall_per_iteration[:iterations]),
    iterations, np.mean(f1_per_iteration[:iterations]),
    iterations, np.mean(auc_per_iteration[:iterations])))

In [ ]:
dataset_type

In [ ]:
# plt.plot(range(0, len(auc_per_iteration)*hyperparams_collection[key]['posts_per_group'], hyperparams_collection[key]['posts_per_group']), 
#          auc_per_iteration)
pd.Series(auc_per_iteration, 
          index=range(0, len(auc_per_iteration)*hyperparams_collection[key]['posts_per_group'], hyperparams_collection[key]['posts_per_group'])
                                                                    ).rolling(20).mean().plot()
plt.xlabel("Posts offset")
plt.ylabel("AUC")

In [ ]:
pd.Series([p[0] for p in predictions_per_iteration]).hist(bins=40)

In [ ]:
# plt.plot(range(0, iteration*hyperparams_collection[key]['posts_per_group'], 
#                hyperparams_collection[key]['posts_per_group']), 
#          predictions_per_iteration)
pd.Series([p[0] for p in predictions_per_iteration]).rolling(5000).std().plot()
plt.xlabel("Posts offset")
plt.ylabel("Predictions")

In [ ]:
attention_user = [v for v in models_collection[key].get_layer('attention_user').get_weights()[0].flatten()]


In [ ]:
attention_layer = models_collection[key].get_layer('user_encoder').get_weights()[-2]
assert(attention_layer.shape == (128, 1))
pd.Series([v for v in attention_layer]).rolling(30).mean().plot()

In [ ]:
# pickle.dump(auc_per_iteration, open("auc_per_iteration_selfharm53.pkl", "wb+"))
# pickle.dump(f1_per_iteration, open("f1_per_iteration_selfharm53.pkl", "wb+"))
# pickle.dump(predictions_per_iteration, open("predictions_per_iteration_selfharm53.pkl", "wb+"))
# pickle.dump(attention_user, open("attention_user_selfharm53.pkl", "wb+"))
# pickle.dump(attention_layer, open("attention_post_selfharm53.pkl", "wb+"))


In [ ]:
# model = load_saved_model("models/lstm_depression_hierarchical52", hyperparams_collection[key])

In [ ]:
plt.plot(range(0, len(scores_per_iteration[:15])*hyperparams_collection[key]['posts_per_group'], hyperparams_collection[key]['posts_per_group']), 
         scores_per_iteration[:15])
plt.xlabel("Posts offset")
plt.ylabel("AUC")

In [ ]:
# attention_user = [v for v in models_collection[key].get_layer('attention_user').get_activations()[0].flatten()]
l = models_collection[key].get_layer('attention_user')
# l.activation(samplegenerator)

In [ ]:
scores_per_iteration

In [ ]:
for e in nrc_lexicon:
    print(e)
    print(nrc_lexicon[e].intersection(set(tokenize(text.lower()))))


In [ ]:
# save_model_and_params(cur_model, 'models/lstm_selfharm_seq16_2', hyperparams, hyperparams_features)

In [ ]:
# models_collection = {}
# hyperparams_collection = {}


In [ ]:
pd.Series([v for v in models_collection[key].get_layer('attention_user').get_weights()[0].flatten()]).rolling(10
                                                                                                             ).mean().plot()

In [ ]:
pd.Series([v for v in models_collection[key].get_layer('attention_user').get_weights()[0].flatten()]).abs().rolling(10
                                                                                                             ).mean().plot()

In [ ]:
# pd.Series([v for v in models_collection[key].get_layer('attention_user').get_weights()[0].flatten()]).to_csv(
#     'attention_user_weights_lstm_selfharm_erisk_all_1group.pkl')

In [ ]:
pd.Series([v for v in models_collection[key].get_layer('attention').get_weights()[0].flatten()]).rolling(5).mean().plot()

In [ ]:
pd.Series([abs(v) for v in models_collection[key].get_layer('output_layer').get_weights()[0].flatten()]).plot()

In [ ]:
d1, d2 = initialize_datasets(user_level_data, subjects_split, hyperparams, hyperparams_features, 
                        validation_set='valid', session=None)

In [ ]:
writings_df.source

### Attention and predictions analysis

In [ ]:
model_nr = 56
model_name = "models/lstm_%s_hierarchical53" % dataset_type
hyperparams, _ = load_params(model_name)
model = load_saved_model(model_name, hyperparams)
auc_per_iteration = pickle.load(open("auc_per_iteration_depression53.pkl", "rb"))
f1_per_iteration = pickle.load(open("f1_per_iteration_depression53.pkl", "rb"))
predictions_per_iteration = pickle.load(open("predictions_per_iteration_depression53.pkl", "rb"))
attention_user = pickle.load(open("attention_user_depression53.pkl", "rb"))
attention_layer = pickle.load(open("attention_post_depression53.pkl", "rb"))

In [ ]:
pd.Series(attention_user).rolling(10).mean().plot()
plt.ylabel("Attention weight")

In [ ]:
pd.Series([abs(l[0]) for l in attention_layer]).rolling(50).mean().plot()
plt.ylabel("Attention weight")

In [ ]:
index = range(0, min(len(auc_per_iteration)*hyperparams['posts_per_group'], 2000), 
                      hyperparams['posts_per_group'])
pd.Series(auc_per_iteration, 
          index=index
                                                                    ).rolling(5).mean().plot()
plt.xlabel("Posts offset")
plt.ylabel("AUC")

In [ ]:
[l.shape for l in model.get_layer('user_encoder').get_weights()]
# hyperparams

In [ ]:
len(model.get_layer('attention_user').get_weights()[0])


In [ ]:
[(l.name, len(l.get_weights()), len(l.get_weights()[0]) if len(l.get_weights()) else 0) for l in model.layers]

In [ ]:
[(len(l), len(l[0]) if len(l) and type(l[0])==list else 0) for 
 l in model.get_layer('user_encoder').get_weights()]

In [ ]:
userl = model.get_layer('user_encoder').layer


In [ ]:
userl.get_layer('lstm_user').get_weights()[0]

In [ ]:
lstm_weights = userl.get_layer(index=3).get_weights()

In [ ]:
lstm_weights[1].shape

### Attention activations

In [ ]:
model_nr=101
# dataset_type = "anorexia"

hyperparams = json.load(open("models/lstm_%s_hierarchical%d.hp.json" % (dataset_type, model_nr)))

if 'optimizer' not in hyperparams:
    hyperparams['optimizer'] = optimizers.Adam(lr=hyperparams['lr'], #beta_1=0.9, beta_2=0.999, epsilon=0.0001,
                                   decay=hyperparams['decay'])

In [ ]:
original_model = build_hierarchical_model(hyperparams, hyperparams_features, embedding_matrix, 
                                         emotions, stopword_list, liwc_categories,
                       ignore_layer=hyperparams['ignore_layer'], activations=False)
original_model.load_weights("models/lstm_%s_hierarchical%d" % (dataset_type, model_nr) + '_weights.h5', by_name=True)

activation_model_user = build_hierarchical_model(hyperparams, hyperparams_features, embedding_matrix, 
                                         emotions, stopword_list, liwc_categories,
                       ignore_layer=hyperparams['ignore_layer'], activations='attention_user')
# activation_model_user.load_weights("models/lstm_%s_hierarchical%d"  % (dataset_type, model_nr) + '_weights.h5', by_name=True)

user_representation_model = build_hierarchical_model(hyperparams, hyperparams_features, embedding_matrix, 
                                         emotions, stopword_list, liwc_categories,
                       ignore_layer=hyperparams['ignore_layer'], activations='output_layer')
user_representation_model.load_weights("models/lstm_%s_hierarchical%d"  % (dataset_type, model_nr) + '_weights.h5', by_name=True)


activation_model = build_hierarchical_model(hyperparams, hyperparams_features, embedding_matrix, 
                                         emotions, stopword_list, liwc_categories,
                       ignore_layer=hyperparams['ignore_layer'], activations='attention')
# activation_model.load_weights("models/lstm_%s_hierarchical%d" % (dataset_type, model_nr) + '_weights.h5', by_name=True)



# activation_outputs = [layer.output for layer in original_model.layers]
# activation_model = tf.keras.Model(inputs, activation_outputs)

In [ ]:
user_representation_model.summary()

In [ ]:
positive_subjects = list(set(writings_df[writings_df['label']==1].subject))
negative_subjects = list(set(writings_df[writings_df['label']==0].subject)) #[:len(positive_subjects)]

In [ ]:
print(len(positive_subjects), len(negative_subjects))

In [ ]:
# subji = 15
subjects_split_test_positive = {'test':
                                 [s for s in subjects_split['test'] if s in positive_subjects][:]}
subjects_split_test_negative = {'test':
                                 [s for s in subjects_split['test'] if s in negative_subjects][:]}

In [ ]:
samplegenerator_positive = DataGenerator(user_level_data, subjects_split_test_positive, 
                                     set_type='test', 
                                   hierarchical=True,
                                seq_len=hyperparams['maxlen'], #batch_size=hyperparams['batch_size'],
                                batch_size=1,
                                max_posts_per_user=None,
                                   pad_with_duplication=False,
                                    posts_per_group=hyperparams['posts_per_group'],
#                                     post_groups_per_user=None, 
                                    post_groups_per_user=1, 
                                     sample_seqs=False, shuffle=False)
samplegenerator_negative = DataGenerator(user_level_data, subjects_split_test_negative, 
                                     set_type='test', 
                                   hierarchical=True,
                                seq_len=hyperparams['maxlen'], #batch_size=hyperparams['batch_size'],
                                batch_size=1,
                                max_posts_per_user=None,
                                   pad_with_duplication=False,
                                    posts_per_group=hyperparams['posts_per_group'],
#                                     post_groups_per_user=None, 
                                    post_groups_per_user=1, 
                                     sample_seqs=False, shuffle=False)


In [ ]:
print(len(samplegenerator_positive), len(samplegenerator_negative))

### T2 encodings

In [ ]:
writings_df_t2=pickle.load(open('data/writings_df_T2_liwc.pkl', 'rb'))
# writings_df_t2=pickle.load(open('data/writings_df_t2_test_liwc.pkl', 'rb'))
user_level_data_t2, subjects_split_t2, vocabulary = load_erisk_data(writings_df_t2, 
                                                            voc_size=hyperparams_features['max_features'],
                                                           emotion_lexicon=nrc_lexicon,
                                                           emotions=emotions,
                                                           user_level=hyperparams_features['user_level'],
                                                                                logger=logger,
#                                                            vocabulary=pickle.load(open('vocabulary_40K_all.pkl', 'rb')),
#                                                            vocabulary=pickle.load(open('vocab_clpsych_10000.pkl', 'rb')),
                                                              vocabulary=vocabulary_dict,
                                                              by_subset=True,
                                                              labelcol = 'anx',
#                                                               label_index={'depression':1, "ptsd":0}
                                                                               )
subjects_split_t2['test'] += subjects_split_t2['train']+subjects_split_t2['valid']

In [ ]:
user_embeddings_per_subject = {}
for subject in subjects_split_t2['test']:
    samplegenerator_t2 = DataGenerator(user_level_data_t2, {'test':[subject]}, 
                                     set_type='test', 
                                   hierarchical=True,
                                seq_len=hyperparams['maxlen'], #batch_size=hyperparams['batch_size'],
                                batch_size=1,
                                max_posts_per_user=None,
                                   pad_with_duplication=False,
                                    posts_per_group=hyperparams['posts_per_group'],
#                                     post_groups_per_user=None, 
                                    post_groups_per_user=None, 
                                     sample_seqs=False, shuffle=False)
    print(subject, len(samplegenerator_t2))
    user_embeddings_per_subject[subject] = user_representation_model.predict(samplegenerator_t2)

In [ ]:
pickle.dump(user_embeddings_per_subject, open('user_embeddings_t2_2020_model101.pkl', 'wb+'))

In [ ]:
user_representation_model.predict(samplegenerator_t2).shape

#### Attention analysis

In [ ]:
attention_words_positive = activation_model.predict(samplegenerator_positive)
attention_words_negative = activation_model.predict(samplegenerator_negative)
predictions_positive = original_model.predict(samplegenerator_positive)
predictions_negative = original_model.predict(samplegenerator_negative)
attention_positive_user = activation_model_user.predict(samplegenerator_positive)
attention_negative_user = activation_model_user.predict(samplegenerator_negative)

In [ ]:
dataset_type

In [ ]:
predictions_negative.shape

In [ ]:
print(len(samplegenerator_positive), len(samplegenerator_negative), 
      predictions_positive.shape, predictions_negative.shape)

In [ ]:
writings_df[writings_df['subject']==subjects_split_test_positive['test'][0]][['text']].head()
# writings_df[writings_df['subject']==subjects_split_test_negative['test'][0]][['text']]

In [ ]:
print(predictions_negative.shape, attention_words_negative.shape, attention_negative_user.shape)
print(predictions_negative.sum(), attention_words_negative.sum(axis=2), attention_negative_user.sum(axis=1))

In [ ]:
attention_positive_user.shape
# len(writings_df[writings_df['subject']==subjects_split_test_positive['test'][2]])
len(predictions_positive)

In [ ]:
def build_word_scores(writings_df, subjects, generator, predictions, attention_user, attention_words, 
                      words_scores=None, nr_words=None, users=None):
    unk = " "
    if not words_scores:
        words_scores = {t:0 for t in vocabulary_list}
        words_scores_all = {t:[] for t in vocabulary_list}
        nr_words = {t:0 for t in vocabulary_list}
        words_scores[unk] = 0
        words_scores_all[unk] = []
        nr_words[unk] = 0
    highlighted_texts = []
    # NOTE: I am considering only first post group for each subject?
    for i in range(len(predictions)):
        if users and i not in users:
            continue
#         if predictions[i]<0.8:
#             continue
#         if i%57!=0:
#             continue
    #     if predictions_positive_user[i].mean() < 0.02:
    #         continue
#         sampletexts = writings_df[writings_df['subject']==subjects[i]][['tokenized_text']].values
        post_lengths = []
        for p in range(50):
#             if  abs(attention_user[i][p]) < 0.05:
#                 continue
            print(i, p)
            post_tokens = generator[i][0][0][0][p]
            post_len = sum(post_tokens!=0)
            post_lengths.append(post_len)
            nr_posts = 50
            if post_len == 0:
                nr_posts -= 1
                continue
#             print("post len", post_len, list(enumerate([vocabulary_list[j] if (j!=0 and j<len(vocabulary_list)) else " "
#                           for j in post_tokens ])))
            post_with_weights = [(k, tok, round(attention_words[i][p][k], 10)) 
                                 for (k, tok) in list(
                enumerate([vocabulary_list[j] if (j!=0 and j<len(vocabulary_list)) else unk
                          for j in post_tokens ]))]
            highlighted_text = [(">>>", attention_user[i][p])]
            for (pos, tok, att) in post_with_weights:
                score = att * attention_user[i][p] * post_len
                words_scores[tok] += score
                words_scores_all[tok].append(score)
                highlighted_text.append((tok,score))
                nr_words[tok] += 1
#             print(post_with_weights)
            highlighted_texts.append(highlighted_text)
            
            print("post len", post_len, "nr posts", nr_posts)
        #     print([list(enumerate([t for t in sampletexts[-3:]])))
            print("user attention", attention_user[i][p], "avg word attention", attention_words[i][p].mean())
            print("prediction", predictions[i])
            
            pd.Series(attention_words[i][p]).plot(label="%d_%d" %(i, p))
#             pd.Series(attention_positive_user[i]).plot()
#         plt.scatter(np.exp(attention_user[i]), post_lengths)
#         plt.scatter(attention_words[i].mean(axis=1), post_lengths)
    words_scores = {w: s/nr_words[tok] for (w,s) in words_scores.items()}
    plt.legend()
    return words_scores, words_scores_all, highlighted_texts

In [ ]:
%%time
words_scores_positive, words_scores_positive_all, highlighted_texts = build_word_scores(writings_df, subjects_split_test_positive['test'], samplegenerator_positive,
                                 predictions_positive, attention_positive_user, attention_words_positive,
                                                            users=[2,3,5,7,88,99,44,66])

words_scores_negative, words_scores_negative_all, highlighted_texts_negative = build_word_scores(writings_df, subjects_split_test_negative['test'], samplegenerator_negative,
                                 predictions_negative, attention_negative_user, attention_words_negative,
                                                                     users=[2,4,7])



In [ ]:
def generate_highlighted_html(texts):
    htmltext = """"<html> """
    for text in texts:
        for i, (tok, score) in enumerate(text):
            if i==0:
                htmltext += '<span style="background-color: rgba(200,0,0,%f)"> %s </span>' % (20*abs(score), tok)
            else:
                htmltext += '<span style="background-color: rgba(0,0,200,%f)"> %s </span>' % (10*abs(score), tok)
        
#         htmltext += "</span>  <br> \n"
        htmltext += " <br> \n"
    htmltext += """</html>"""
    return htmltext
        

In [ ]:
# %%html
from IPython.display import display, Markdown
display(Markdown(generate_highlighted_html(highlighted_texts)))

In [ ]:
sorted(words_scores_negative.items(), key=lambda i: -abs(i[1]))

In [ ]:
word = 'i'
print(pd.Series(words_scores_positive_all[word]).describe(), pd.Series(words_scores_negative_all[word]).describe())

In [ ]:
pd.Series(words_scores_negative_all[word]).hist(bins=20, label="not self-harming", alpha=0.7)
pd.Series(words_scores_positive_all[word]).hist(bins=20, label="self-harming", alpha=0.7)
plt.xlabel('attention weights')
plt.legend()

In [ ]:
pd.Series(np.concatenate([attention_positive_user,attention_negative_user]).mean(axis=0)).plot()

In [ ]:
dataset_type

In [ ]:
pd.Series(np.concatenate([attention_positive_user]).mean(axis=0)).plot(label="depressed")
pd.Series(np.concatenate([attention_negative_user]).mean(axis=0)).plot(label="not depressed")
plt.ylabel("attention weights for user encoder")
plt.legend()

In [ ]:
pd.Series(attention_words_positive.mean(axis=(1,0))).plot(label="depressed")
pd.Series(attention_words_negative.mean(axis=(1,0))).plot(label="not depressed")
plt.ylabel("attention weights for post encoder")
plt.legend()

In [ ]:
for word in vocabulary_list:
    if not words_scores_negative[word]:
#         if words_scores_positive[word]:
#             print(word)
        continue
    if not words_scores_positive[word]:
        continue
    if pd.Series(words_scores_positive_all[word]).mean()/pd.Series(words_scores_negative_all[word]).mean() > 5:
        print(word)

#### Output layer analysis

In [ ]:
positive_representations = user_representation_model.predict(samplegenerator_positive)
negative_representations = user_representation_model.predict(samplegenerator_negative)

In [ ]:
plt.plot(abs(positive_representations.mean(axis=0)))

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
# pca.fit_transform(positive_representations)

In [ ]:
print(positive_representations.shape, negative_representations.shape,
      np.concatenate([positive_representations, negative_representations[:len(positive_representations)]]).shape)

In [ ]:
training_examples = np.concatenate([positive_representations, 
                                    negative_representations[:len(positive_representations)]])
# making them balanced
pca.fit_transform(training_examples)
positive_reduced = pca.transform(positive_representations)
negative_reduced = pca.transform(negative_representations)

In [ ]:
params = {'axes.labelsize': 16,
          'axes.titlesize': 16,
          'legend.fontsize': 16,
         'xtick.labelsize': 16,
         'ytick.labelsize': 16}
plt.rcParams.update(params)

In [ ]:
plt.scatter([x for x,y in negative_reduced], [y for x,y in negative_reduced], c='g', alpha=0.5, s=5, label='negative')

plt.scatter([x for x,y in positive_reduced], [y for x,y in positive_reduced], c='r', alpha=0.5, s=5, label='positive')
plt.legend()

In [ ]:
import seaborn as sns

def plot_kde(positives, negatives):
    graph = sns.jointplot([x for x,y in positives], [y for x,y in positives],
                     cmap="Reds", kind="kde", marginal_kws={"color":"r", "alpha":.2}, shade=True, 
                          shade_lowest=False, alpha=.5)
    graph.x = [x for x,y in negatives]
    graph.y = [y for x,y in negatives]
    graph.plot_joint(sns.kdeplot, cmap="Blues", shade=True, shade_lowest=False, alpha=.5)
    graph.plot_marginals(sns.kdeplot, color='b', shade=True, alpha=.2, legend=False)
    ax = plt.gca()


    ax.legend(["positive", "negative"])

plot_kde(positive_reduced, negative_reduced)

<html> <mark> text </mark> more

<span style="background-color: rgba(0,0,200,0.2)">This text is highlighted in yellow.</span>
</html>

#### Plotting by LIWC features

In [ ]:
liwc_features_positive = []
liwc_features_negative = []
for i, subject in enumerate(subjects_split_test_positive['test']):
    liwc_features_positive.extend(user_level_data[subject]['liwc'])
for i, subject in enumerate(subjects_split_test_negative['test']):
    liwc_features_negative.extend(user_level_data[subject]['liwc'])

pcaliwc = PCA(n_components=2)
pcaliwc.fit_transform(liwc_features_positive+liwc_features_negative)
liwc_reduced_positive = pcaliwc.transform(liwc_features_positive)
liwc_reduced_negative = pcaliwc.transform(liwc_features_negative)

plot_kde(liwc_reduced_positive, liwc_reduced_negative)


In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2, random_state=0).fit(training_examples)

kmeans.predict(np.array(negative_representations[:10]))


In [ ]:
dataset_type

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=0).fit(positive_representations)
positive_clusters = kmeans.predict(np.array(positive_representations))


In [ ]:
positive_clusters

## Predict

In [ ]:
classes=1
hyperparams_loaded = json.load(open('%s.hp.json' % hyperparams_features['pretrained_model_path']))
if 'optimizer' not in hyperparams_loaded:
    hyperparams_loaded['optimizer'] = optimizers.Adam(lr=hyperparams_loaded['lr'], #beta_1=0.9, beta_2=0.999, epsilon=0.0001,
                                   decay=hyperparams_loaded['decay'])


In [ ]:
# hyperparams_loaded = hyperparams_collection[key]
hyperparams_features['pretrained_model_path']
# hyperparams_collection[key] = hyperparams_loaded

In [ ]:
models_collection = {}
key='lstm_seq'
# hyperparams_loaded['trainable_embeddings']=True
models_collection[key] = load_saved_model_weights(hyperparams_features['pretrained_model_path'], 
                                                      hyperparams_loaded, 
                                                      emotions, stopword_list, liwc_categories, classes=classes, 
                                                      h5=True)

In [ ]:
# models_collection[key].get_output_at?
# combined_model = models_collection[key]
writings_df.columns

In [ ]:
%time
all_predictions = {}
# for subject in set(writings_df[writings_df['subset']=='test'].subject.values):
for subject in set(subjects_split['test']):
    all_predictions[subject] = {'true_label': None, 'predicted_label': None,
                               'predicted_score': None, 'texts': None}
    try:
        user_level_data_subject = {subject: user_level_data[subject]}
    except:
        continue
    if classes==1:
        true_label = writings_df[writings_df['subject']==subject].label.values[0]
    else:
        true_label = user_level_data[subject]['label']
#         true_label = label_index[user_level_data[subject]['condition']]
    print(subject, "Label", true_label)
    prediction = models_collection[key].predict(DataGenerator(user_level_data_subject, {'test':[subject]}, 
                                         set_type='test', 
                                    seq_len=hyperparams_loaded['maxlen'],   
#                                         batch_size=len(subjects_split['test']), # on all data at once
                                        batch_size=hyperparams_loaded['batch_size'], # on all data at once
                                       hierarchical=hyperparams_loaded['hierarchical'],
                                         max_posts_per_user=None,
                                       pad_with_duplication=False,
                                        posts_per_group=hyperparams_loaded['posts_per_group'],
                                        post_groups_per_user=1,  compute_liwc=True,
                                        liwc_words_for_categories=liwc_words_for_categories,
                                         sample_seqs=False, shuffle=False), verbose=1)
    if classes==1:
        model_prediction = int(prediction[0][0]>=0.5)
    else:
        model_prediction = np.argmax(prediction[0])
    print("Prediction: ", prediction[0], model_prediction)
    # TODO: is this aggregation of texts correct?
    if (true_label!=model_prediction):
        print("Misclassification:", len(
            user_level_data_subject[subject]['raw']), "words")
    all_predictions[subject]['true_label'] = true_label
    all_predictions[subject]['predicted_label'] = model_prediction
    all_predictions[subject]['predicted_score'] = prediction[0][0]
    all_predictions[subject]['texts'] = "\n".join(
            user_level_data_subject[subject]['raw'][:hyperparams_loaded['posts_per_group']])
    all_predictions[subject]['liwc'] = user_level_data_subject[subject]['liwc'][
        :hyperparams_loaded['posts_per_group']]

In [ ]:
predictions_df = pd.DataFrame.from_dict(all_predictions, orient='index')

In [ ]:
# predictions_df = pickle.load(open('predictions_df_depression_erisk_lstmhierarch56.pkl', 'rb'))

In [ ]:
predictions_df['misclass'] = abs(predictions_df['true_label'] - predictions_df['predicted_label'])
predictions_df['misclass'] = predictions_df['misclass'].apply(lambda x: 0 if x==0 else 1)
predictions_df['misclass_signed'] = predictions_df['true_label'] - predictions_df['predicted_label']
predictions_df[predictions_df['misclass']==1].predicted_score.describe()
# predictions_df[predictions_df['misclass_signed']==-1].predicted_score.describe()
# predictions_df[predictions_df['misclass']==0].predicted_score.describe()
# predictions_df[predictions_df['misclass']==1].mean()
# predictions_df[predictions_df['misclass']==0].mean()


In [ ]:
predictions_df.groupby(['misclass_signed', 'true_label']).count().predicted_label

In [ ]:
print(sorted(Counter(tokenize("\n".join(predictions_df[predictions_df['misclass']==1].texts.values))).items(), key=lambda t: -t[1])[:50])
print(sorted(Counter(tokenize("\n".join(predictions_df[predictions_df['misclass']==0].texts.values))).items(), key=lambda t: -t[1])[:50])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2
col = 'misclass'
target_val = 1
counter_val = None

misclassified_texts = "\n".join(predictions_df[predictions_df[col]==target_val].texts.values)
if counter_val is None:
    correct_texts = "\n".join(predictions_df[predictions_df[col]!=target_val].texts.values)
else:
    correct_texts = "\n".join(predictions_df[predictions_df[col]==counter_val].texts.values)
vectorizer = TfidfVectorizer(tokenizer=tokenize_tweets)
term_doc=vectorizer.fit_transform([misclassified_texts, correct_texts])
ch2 = SelectKBest(chi2, "all")
X_train = ch2.fit_transform(term_doc, [1,0])

scores = ch2.scores_
pvalues = ch2.pvalues_
words = vectorizer.get_feature_names()
features_sorted = sorted([(words[i], scores[i]) for i in range(len(scores))], key=lambda t:t[1], reverse=True)

print(features_sorted[:12])

In [ ]:
predictions_df['liwc_avg'] = predictions_df['liwc'].apply(lambda x: np.mean(x, axis=0))

In [ ]:
for i, c in enumerate(categories):
    predictions_df[c] = predictions_df['liwc_avg'].apply(lambda x: x[i])
# TODO: compute emotions here

In [ ]:
predictions_df=predictions_df[~predictions_df.isna()]

In [ ]:
predictions_df.dropna(inplace=True)

In [ ]:
# predictions_df = pickle.load(open('predictions_df_depression_clpsych.pkl','rb'))

In [ ]:
liwc_words_for_categories = {}
for categ in liwc_dict.keys():
    liwc_words_for_categories[categ] = set()
    for word in liwc_dict[categ]:
        for t in vocabulary:
            if t==word or (word[-1]=='*' and t.startswith(word[:-1])) \
                or (t==word.split("'")[0]):
                    liwc_words_for_categories[categ].add(t)

In [ ]:
liwc_words_for_categories.keys()

In [ ]:
predictions_df['all_tokens'] = predictions_df['texts'].apply(tokenize)

# TODO: include the title
def extract_emotions(tokens, emotion, relative=True):
    if not tokens:
        return None
    emotion_words = [t for t in tokens 
                     if t in nrc_lexicon[emotion]]
    if relative and len(tokens):
        return len(emotion_words) / len(tokens)
    else:
        return len(emotion_words)
    
    return encoded_emotions

from functools import partial
for emotion in emotions:
    predictions_df[emotion] = predictions_df['all_tokens'].apply(partial(extract_emotions, emotion=emotion, 
                                                                   relative=True))


In [ ]:
def encode_liwc_categories(tokens, category_words, relative=True):
    category_cnt = 0
    if not tokens:
        return None
    text_len = len(tokens)
    for t in tokens:
        for word in category_words:
            if t==word or (word[-1]=='*' and t.startswith(word[:-1])) \
            or (t==word.split("'")[0]):
                category_cnt += 1
                break # one token cannot belong to more than one word in the category
    if relative:
        return category_cnt/text_len
    else:
        return category_cnt

for categ in liwc_dict.keys():
    if categ in predictions_df.columns:
        continue
    print("Encoding for category %s..." % categ)
    predictions_df[categ] = predictions_df['all_tokens'].apply(partial(encode_liwc_categories, 
                                                                   category_words=liwc_dict[categ], 
                                                                   relative=True))


In [ ]:
def barplot_pvalues(categories_pscores, width=3, bar_width=None, yticks_freq=1, show_pval=False, 
                    yticks_start=1, angle=45, group_labels=None, logy=False, nr_bars=4):
    
    nr_bars = len(categories_pscores)
    categories_pscores_aggregated = []
    categories = set([t[0] for t in categories_pscores])
#     for cat in categories:
#         if aggreg=='mean':
#             # Compute mean
#             for c, v in categories_pscores:
#                 s = 0
#                 l = 0
#                 if c==cat:
#                     s += v
#                     l += 1
#             m = s/l
#             categories_pscores_aggregated.append((cat, m))
#         if aggreg=='group':
#             for c, v in categories_pscores:
#                 if c==cat:
#                     categories_pscores_aggregated.append()
    if type(categories_pscores) is list:
        categories_pscores_aggregated = [categories_pscores]
        labels = ['misclassif']
    if type(categories_pscores) is dict:
        labels = []
        for group, scores in categories_pscores.items():
            categories_pscores_aggregated.append(scores)
            labels.append(group)
#     if not bar_width:
    bar_width = 1/(nr_bars+1) * width
#         bar_width = 2/3 * width
 
    positions = []
    for i,scores in enumerate(categories_pscores_aggregated):
        print("plotting", labels[i], len(scores))
        if len(positions)==0:
            positions =  np.arange(0, len(scores) * width, width)
        else:
            positions = [x + bar_width for x in positions]
        plt.bar(
            positions,
#            x = np.arange(0, len(scores) * width, width),
            height=[s for (e,s) in scores], width=bar_width, align='edge', label=labels[i], edgecolor='white')
    if show_pval:
        plt.hlines(0.05, 0,width*nr_bars, linestyles=':', colors='red')
    if logy:
        plt.yscale('log')
    plt.xticks(ticks=range(int(bar_width*2), width*len(scores), width), 
               labels=[e[:] for e,p in scores], rotation=angle)
    if logy:
#         plt.yticks(ticks=[10**i for i in range(-yticks_start,int(np.log10(min([p for (e,p) in scores])))
#                                                ,-yticks_freq)] + [0.05],
#             labels=[10**i for i in range(-yticks_start,int(np.log10(min([p for (e,p) in scores]))),
#                                      -yticks_freq)],# + [0.05], 
#                )
        plt.ylim(0,0.1)
    else:
#         plt.yticks(ticks=[10**i for i in range(-yticks_start,1,
#                                                -yticks_freq)],
#                    labels=[10**i for i in range(-yticks_start,1,
#                                                -yticks_freq)]
#                )
        plt.ylim(0,0.1)
#         plt.yticks(ticks=[10**i for i in range(-yticks_start,int(min([p for (e,p) in scores]))
#                                                ,-yticks_freq)] + [0.05],
#             labels=[10**i for i in range(-yticks_start,int(min([p for (e,p) in scores])),
#                                      -yticks_freq)], # + [0.05], 
#                )
    # Add xticks on the middle of the group bars
#     plt.xlabel(group, fontweight='bold')
#     plt.xticks([r + barWidth for r in range(len(bars1))], ['A', 'B', 'C', 'D', 'E'])
    plt.legend()
    plt.show()


In [ ]:
barplot_pvalues([("first", 0.001), ("seco", 0.01), ("third", 0.00001)], bar_width=0.5, logy=True)

In [ ]:
for c in categories:
    print(c, 'misclass', predictions_df[predictions_df['misclass']==1][c].mean())
    print(c, 'not misclass', predictions_df[predictions_df['misclass']==0][c].mean())

In [ ]:
for c in emotions:
    print(c, 'misclass', predictions_df[predictions_df['misclass']==1][c].mean())
    print(c, 'not misclass', predictions_df[predictions_df['misclass']==0][c].mean())

In [ ]:
from scipy.stats import pearsonr, ttest_ind
for c in categories:
    print(c)
    print('\tmisclass', predictions_df[predictions_df['misclass']==1][c].mean())
    print('\tnot misclass', predictions_df[predictions_df['misclass']==0][c].mean())    
    ttest = ttest_ind(predictions_df[predictions_df['misclass']==0][c],
                                  predictions_df[predictions_df['misclass']==1][c])
    print('\tttest', ttest)
    print('\tpvalue', ttest[1])

In [ ]:
params = {'axes.labelsize': 25,
          'axes.titlesize': 25,
          'legend.fontsize': 25,
         'xtick.labelsize': 25,
         'ytick.labelsize': 25}
plt.rcParams.update(params)

In [ ]:
from scipy.stats import pearsonr, ttest_ind

def retrieve_differences_misclass_groups(predictions_df, categories, pval_thresh=0.05,
                                        width=5, bar_width=1, angle=60, false_cat=None):
    categories_pscores = {'FN': [], 'FP': []}
#     categories_means = {'FN': [], 'FP': [], 'TN': [], 'TP': []}
    if false_cat == 'FN':
        categories_means = {'FN': [], 'TP': []}
        cols_filters = {'misclass_signed': [('misclass_signed', 1), ('true_label', 1)]}
    elif false_cat == 'FP':
        categories_means = {'FP': [], 'TN': []}
        cols_filters = {'misclass_signed': [('misclass_signed', -1), ('true_label', 0)]}

    else:
#         categories_means = {'FN': [], 'TN': [], 'TP': [], 'FP': []}
        categories_means = {'FN': [], 'TN': [], 'TP': [], 'FP': []}
        cols_filters = {'misclass': [('misclass', 1)]}

    cols_values = {'FN': [('misclass_signed', 1)], 'FP': [('misclass_signed', -1)], 
                   'TN': [('misclass', 0), ('true_label', 0)], 'TP': [('misclass', 0), ('true_label', 1)]}
    
    for c in categories:
        significant = True
        for group in cols_filters:
            cols_filter = cols_filters[group]
            if len(cols_filter)==1:
                col, val = cols_filter[0]
                print(col,val)
                groupdf = predictions_df.query('%s == %d' % (col, val))[c]
                nongroupdf = predictions_df.query('%s != %d' % (col, val))[c]

            if len(cols_filter)==2:
                col1, val1 = cols_filter[0]
                col2, val2 = cols_filter[1]
                groupdf = predictions_df.query('%s == %d' % (col1, val1))[c]
                nongroupdf = predictions_df.query('%s == %d' % (col2, val2))[c]

            ttest = ttest_ind(groupdf, nongroupdf)
            if ttest[1] >= pval_thresh:
                print("Not significant", c)
                significant = False
        if not significant:
            continue
#         for group in cols_values:
        for group in categories_means:
            print(group)
            cols_value = cols_values[group]

            print(c)
            if len(cols_value)==1:
                col, val = cols_value[0]
                groupdf = predictions_df.query('%s == %d' % (col, val))[c]
                nongroupdf = predictions_df.query('%s != %d' % (col, val))[c]

            if len(cols_value)==2:
                col1, val1 = cols_value[0]
                col2, val2 = cols_value[1]
                groupdf = predictions_df.query('%s == %d & %s == %d' % (col1, val1, col2, val2))[c]
                nongroupdf = predictions_df.query('%s != %d | %s != %d' % (col1, val1, col2, val2))[c]

            if len(groupdf)==0:
                print("Group", group, "is empty. Skipping...")
                categories_pscores[group].append((c,0.99))
                categories_means[group].append((c,0.0))
                continue
            print('\t', col, val, groupdf.mean())
            print('\tnon', col, val, nongroupdf.mean())    
            ttest = ttest_ind(groupdf, nongroupdf)
            print('\tttest', ttest)
            print('\tpvalue', ttest[1])
#             if group in ('FN', 'FP'):
#                 categories_pscores[group].append((c,ttest[1]))
#             if ttest[1] < 0.05:
#                 categories_means[group].append((c,groupdf.mean()))
#             else:
            categories_means[group].append((c,groupdf.mean()))

    # emotion_pscores = sorted(emotion_pscores, key=lambda t:t[1])
#     barplot_pvalues(categories_pscores, 3, yticks_start=2, bar_width=bar_width, show_pval=True)
    print(categories_means)
    barplot_pvalues(categories_means, bar_width=10, width=10, angle=angle, nr_bars=len(categories_means), logy=True)



In [ ]:
[e for e in emotions if e in predictions_df.columns]

In [ ]:
retrieve_differences_misclass_groups(predictions_df, [e for e in emotions if e not in ['negative', 'positive']])
#                                     false_cat='FN')

In [ ]:
retrieve_differences_misclass_groups(predictions_df, categories, 
#                                      pval_thresh=0.000001, 
#                                      pval_thresh=0.001, 
                                     width=5, bar_width=0.5, angle=90, false_cat='FP')

In [ ]:
from scipy.stats import pearsonr, ttest_ind
for c in categories:
    print(c)
    print('\tpositive', predictions_df[predictions_df['true_label']==1][c].mean())
    print('\tnegative', predictions_df[predictions_df['true_label']==0][c].mean())    
    ttest = ttest_ind(predictions_df[predictions_df['true_label']==0][c],
                                  predictions_df[predictions_df['true_label']==1][c])
    print('\tttest', ttest)
    print('\tpvalue', ttest[1])

In [ ]:
from scipy.stats import pearsonr, ttest_ind
for c in categories:
    print(c)
    print('\tpredicted positive', predictions_df[predictions_df['predicted_label']==1][c].mean())
    print('\tpredicted_negative', predictions_df[predictions_df['predicted_label']==0][c].mean())    
    ttest = ttest_ind(predictions_df[predictions_df['predicted_label']==0][c],
                                  predictions_df[predictions_df['predicted_label']==1][c])
    print('\tttest', ttest)
    print('\tpvalue', ttest[1])

In [ ]:
from scipy.stats import pearsonr, ttest_ind
categories_pscores = []
for c in categories:
    print(c)
    print('\tmisclassified', predictions_df[predictions_df['misclass']==1][c].mean())
    print('\tnot misclassified', predictions_df[predictions_df['misclass']==0][c].mean())    
    ttest = ttest_ind(predictions_df[predictions_df['misclass']==0][c],
                                  predictions_df[predictions_df['misclass']==1][c])
    print('\tttest', ttest)
    print('\tpvalue', ttest[1])
    if ttest[1] < 0.05:
        categories_pscores.append((c,ttest[1]))
print(sorted(categories_pscores, key=lambda t:t[1]))        
barplot_pvalues(categories_pscores, width=1, show_pval=False, yticks_start=1, yticks_freq=1, angle=90, 
               bar_width=0.5)

        

In [ ]:
# pickle.dump(predictions_df, open('predictions_df_depression_symanto_lstmhierarch64.pkl', 'wb+'))

In [ ]:
# predictions_df = pickle.load(open('predictions_df_depression_erisk.pkl', 'rb'))

### ROC Curve

In [ ]:
# all_predictions
predictions_df.columns
y_test = predictions_df.true_label.values
y_score = predictions_df.predicted_score.values

In [ ]:
from sklearn.metrics import roc_curve, auc
fpr = dict()
tpr = dict()
roc_auc = dict()

fpr, tpr, _ = roc_curve(y_test, y_score)
roc_auc = auc(fpr, tpr)

# Compute micro-average ROC curve and ROC area
# fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
# roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])



In [ ]:
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, 
#          label = dataset_type)
         label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

### Word Cloud

In [ ]:
from wordcloud import WordCloud
# misclassified_users = set(predictions_df[predictions_df['misclass']==1].subject)
text = ""
for tokens in writings_df[writings_df['label']==1].tokenized_text.values:
    text += " ".join(tokens) + " "
# text = " ".join(predictions_df[predictions_df['misclass_signed']==-1].texts)
# Create and generate a word cloud image:
wordcloud = WordCloud( background_color='white', max_words=1000, max_font_size=70).generate(text)

# Display the generated image:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
tweet_tokenizer?

## Explain

In [ ]:
def plot_saliency(loaded_model, pure_txt, text_class, pred_labels, text_sequence):
    text_class = 'Positive' if text_class==1 else 'Negative'
    pred_labels = 'Positive' if pred_labels==1 else 'Negative'

    input_tensors = [loaded_model.input, K.learning_phase()]
    model_input = loaded_model.layers[2].input # the input for convolution layer
    model_output = loaded_model.output[0][1]
    gradients = loaded_model.optimizer.get_gradients(model_output,model_input)
    compute_gradients = K.function(inputs=input_tensors, outputs=gradients)
    matrix = compute_gradients([text_sequence.reshape(1,30), text_class])[0][0]
    matrix = matrix[:len(pure_txt),:]

    matrix_magnify=np.zeros((matrix.shape[0]*10,matrix.shape[1]))
    for i in range(matrix.shape[0]):
        for j in range(10):
            matrix_magnify[i*10+j,:]=matrix[i,:]

    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.imshow(normalize_array(np.absolute(matrix_magnify)), interpolation='nearest', cmap=plt.cm.Blues)
    plt.yticks(np.arange(5, matrix.shape[0]*10, 10), pure_txt, weight='bold',fontsize=24)
    plt.xticks(np.arange(0, matrix.shape[1], 50), weight='bold',fontsize=24)
    plt.title('True Label: "{}" Predicted Label: "{}"'.format(text_class,pred_labels), weight='bold',fontsize=24)
    plt.colorbar()
    plt.show()

In [ ]:
# plot_saliency(models_collection[key], "this is some text to classify", 0, 0, ["this", "is", "some"])

In [ ]:
user_level_data3, subjects_split2, vocabulary = load_erisk_data(writings_df, 
                                                            voc_size=hyperparams_features['max_features'],
                                                           emotion_lexicon=nrc_lexicon,
                                                           emotions=[],
                                                                liwc_categories=[],
                                                           user_level=hyperparams_features['user_level'],
                                                                                logger=logger,
#                                                            vocabulary=pickle.load(open('vocabulary_40K_all.pkl', 'rb')),
#                                                            vocabulary=pickle.load(open('vocab_clpsych_10000.pkl', 'rb')),
                                                              vocabulary=vocabulary_dict,
                                                              by_subset=False
                                                                               )

In [ ]:
%%time
import lime.lime_text as lt
import keras
# my self written library
import numpy as np

# This is the text that I want to explain


#
# recodeText = pl.generate_one_hot(text=inputText, alphabet=alphabet, maxChars=maxChars)

model = models_collection[key]



# subjects = ['subject14410000', 'subject68010000', 'test_subject315', 'test_subject2270', 
#             'test_subject9306', 'test_subject9359']
subjects = [s for s in set(writings_df.subject.values) if s in user_level_data3]
labels = [writings_df[writings_df['subject']==subject].label.values[0] for subject in subjects]
overall_importance_vocabulary = {}

def textForSubject(subject):
    user_level_data_subject = user_level_data3[subject]
    text = "\n".join([" ".join(t) for t in user_level_data_subject['texts'][
        :hyperparams_collection[key]['posts_per_group']]])
    return text
    
# pipeline-like function
# takes raw text as input, converts it to one-hot character vectors via the alphabet
# feeds it into keras' model.predict() function to receive predictions
# works for lists of text as well as for single strings
# TODO: need to make this actually work on the text input, not dummy. it needs to feed new things
def predictFromText(inputTexts):
    if type(inputTexts) != list:
        inputTexts = [inputTexts]
    
    # list for predictions
    predStorage = []
    # loop through input list and predict

    predictions = models_collection[key].predict(DataGeneratorFromText(inputTexts,
                                        seq_len=hyperparams_collection[key]['maxlen'],
                                        liwc_words_for_categories=liwc_words_for_categories,
#                                         emotions=[], liwc_categories=[]
                                                             ), verbose=1)
    for i, prediction in enumerate(predictions):
        pred = int(prediction>=0.5)

#         recodeText = pl.generate_one_hot(text=textInput, alphabet=alphabet, maxChars=maxChars)
#         pred = model.predict(recodeText.transpose())
        # control output of function
#         print("TEXT", str(inputTexts[i][:100]), "PRED", prediction, "\n")
        predStorage.append((1-prediction[0], prediction[0]))
    return(np.asarray(predStorage))

inputTexts = [textForSubject(subject) for subject in subjects]

print(len(inputTexts))
print(labels)

# print(inputTexts)
# print(predictFromText(inputTexts))
# Lime Explainer
# bow controls if words are perturbed or overwritten with UNKWORDZ
# False makes sense, if location of words is important as in this classifier
explainer = lt.LimeTextExplainer(
#     kernel_width=25, 
    verbose=False, class_names=['not depressed', 'depressed'],
#                            feature_selection="auto", split_expression=" ", bow=False
)
for i, inputText in enumerate(inputTexts):
    print("EXPLAINING...", i)
    exp = explainer.explain_instance(text_instance=inputText, labels=(0,1), 
#                                      labels=[labels[i]],
                         classifier_fn=predictFromText, 
                                     num_features=100, num_samples=1000
                                    )
    for word, weight in exp.as_list():
        if word not in overall_importance_vocabulary:
            overall_importance_vocabulary[word] = []
        overall_importance_vocabulary[word].append(weight)
    # this works, yields an array with probabilities for both classes
    # print(predictFromText(textInputList = listTexts))
    # print(len(predictFromText(inputTexts[0])), len(inputTexts))



In [ ]:
sorted(user_level_data3)

In [ ]:
exp.as_list()

In [ ]:
explainer.explain_instance?

In [ ]:
exp.as_pyplot_figure()

In [ ]:
exp.show_in_notebook()

In [ ]:
explainer.explain_instance?

In [ ]:
overall_importance_vocabulary_averages = {word: np.mean(overall_importance_vocabulary[word]) 
                                          for word in overall_importance_vocabulary 
                                          if len(overall_importance_vocabulary[word])>=2}
overall_importance_vocabulary_averages_abs = {word: np.abs(np.mean(overall_importance_vocabulary[word])) 
                                              for word in overall_importance_vocabulary
                                             if len(overall_importance_vocabulary[word])>=2}

In [ ]:
sorted(overall_importance_vocabulary_averages_abs.items(), key=lambda t: t[1], reverse=True)[:20]

In [ ]:
[(t[0], np.mean(t[1]), len(t[1])) for t in 
 sorted(overall_importance_vocabulary.items(), key=lambda t: len(t[1]), reverse=True)[:20]]

In [ ]:
for word, weight in sorted(overall_importance_vocabulary_averages_abs.items(), key=lambda t: t[1], reverse=True)[:20]:
    print(word, overall_importance_vocabulary[word])

In [ ]:
# pickle.dump(overall_importance_vocabulary, open('overall_importance_vocabulary_depression.pkl', 'wb+'))

In [ ]:
# overall_importance_vocabulary = pickle.load(open('overall_importance_vocabulary_selfharm.pkl', 'rb'))

In [ ]:
dataset_type

In [ ]:
pd.Series([len(overall_importance_vocabulary[w]) for w in overall_importance_vocabulary]).hist(log=True)

In [ ]:
explainer.explain_instance?

In [ ]:
for x in DataGenerator(user_level_data3, {'test':[subject]}, 
                                         set_type='test', 
                                    seq_len=hyperparams_collection[key]['maxlen'],   
#                                         batch_size=len(subjects_split['test']), # on all data at once
                                        batch_size=hyperparams_collection[key]['batch_size'], # on all data at once
                                       hierarchical=hyperparams_collection[key]['hierarchical'],
                                         max_posts_per_user=None,
                                       pad_with_duplication=False,
                                        posts_per_group=hyperparams_collection[key]['posts_per_group'],
                                        post_groups_per_user=1,  compute_liwc=True,
                                        liwc_words_for_categories=liwc_words_for_categories,
                                         sample_seqs=False, shuffle=False,
                                        emotions=[], liwc_categories=[]
                                                             ):
    print(x)
    break

### 2D visualization of emotions

In [ ]:
import datetime
def normalize_date(date):
        return " ".join(date.strip().split())
writings_df['datetime'] = writings_df.date.apply(lambda d: datetime.datetime.strptime(
            normalize_date(d), '%Y-%m-%d %H:%M:%S'))
date_cutoff1 = datetime.datetime.strptime('2010','%Y').date()
date_cutoff2 = datetime.datetime.strptime('2020','%Y').date()

In [ ]:
users_emotions = writings_df[
    writings_df['datetime'].between(date_cutoff1, date_cutoff2)
].groupby('subject').mean()[list(categories) + emotions + ['label']]

In [ ]:
from sklearn.decomposition import PCA
emotions_dict=users_emotions.dropna().to_dict(orient='index')
subjects = list(emotions_dict.keys())
emotions_matrix=[list(emotions_dict[s].values())[:-1] for s in subjects]
emotions_labels=[emotions_dict[s]['label'] for s in subjects]

In [ ]:
pca = PCA(n_components=2)
pca.fit(np.array(emotions_matrix).transpose())
col = plt.scatter(pca.components_[0], pca.components_[1], c=['r' if l==1 else 'b' for l in emotions_labels],
           s=[0.5 for s in emotions_labels])

ax = col.axes
handles, labels = ax.get_legend_handles_labels()

import matplotlib.patches as mpatches


# manually define a new patch 
handles.append(mpatches.Patch(color='red', label='anorexic users'))
handles.append(mpatches.Patch(color='blue', label='non anorexic users'))


# plot the legend
plt.legend(handles=handles, loc='upper center')




In [ ]:
pca.

## Cross-validation

In [ ]:
experiment = initialize_experiment(hyperparams, nrc_lexicon_path, emotions, pretrained_embeddings_path, 
                          dataset_type, transfer_type)
experiment.add_tag("cross-validation")

In [ ]:
folds = 5

f1_scores = []
for test_slice in range(folds): 
    logger.info("Testing on slice %d, training on the rest...\n" % test_slice)
    user_level_data, subjects_split, vocabulary = load_erisk_data(writings_df, 
                                                            voc_size=hyperparams_features['max_features'],
                                                           emotion_lexicon=nrc_lexicon,
                                                           emotions=emotions,
                                                           user_level=hyperparams_features['user_level'],
                                                                                logger=logger,
                                                           vocabulary=pickle.load(open('all_vocab_clpsych_erisk_2000_dict.pkl', 'rb')),
                                                              by_subset=False,
                                                              nr_slices=folds,
                                                                valid_prop=0.1,
                                                                  test_slice=test_slice
                                                                 )
    model, history = train(user_level_data, subjects_split, 
              hyperparams, hyperparams_features, 
              embedding_matrix, emotions, stopword_list, liwc_categories,
                  experiment,
                validation_set='test',
              version=8, epochs=50,
    )
    f1_scores.append(sum(history.history['val_f1_m'][-6:-1])/5)
experiment.add_metric("average_f1", sum(f1_scores)/5)

## Predict on server data

In [ ]:
# initialize_vars(sess)

In [ ]:
# model_path = "models/lstm_selfharm_seq16"

In [ ]:
hyperparams1, hyperparams_features1 = load_params(model_path)


In [ ]:
# hyperparams_features1['embeddings_path']=[
#     '/home/anasab//eRisk/embeddings/finetuned_glove_clpsych_erisk_stop_20000_2.pkl']

In [ ]:
hyperparams_features1

In [ ]:
loaded_model = load_saved_model(model_path, hyperparams1)


In [ ]:
vocabulary_list = pickle.load(open(hyperparams_features1['vocabulary_path'], 'rb'))
vocabulary_dict={}
for i,w in enumerate(vocabulary_list):
    vocabulary_dict[w] = i
embedding_matrix = load_embeddings2(hyperparams_features1['embeddings_path'], 
                                    hyperparams_features1['embedding_dim'], vocabulary_dict)
liwc_words_for_categories = pickle.load(open(hyperparams_features1['liwc_words_cached'], 'rb'))

In [ ]:
# if 'optimizer' not in hyperparams1:
#         hyperparams1['optimizer'] = optimizers.Adam(lr=hyperparams1['lr'], #beta_1=0.9, beta_2=0.999, epsilon=0.0001,
#                                        decay=hyperparams1['decay'])
# loaded_model = initialize_model(hyperparams1, hyperparams_features1, embedding_matrix, emotions, stopword_list,
#                     liwc_categories)


In [ ]:
loaded_model.load_weights(model_path + "_weights.h5", by_name=True)
# metrics_class = Metrics(threshold=0.5)
# loaded_model.compile(hyperparams1['optimizer'], binary_crossentropy_custom,
#                   metrics=[metrics_class.f1_m, metrics_class.precision_m, metrics_class.recall_m])


### Server data

In [ ]:
models_runs = {
    'bert': 0,
    'lstm_seq': 1,
    'cnn_hierarch': 2,
    'transfer': 3,
    'ensemble': 4,
}

In [ ]:
model_key = 'lstm_seq'

In [ ]:
from collections import Counter
def load_erisk_server_data(datarounds_json, voc_size, emotion_lexicon, tokenizer,
                           liwc_words_for_categories,
                           emotions =  
                    ['anger', 'anticipation', 'disgust', 'fear', 'joy', 
                     'negative', 'positive', 'sadness', 'surprise', 'trust'],
                    liwc_categories = liwc_categories, 
                    pronouns = ["i", "me", "my", "mine", "myself"],
                    user_level=True, vocabulary=vocabulary_dict,
                   logger=logger):
    def __encode_liwc_categories(tokens, liwc_words_for_categories, relative=True):
        categories_cnt = [0 for c in liwc_categories]
        if not tokens:
            return categories_cnt
        text_len = len(tokens)
        for i, category in enumerate(liwc_categories):
            for t in tokens:
                if t in liwc_words_for_categories[category]:
                    categories_cnt[i] += 1
            if relative and text_len:
                categories_cnt[i] = categories_cnt[i]/text_len
        return categories_cnt
    logger.debug("Loading data...\n")
     
    subjects_split = {'test': []}
 
    user_level_texts = {}
    for datapoints_json in datarounds_json:
        for datapoint in datapoints_json:
            words = []
            raw_text = ""
            if "title" in datapoint:
                tokenized_title = tokenizer.tokenize(datapoint["title"])
                words.extend(tokenized_title)
                raw_text += datapoint["title"]
            if "content" in datapoint:
                tokenized_text = tokenizer.tokenize(datapoint["content"])
                words.extend(tokenized_text)
                raw_text += datapoint["content"]
            
            liwc_categs = __encode_liwc_categories(words, liwc_words_for_categories)
            if datapoint["nick"] not in user_level_texts.keys():
                user_level_texts[datapoint["nick"]] = {}
                user_level_texts[datapoint["nick"]]['texts'] = [words]
                user_level_texts[datapoint["nick"]]['liwc'] = [liwc_categs]
                user_level_texts[datapoint["nick"]]['raw'] = [raw_text]
                subjects_split['test'].append(datapoint['nick'])
            else:
                user_level_texts[datapoint["nick"]]['texts'].append(words)
                user_level_texts[datapoint["nick"]]['liwc'].append(liwc_categs)
                user_level_texts[datapoint["nick"]]['raw'].append(raw_text)
            
    return user_level_texts, subjects_split, vocabulary

In [ ]:
def read_json_datapoint(jlpath):
    datapoints = []
    with open(jlpath) as f:
        for line in f:
            datapoints.append(json.loads(line))
    return datapoints

In [ ]:
# %%time
# erisk_server_data, erisk_server_subjects_split, vocabulary = load_erisk_server_data(data,
#                        tokenizer=regtokenizer,
#                        liwc_words_for_categories=liwc_words_for_categories,
#                     voc_size=hyperparams_features['max_features'],
#                     emotion_lexicon=nrc_lexicon,
#                     emotions=emotions,
#                     user_level=hyperparams_features['user_level'],
#                        vocabulary=vocabulary_dict,
#     #                                                            vocabulary=pickle.load(open('vocabulary20K_selfharm.pkl', 'rb'))
#                     logger=logger)

In [ ]:
erisk_server_data

In [ ]:
%%time
#TODO: predict only on last 50 posts

with session_collection[model_key].as_default():
    with session_collection[model_key].graph.as_default():
        server_erisk_predictions = models_collection[model_key].predict(DataGenerator(erisk_server_data, erisk_server_subjects_split, 
                                             set_type='test', vocabulary=vocabulary_dict, 
                                           hierarchical=hyperparams_collection[model_key]['hierarchical'],
                                        seq_len=hyperparams['maxlen'], batch_size=hyperparams['batch_size'],
                                             max_posts_per_user=None,
                                           pad_with_duplication=False,
                                            posts_per_group=hyperparams['posts_per_group'],
                                            post_groups_per_user=None, 
                                             sample_seqs=False, shuffle=False,
                                                    compute_liwc=False)
                                                                       )
        pd.Series(server_erisk_predictions.flatten()).describe()

In [ ]:
pd.Series(server_erisk_predictions.flatten()).describe()

### Seding results to server!

In [ ]:
# subjects = [d['nick'] for d in read_json_datapoint("client/data0.jl")]
# results = {s: 0 for s in subjects}

In [ ]:
# def get_next_data(rnd, results):
#     # TODO: send results to get data
#     response = build_response(results)
#     # Send response
#     data = {"..."}
#     # Make sure it's the correct round
#     assert data['number'] == rnd
#     serialize_data(data)
#     return data

In [ ]:
# def get_next_data_dummy(rnd, results):
#     return read_json_datapoint("client/data0.jl")

In [ ]:
# Results for round and model
results = {key: {} for key in models_runs}

In [ ]:
def get_data_chunk(rnds):
    # Send same results for a chunk of rounds to get new posts
    data = [read_json_datapoint("data_server/data%d.jl" % i) for i in rnds]
#     data_chunks = []
#     for rnd in rnds:
#         # TODO: REPLACE THIS WITH THE CORRECT ONE
#         data = get_next_data_dummy(rnd, results)
#         data_chunks.append(data)
#         all_data[rnd] = data
    return data

In [ ]:
%%time
def predict_for_round_chunk(model, hyperparams, hyperparams_features, vocabulary, data_chunk, subjects=[],
                           model_key='', cache_round=None): 
    # preload for subjects not occurring in the round with results from previous round
#     results = {s: 0 for s in subjects}
    if cache_round:
        results = load_results(model_key, cache_round)
    else:
        results = {s: 0 for s in subjects}
        
    
    erisk_server_data, erisk_server_subjects_split, vocabulary = load_erisk_server_data(data_chunk,
                       tokenizer=regtokenizer,
                       liwc_words_for_categories=liwc_words_for_categories,
                    voc_size=hyperparams_features['max_features'],
                    emotion_lexicon=nrc_lexicon,
                    emotions=emotions,
                    user_level=1,
                       vocabulary=vocabulary,
                    logger=logger)

    for features, subjects, _ in DataGenerator(erisk_server_data, erisk_server_subjects_split, 
                                         set_type='test', vocabulary=vocabulary, 
                                       hierarchical=hyperparams['hierarchical'],
                                    seq_len=hyperparams['maxlen'], batch_size=hyperparams['batch_size'],
                                         max_posts_per_user=None,
                                       pad_with_duplication=False,
                                        posts_per_group=hyperparams['posts_per_group'],
                                        post_groups_per_user=None, 
                                         sample_seqs=False, shuffle=False,
                                      return_subjects=True):
        predictions = model.predict_on_batch(features)
        print(len(features[0]), len(subjects), len(predictions), len(results))
        for i,s in enumerate(subjects):
            results["subject" + str(s)] = predictions[i].item()
    return(results)


In [ ]:
def build_response(results, decision_thresh=0.5, model_name='', rnd=0):
    response = []
    for subject, score in results.items():
        prediction = 1 if score >= decision_thresh else 0
        response.append({'nick': subject, 'score': score, 'decision': prediction})
    json.dump(response, open("data_server/response_run%s_rnd%d.json" % (model_name, rnd), 'w+'))
    return response
# build_response(results, 50000)

In [ ]:
# def get_ensemble_results(rnd, all_results, model_keys_to_average=['lstm_seq', 'cnn_hierarch']):
#     subjects = [s for s in all_results[model_keys_to_average[0]][rnd]]
#     results_ensemble = {}
#     for sub in subjects:
#         s = 0
#         for k in model_keys_to_average:
#             s += all_results[k][rnd][sub]
#         results_ensemble[sub] = s/len(model_keys_to_average)
#     return results_ensemble

In [ ]:
def get_ensemble_results(results_to_average):
    subjects = [s for s in results_to_average[0]]
    results_ensemble = {}
    for sub in subjects:
        s = 0
        for res in results_to_average:
            s += res[sub]
        results_ensemble[sub] = s/len(results_to_average)
    return results_ensemble

In [ ]:
def get_transfer_results(rnd, all_results, model_to_average='lstm_seq', rounds_back=100):
    subjects = [s for s in all_results[model_to_average][rnd]]
    results_ensemble = {}
    for sub in subjects:
        s = 0
        existing_rounds = 0
        for prev_rnd in range(rnd-rounds_back, rnd+1):
#             print("rolling rnds", prev_rnd)
            if prev_rnd in all_results[model_to_average]:
                s += all_results[model_to_average][prev_rnd][sub]
                existing_rounds += 1
        results_ensemble[sub] = s/existing_rounds
#         print("Have found a rolling window of %d for the transfer model" % existing_rounds)
    return results_ensemble

In [ ]:
def load_results(model_key, rnd):
    results = {}
    with open("data_server/response_run%s_rnd%d.json" % (models_runs[model_key], rnd)) as f:
        response = json.loads(f.read())
        for line in response:
            results[line['nick']] = line['score']
    return results

In [ ]:
# results['lstm_seq'][20] = load_results('lstm_seq', 20)
# results['lstm_seq'][40] = 
results# results['bert'][40] = load_results('bert', 40)

In [ ]:
rnds = range(500,600)
decision_thresh = 0.5
data_chunks = get_data_chunk(rnds)
subjects = [d['nick'] for d in read_json_datapoint("data_server/data0.jl")]
# for key in ['transfer', 'ensemble']:
for model_key in [
                  'lstm_seq',
                    'bert',
                  'cnn_hierarch', 
                  'transfer', 
                  'ensemble',
                ]:
    print(model_key)
    end_rnd = rnds[-1]
#     if model_key=='lstm_seq':
#         results[model_key][end_rnd]=load_results('lstm_seq', 20)
    if model_key=='cnn_hierarch':
        results[model_key][end_rnd]=load_results('cnn_hierarch', 40)
    elif model_key=='bert':
        results[model_key][end_rnd]=load_results('bert', end_rnd)
    elif model_key=='ensemble':
        model_keys_to_average=['bert', 'cnn_hierarch', 'lstm_seq']
        missing_models = [m for m in model_keys_to_average if not results[m]]
        if len(missing_models)!=0:
            print("Missing models! cannot compute ensemble results", missing_models)
            continue
        results_to_average = [results[m][end_rnd] for m in model_keys_to_average]
#         results[model_key][end_rnd] = get_ensemble_results(rnd, results, 
#                                                 model_keys_to_average)
        results[model_key][end_rnd] = get_ensemble_results(results_to_average)
    ## For now
    elif model_key=='transfer':
        results[model_key][end_rnd]=get_transfer_results(
            end_rnd, results, model_to_average='lstm_seq', rounds_back=60)
#         results[model_key][end_rnd]=results['lstm_seq'][end_rnd]
    ##
    else:
        with session_collection[model_key].as_default():
            with session_collection[model_key].graph.as_default():
                results[model_key][end_rnd] = predict_for_round_chunk(models_collection[model_key], 
                                              hyperparams_collection[model_key], hyperparams_features, 
                                              vocabulary_dict, 
                                          data_chunks, subjects=subjects, model_key=model_key, cache_round=499)

    
    print(len(results[model_key][end_rnd].values()), "positive:", 
      len([r for r in results[model_key][end_rnd].values() if r >=0.5]))
    response1 = build_response(results[model_key][end_rnd], rnd=end_rnd, 
                               model_name=models_runs[model_key], decision_thresh=decision_thresh)


In [ ]:
results['lstm_seq'].keys()

In [ ]:
pd.Series(list(results['transfer'][180].values())).describe()

In [ ]:
from scipy.stats import pearsonr
pearsonr(list(results['bert'][220].values()), list(results['bert'][180].values()))

In [ ]:
pd.Series(list(results['lstm_seq'][160].values()))

### Predict on eRisk data

In [ ]:
# results = {}
# labels = {}
# featuresall = {}
# with session_collection['lstm_seq2'].as_default():
#     with session_collection['lstm_seq2'].graph.as_default():
        # for features, subjects, lbls in DataGenerator(user_level_data, subjects_split, 
        #                                          set_type='train', vocabulary=vocabulary_dict,
        #                                        hierarchical=hyperparams1['hierarchical'],
        #                                     seq_len=hyperparams1['maxlen'], batch_size=hyperparams1['batch_size'],
        #                                          max_posts_per_user=None,
        #                                        pad_with_duplication=False,
        #                                         posts_per_group=hyperparams1['posts_per_group'],
        #                                         post_groups_per_user=None, 
        #                                          sample_seqs=False, shuffle=False,
        #                                                return_subjects=True):

        #     predictions = loaded_model.predict_on_batch(features)
        #     print(len(features[0]), len(subjects), len(predictions), len(labels), len(results))
        #     for i,s in enumerate(subjects):
        #         if s not in results:
        #             results[s] = []
        #             featuresall[s] = []
        #         results[s].append(predictions[i].item())
        #         featuresall[s].append([features[j][i] for j in range(len(features))])
        #         labels[s] = lbls[i]


In [ ]:
for subject in results:
    if not labels[subject]:
        if np.std(results[subject])>0.0:
            print(subject), print(results[subject][0], results[subject][-1]-results[subject][0])
            pd.Series(results[subject]).rolling(window=5).mean().plot()

In [ ]:
[featuresall[4278][i][0].sum() for i in range(len(featuresall[4278]))]

In [ ]:
len(user_level_data['subject4278']['raw'])

In [ ]:
for x, s, y in DataGenerator(user_level_data, subjects_split, 
                                         set_type='test', vocabulary=vocabulary_dict,
                                       hierarchical=hyperparams1['hierarchical'],
                                    seq_len=hyperparams1['maxlen'], batch_size=hyperparams1['batch_size'],
                                         max_posts_per_user=None,
                                       pad_with_duplication=False,
                                        posts_per_group=hyperparams1['posts_per_group'],
                                        post_groups_per_user=None, 
                                         sample_seqs=False, shuffle=False,
                                               return_subjects=True):
    print("subject", s, "features", x[0].sum(axis=1))
    

In [ ]:
model_key='lstm_seq'
with session_collection[model_key].as_default():
    with session_collection[model_key].graph.as_default():
        res = models_collection[model_key].evaluate_generator(DataGenerator(user_level_data, subjects_split, 
                                              liwc_words_for_categories=liwc_words_for_categories,
                                         set_type='test', vocabulary=vocabulary_dict,
                                       hierarchical=hyperparams_collection[model_key]['hierarchical'],
                                    seq_len=hyperparams_collection[model_key]['maxlen'], 
                                    batch_size=hyperparams['batch_size'],
                                         max_posts_per_user=None,
                                       pad_with_duplication=False,
                                        posts_per_group=hyperparams_collection[model_key]['posts_per_group'],
                                        post_groups_per_user=1,#None, 
                                         sample_seqs=False, shuffle=False,
                                             compute_liwc=False))
        print(res)

In [ ]:
sessions_collection

In [ ]:
# hyperparams_collection['cnn_bert'] = {'hierarchical': False, 'maxlen': 512, 'posts_per_group': 50}

In [ ]:
# hyperparams_collection

###### Feature importance

In [ ]:
features = [
    (e, 'nrc') for e in emotions] + ['pers_pronouns'] + [
    (c, 'liwc') for c in list(categories) if c in writings_df.columns] + [
(st, 'stopword') for st in stopword_list]
weights = model.get_layer('output_layer').get_weights()[0].tolist()[-(len(features)):]

print(len(weights), len(features))
feature_importance = {}
for (i, f) in enumerate(features):
    feature_importance[f] = weights[i][0]

sorted(feature_importance.items(), key=lambda t: abs(t[1]), reverse=True)

## Evaluate per user

In [ ]:
def get_data_for_point(subject, voc, hyperparams_features=hyperparams_features, nrc_lexicon=nrc_lexicon,
                      emotions=emotions):
    eval_writings_df = writings_df[writings_df['subject']==subject]
    correct_label = eval_writings_df.label.values[0]
    (x_train, y_train), (x_valid, y_valid), (x_test, y_test), voc = load_erisk_data(eval_writings_df,
                        seq_len=hyperparams_features['maxlen'],
                        voc_size=hyperparams_features['max_features'],
                        emotion_lexicon=nrc_lexicon,
                        emotions=emotions, user_level=False,
                        train_prop=0.0, vocabulary=voc)
    return x_test, y_test, correct_label

In [ ]:
def predict_per_user(writings_df, majority_prop=0.2, train_prop=0.7, majority_nr=0, validate=False, voc=None,
                    random=False, nr_slices=5, test_slice=2):
    all_predictions = []
    all_labels = []
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    thresh=0.5
    majority_proportion=majority_prop
    valid_prop = 0.3
    
    if 'subset' in writings_df.columns:
        training_subjects = list(set(writings_df[writings_df['subset']=='train'].subject))
        test_subjects = list(set(writings_df[writings_df['subset']=='test'].subject))
    else:
        all_subjects = sorted(list(set(writings_df.subject)))
        training_subjects_size = int(len(all_subjects) * train_prop)
        test_subjects_size = len(all_subjects) - training_subjects_size
        # Cross-validation, with fixed slice as input
        test_prop = 1-train_prop
        test_slice = min(test_slice, nr_slices)
        logger.debug("start index: %f, from %f\n" % (
            len(all_subjects)*(1/nr_slices)*test_slice, test_prop*test_slice))
        start_slice = int(len(all_subjects)*(1/nr_slices)*test_slice)
        test_subjects = all_subjects[start_slice: start_slice+test_subjects_size]
        training_subjects = [s for s in all_subjects if s not in test_subjects]
    training_subjects = sorted(training_subjects) # ensuring reproducibility
    valid_subjects_size = int(len(training_subjects) * valid_prop)
    valid_subjects = training_subjects[:valid_subjects_size]
    training_subjects = training_subjects[valid_subjects_size:]
    
    if validate:
        subjects = valid_subjects
    else:
        subjects = test_subjects
    for subject in subjects:
        x_test_user, y_test_user, label = get_data_for_point(subject, voc=voc)
        outputs = model.predict(x_test_user)
        if random:
            sigma = np.std(outputs)
            mu = np.mean(outputs)
            print("generating random outputs with sigma", sigma, "and mu", mu)
            outputs = sigma*np.random.randn(len(outputs))+mu
        positive_pred = sum(outputs>=thresh)
        negative_pred = sum(outputs<thresh)
        majority_pred = 0
        if majority_proportion and positive_pred >= majority_proportion*negative_pred:
            majority_pred = 1
        if majority_nr and positive_pred>=majority_nr:
            majority_pred = 1
        if label == 1:
            if majority_pred == 1:
                tp+=1
            else:
                fn+=1
        else:
            if majority_pred == 0:
                tn+=1
            else:
                fp+=1
        print(negative_pred, positive_pred, majority_pred)
        all_predictions.append(majority_pred)
        all_labels.append(label)
    def prec_recall_f1(tp, fp, tn, fn):
        recall = tp/(tp+fn+0.0000001)
        precision = tp/(tp+fp+0.0000001)
        f1 = 2*precision*recall/(precision+recall+0.0000001)
        print("Recall", recall, "Precision", precision, "F1", f1)
    if majority_prop:
        print("Vote proportion", majority_prop)
    if majority_nr:
        print("Vote points", majority_nr)
    prec_recall_f1(tp, fp, tn, fn)

        

In [ ]:
predict_per_user(writings_df=writings_df, voc=voc, majority_prop=0.2)

## Cross-validation

In [ ]:
results_per_slice = {}

In [ ]:
nr_slices=5
logger.setLevel(logging.INFO)
for tslice in range(nr_slices): 
    (x_train, y_train), (x_valid, y_valid), (x_test, y_test), voc = load_erisk_data(writings_df, 
                                                                seq_len=hyperparams_features['maxlen'],
                                                                voc_size=hyperparams_features['max_features'],
                                                               emotion_lexicon=nrc_lexicon,
                                                               emotions=emotions,
                                                               user_level=hyperparams_features['user_level'],
                                                                                    test_slice=tslice,
                                                                                    nr_slices=nr_slices,
    #                                                            vocabulary=pickle.load(open('vocabulary20K_selfharm.pkl', 'rb'))
                                                                                   logger=logger)
    model, history = train_model(model, x_train, y_train, x_valid, y_valid,
           epochs=200, batch_size=hyperparams['batch_size'],
                      class_weight={0:0.5, 1:5}, start_epoch=0,
                      callback_list = [freeze_layer, weights_history, reduce_lr],
                      workers=2, verbose=0)
    results_per_slice[tslice] = model.evaluate(x_test, y_test)
    logger.info("Results for slice %d: %s\n" % (tslice, results_per_slice[tslice]))

In [ ]:
print("Average F1 score: ", np.array([results_per_slice[s][1] for s in results_per_slice.keys()]).mean(),
     "all F1 scores: ", {s: v[1] for (s,v) in results_per_slice.items()} )

## Extra analysis


In [ ]:
def merge_tokens(row):
    tokens = []
    if row.tokenized_text:
        tokens += row.tokenized_text
    if row.tokenized_title:
        tokens += row.tokenized_title
    return tokens
writings_df['all_tokens'] = writings_df.apply (lambda row: merge_tokens(row), axis=1)

In [ ]:
# TODO: include the title
def extract_emotions(tokens, emotion, relative=True):
    if not tokens:
        return None
    emotion_words = [t for t in tokens 
                     if t in nrc_lexicon[emotion]]
    if relative and len(tokens):
        return len(emotion_words) / len(tokens)
    else:
        return len(emotion_words)
    
    return encoded_emotions

from functools import partial
for emotion in emotions:
    writings_df[emotion] = writings_df['all_tokens'].apply(partial(extract_emotions, emotion=emotion, 
                                                                   relative=True))


In [ ]:
writings_df['pronouns'] = writings_df['all_tokens'].apply(partial(encode_pronouns, relative=True))

In [ ]:
writings_df[['text', 'label', 'pronouns', 'text_len'] + emotions].corr()

In [ ]:
writings_df[['text', 'label', 'pronouns', 'text_len'] + emotions].groupby('label').mean()

In [ ]:
from nltk.sentiment import SentimentAnalyzer, SentimentIntensityAnalyzer

In [ ]:
sid = SentimentIntensityAnalyzer()


In [ ]:
sid.polarity_scores("We are here today happiness is all around")

In [ ]:
writings_df['neg_vader'] = writings_df.text.apply(lambda t: sid.polarity_scores(t)['neg']
                                                 if type(t)==str else 0)

In [ ]:
writings_df

In [ ]:
writings_df['pos_vader'] = writings_df.text.apply(lambda t: sid.polarity_scores(t)['pos']
                                                 if type(t)==str else 0)

In [ ]:
writings_df[['text', 'label', 'pronouns', 'text_len', 'neg_vader', 'pos_vader'] + emotions].groupby('label').mean()

In [ ]:
writings_df[['text', 'label', 'pronouns', 'text_len', 'neg_vader', 'pos_vader'] + emotions].corr('spearman')

### LIWC

In [ ]:
from liwc_readDict import readDict

liwc = readDict('/home/ana/resources/FakeOrFact/features/LIWC/LIWC/liwc.dic')

In [ ]:
categories = [c for (w,c) in liwc]
set(categories)

In [ ]:
liwc

In [ ]:
liwc_dict = {}
for (w, c) in liwc:
    if c not in liwc_dict:
        liwc_dict[c] = []
    liwc_dict[c].append(w)

    
    

In [ ]:
liwc_dict['pronoun']

In [ ]:
def encode_liwc_categories(tokens, category_words, relative=True):
    category_cnt = 0
    if not tokens:
        return None
    text_len = len(tokens)
    for t in tokens:
        for word in category_words:
            if t==word or (word[-1]=='*' and t.startswith(word[:-1])) \
            or (t==word.split("'")[0]):
                category_cnt += 1
                break # one token cannot belong to more than one word in the category
    if relative:
        return category_cnt/text_len
    else:
        return category_cnt

In [ ]:
    def __encode_liwc_categories(self, tokens, relative=True):
        categories_cnt = [0 for c in self.liwc_categories]
        if not tokens:
            return None
        text_len = len(tokens)
        for i, category in enumerate(self.liwc_categories):
            for t in tokens:
                print("token", t)
                if t in self.liwc_words_for_categories[category]:
                    categories_cnt[i] += 1
            if relative:
                categories_cnt[i] = categories_cnt[i]/text_len
        return categories_cnt

In [ ]:
%%time
from functools import partial
# for categ in ['negemo', 'posemo', 'affect', 'sad', 'anx', 'pronoun']:#liwc_dict.keys():
for categ in liwc_dict.keys():
    if categ in writings_df.columns:
        continue
    print("Encoding for category %s..." % categ)
    writings_df[categ] = writings_df['all_tokens'].apply(partial(encode_liwc_categories, 
                                                                   category_words=liwc_dict[categ], 
                                                                   relative=True))


In [ ]:
%%time
# categories for all words in vocabulary
# vocabulary = pickle.load(open("all_vocab_clpsych_erisk_50000.pkl", "rb"))
liwc_words_for_categories = {}
for categ in liwc_dict.keys():
    liwc_words_for_categories[categ] = set()
    for word in liwc_dict[categ]:
        for t in vocabulary:
            if t==word or (word[-1]=='*' and t.startswith(word[:-1])) \
                or (t==word.split("'")[0]):
                    liwc_words_for_categories[categ].add(t)

In [ ]:
pickle.dump(liwc_words_for_categories, open("data/liwc_categories_for_vocabulary_erisk_clpsych_stop_40K.pkl", "wb+"))

In [ ]:
writings_df.groupby('subject').mean()[['label', 'negemo', 'posemo', 'affect', 'sad', 'anx', 'pronoun']].corr()

In [ ]:
writings_df[['label', 'negemo', 'posemo', 'affect', 'sad', 'anx', 'pronoun']].groupby('label').mean()

In [ ]:
writings_df.groupby('subject').mean()[['label'] + categories].corr()

## Hyperparameter tuning

In [ ]:
# Declare your hyperparameters search:
tune_epochs=150
config = {
      "algorithm": "bayes",
      "parameters": {
          "lstm_units": {"type": "integer", "min": 32, "max": 300},
          "lstm_units_user": {"type": "integer", "min": 5, "max": 40},
          "dense_bow_units": {"type": "integer", "min": 5, "max": 35},
          "filters": {"type": "integer", "min": 30, "max": 250},
          "kernel_size": {"type": "integer", "min": 3, "max": 7},
          "dense_sentence_units": {"type": "integer", "min": 0, "max": 10},
          "dense_user_units": {"type": "integer", "min": 0, "max": 0},
          "bert_dense_units": {"type": "integer", "min": 10, "max": 200},
          "bert_finetune_layers": {"type": "integer", "min": 0, "max": 2},
          "lr": {"type": "float", "min": 0.0005, "max": 0.2, "scalingType": "loguniform"},
          "l2_dense": {"type": "float", "min": 0.0000001, "max": 0.1, "scalingType": "loguniform"},
          "l2_embeddings": {"type": "float", "min": 0.00000001, "max": 0.2, "scalingType": "loguniform"},
          "l2_bert": {"type": "float", "min": 0.00000001, "max": 0.2, "scalingType": "loguniform"},
          "dropout": {"type": "float", "min": 0, "max": 0.5, "scalingType": "uniform"},
          "norm_momentum": {"type": "float", "min": 0, "max": 0.99, "scalingType": "uniform"},
          "optimizer": {"type": "categorical", "values": [""]},#"adam", "adagrad", ""]},
          "batch_size": {"type": "integer", "min": 5, "max": 128, "scalingType": "loguniform"},
          "positive_class_weight": {"type": "integer", "min": 3, "max": 10},
          "trainable_embeddings": {"type": "discrete", "values": [True, False]},
          "sample_seqs": {"type": "discrete", "values": [True, False]},
          "bert_trainable": {"type": "discrete", "values": [True, False]},
          "bert_pooling": {"type": "categorical", "values": ['first', 'mean']},
#           "hierarchical": {"type": "discrete", "values": [True, False]},
          "freeze_patience": {"type": "integer", "min": 2, "max": tune_epochs+1},
          "lr_reduce_factor": {"type": "float", "min": 0.0001, "max": 0.8},
          "scheduled_lr_reduce_factor": {"type": "float", "min": 0.0001, "max": 0.8},
          "lr_reduce_patience": {"type": "integer", "min": 2, "max": tune_epochs+1},
          "scheduled_lr_reduce_patience": {"type": "integer", "min": 2, "max": tune_epochs+1},
          "early_stopping_patience": {"type": "integer", "min": 2, "max": tune_epochs+1},
          "decay": {"type": "float", "min": 0.00000001, "max": 0.5, "scalingType": "loguniform"},
#           "ignore_layers_values": {"type": "categorical", "values": ["attention", "batchnorm", "bert_layer"]},
          "sampling_distr": {"type": "categorical", "values": ["exp", "uniform"]},
          "posts_per_group": {"type": "integer", "min": 10, "max": 100},
          "post_groups_per_user": {"type": "integer", "min": 1, "max": 50},
#           "posts_per_user": {"type": "integer", "min": 0, "max": 1000},
          "maxlen": {"type": "integer", "min":64, "max": 512},
      },
      "spec": {
          "metric": "loss",
          "objective": "minimize",
      },
  }
optimizer = Optimizer(config, api_key="eoBdVyznAhfg3bK9pZ58ZSXfv")

hyperparams_config = hyperparams

for experiment in optimizer.get_experiments(project_name="mental"):
    experiment.add_tag("tune")
    experiment.add_tag(dataset_type)
    
    print(hyperparams_config)
    

    data_generator_train = DataGenerator(user_level_data, subjects_split, set_type='train',
                                         seq_len=hyperparams_config["maxlen"],
                                                     sample_seqs=hyperparams_config['sample_seqs'],
                                                     sampling_distr=hyperparams_config['sampling_distr'],
                                                    posts_per_group=hyperparams_config['posts_per_group'],
                                                    post_groups_per_user=1,#hyperparams_config['post_groups_per_user'],
                                                    max_posts_per_user=hyperparams_config['posts_per_user'],
                                    hierarchical=hyperparams['hierarchical'])
    data_generator_valid = DataGenerator(user_level_data, subjects_split, set_type='valid',
                                         seq_len=hyperparams_config["maxlen"],
                                         
                                                    posts_per_group=hyperparams_config['posts_per_group'],
                                                    post_groups_per_user=hyperparams_config['post_groups_per_user'],
                                                    max_posts_per_user=None,
                                                    sample_seqs=False, 
                                                     shuffle=False,
                                                hierarchical=hyperparams['hierarchical'])
    try:
        if hyperparams['hierarchical']:
            model = build_hierarchical_model(hyperparams_config, hyperparams_features, embedding_matrix, emotions, stopword_list,
                            liwc_categories=[c for c in categories if c in writings_df.columns]
        ,
                           ignore_layer=hyperparams_config['ignore_layer'])
        else:
            model = build_model(hyperparams_config, hyperparams_features, embedding_matrix, emotions, stopword_list,
                            liwc_categories=[c for c in categories if c in writings_df.columns]
        ,
                           ignore_layer=hyperparams_config['ignore_layer'])
        model.summary()

        freeze_layer = FreezeLayer(model, patience=hyperparams['freeze_patience'], set_to=not hyperparams['trainable_embeddings'])
        weights_history = WeightsHistory()
        lr_history = LRHistory()
        reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=hyperparams['reduce_lr_factor'],
                                  patience=hyperparams['reduce_lr_patience'], min_lr=0.000001, verbose=1)
        lr_schedule = callbacks.LearningRateScheduler(lambda epoch, lr: 
                                                      lr if (epoch+1)%hyperparams['scheduled_reduce_lr_freq']!=0 else
                                                      lr*hyperparams['scheduled_reduce_lr_factor'], verbose=1)

        model, history = train_model(model, hyperparams, data_generator_train, data_generator_valid,
                           epochs=tune_epochs,
                          class_weight={0:1, 1:experiment.get_parameter('positive_class_weight')}, 
                                     start_epoch=0,
                          callback_list = [
    #                                  weights_history, 
                                           reduce_lr, 
    #                                        lr_history, 
                                           lr_schedule
                                          ],
                          model_path='models/experiment', workers=4)
    except tf.errors.ResourceExhaustedError as e:
        print(e)
        sess.close()
        sess = tf.Session(config=sess_config)
        initialize_vars(sess)

    
    loss = history.history['loss'][-1]
    
    
    # Test the model
    for param in config['parameters'].keys():    
        hyperparams_config[param] = experiment.get_parameter(param)
    if not hyperparams_config['optimizer']:
        hyperparams_config['optimizer'] = optimizers.Adam(lr=hyperparams_config['lr'], 
                                   decay=hyperparams_config['decay'])
#     hyperparams_config["ignore_layer"] = []
#     if hyperparams_config["ignore_layers_values"]:
#         hyperparams_config["ignore_layer"] = [hyperparams_config["ignore_layers_values"]]
    hyperparams_config["ignore_layer"] = ["bert_layer", "batchnorm"]
        
#     freeze_layer = FreezeLayer(model, patience=experiment.get_parameter('freeze_patience'),
#                               set_to=not experiment.get_parameter('trainable_embeddings'))
    reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                            factor=experiment.get_parameter('lr_reduce_factor'),
                                            patience=experiment.get_parameter('lr_reduce_patience'), 
                                            min_lr=0.00000001, verbose=1)
    
    # Report the loss, if not auto-logged:
    experiment.log_metric("loss", loss)